In [1]:
import psycopg2
from psycopg2.extras import execute_values
import pandas as pd
import os
import sys
import base64
from bs4 import BeautifulSoup
import re

In [10]:
import tiktoken
import PyPDF2
import re

In [2]:
# Caminho para o diretório onde o módulo conexoes.py está localizado
caminho_para_conexoes = '/Users/alexandreloureiro/bb/mefis/pln'

# Adicionando o caminho ao sys.path
if caminho_para_conexoes not in sys.path:
    sys.path.append(caminho_para_conexoes)

# Importando a função postgres do módulo conexoes
from conexoes import postgres

# Conectando ao banco de dados
conn, engine = postgres()
cursor = conn.cursor()


In [14]:
import pdfplumber

# Caminho para o arquivo PDF
pdf_path = '/Users/alexandreloureiro/bb/mefis/estudo_risco.pdf'

# Caminho para o arquivo de saída
output_path = 'relatorio.txt'

# Abrir o arquivo PDF
with pdfplumber.open(pdf_path) as pdf:
    # Inicializar uma lista para armazenar o texto de cada página
    paginas_texto = []
    
    # Iterar por todas as páginas
    for page_num, page in enumerate(pdf.pages, start=1):
        print(f'Processando página {page_num} de {len(pdf.pages)}...')
        
        # Extrair o texto da página, mantendo o layout
        texto = page.extract_text(x_tolerance=1, y_tolerance=1)
        
        if texto:
            paginas_texto.append(texto)
        else:
            print(f'Aviso: Nenhum texto encontrado na página {page_num}.')

# Unir o texto de todas as páginas
texto_completo = '\n\n'.join(paginas_texto)

# Salvar o texto no arquivo 'relatorio.txt'
with open(output_path, 'w', encoding='utf-8') as f:
    f.write(texto_completo)

print(f'\nExtração concluída. O texto foi salvo em "{output_path}".')



Processando página 1 de 66...
Processando página 2 de 66...
Processando página 3 de 66...
Processando página 4 de 66...
Processando página 5 de 66...
Processando página 6 de 66...
Processando página 7 de 66...
Processando página 8 de 66...
Processando página 9 de 66...
Processando página 10 de 66...
Processando página 11 de 66...
Processando página 12 de 66...
Processando página 13 de 66...
Processando página 14 de 66...
Processando página 15 de 66...
Processando página 16 de 66...
Processando página 17 de 66...
Processando página 18 de 66...
Processando página 19 de 66...
Processando página 20 de 66...
Processando página 21 de 66...
Processando página 22 de 66...
Processando página 23 de 66...
Processando página 24 de 66...
Processando página 25 de 66...
Processando página 26 de 66...
Processando página 27 de 66...
Processando página 28 de 66...
Processando página 29 de 66...
Processando página 30 de 66...
Processando página 31 de 66...
Processando página 32 de 66...
Processando págin

In [9]:
import PyPDF2

# Caminho para o arquivo PDF
pdf_path = '/Users/alexandreloureiro/bb/mefis/estudo_risco.pdf'

# Caminho para o arquivo de saída
output_path = 'relatorio2.txt'

# Abrir o arquivo PDF em modo de leitura binária
with open(pdf_path, 'rb') as pdf_file:
    # Criar um objeto leitor do PDF
    pdf_reader = PyPDF2.PdfReader(pdf_file)
    
    # Inicializar uma lista para armazenar o texto de cada página
    paginas_texto = []
    
    # Iterar por todas as páginas do PDF
    for page_num in range(len(pdf_reader.pages)):
        print(f'Processando página {page_num + 1} de {len(pdf_reader.pages)}...')
        # Obter a página atual
        page = pdf_reader.pages[page_num]
        # Extrair o texto da página
        texto = page.extract_text()
        if texto:
            paginas_texto.append(texto)
        else:
            print(f'Aviso: Nenhum texto encontrado na página {page_num + 1}.')

# Unir o texto de todas as páginas
texto_completo = '\n\n'.join(paginas_texto)

# Salvar o texto no arquivo 'relatorio2.txt'
with open(output_path, 'w', encoding='utf-8') as f:
    f.write(texto_completo)

print(f'\nExtração concluída. O texto foi salvo em "{output_path}".')


Processando página 1 de 66...
Processando página 2 de 66...
Processando página 3 de 66...
Processando página 4 de 66...
Processando página 5 de 66...
Processando página 6 de 66...
Processando página 7 de 66...
Processando página 8 de 66...
Processando página 9 de 66...
Processando página 10 de 66...
Processando página 11 de 66...
Processando página 12 de 66...
Processando página 13 de 66...
Processando página 14 de 66...
Processando página 15 de 66...
Processando página 16 de 66...
Processando página 17 de 66...
Processando página 18 de 66...
Processando página 19 de 66...
Processando página 20 de 66...
Processando página 21 de 66...
Processando página 22 de 66...
Processando página 23 de 66...
Processando página 24 de 66...
Processando página 25 de 66...
Processando página 26 de 66...
Processando página 27 de 66...
Processando página 28 de 66...
Processando página 29 de 66...
Processando página 30 de 66...
Processando página 31 de 66...
Processando página 32 de 66...
Processando págin

In [4]:
texto_completo

' \nAnexo 1 da DDC Diris 89044 , de 17.12.202 1. \n____________________________________________________________________________________ _ \n \n_____________________________________________________________________________________  \n \nEstudo Técnico de Identificação de Riscos                                                                                                             1 #Confidencial  \n  \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n  Estudo Técnico de  \nIdentificação de Riscos  \n \n \n \n \n \n \nDiretoria Gestão de Riscos  \nGerência Integração de Riscos  \n \n \nAnexo 1 da DDC Diris 89044 , de 17.12.202 1. \n____________________________________________________________________________________ _ \n \n_____________________________________________________________________________________  \n \nEstudo Técnico de Identificação de Riscos                                                                                 

In [2]:
from pdfminer.high_level import extract_text
import re
import os


# Lista de caminhos para os arquivos PDF
pdf_files = [
    '/Users/alexandreloureiro/bb/mefis/estudo_risco.pdf',
    '/Users/alexandreloureiro/bb/mefis/estrategia.pdf',
    '/Users/alexandreloureiro/bb/mefis/notas/ecbb.pdf',
    '/Users/alexandreloureiro/bb/mefis/notas/mrld_5.pdf',
]

for pdf_path in pdf_files:
    # Verificar se o arquivo existe
    if not os.path.isfile(pdf_path):
        print(f'Arquivo não encontrado: {pdf_path}')
        continue

    # Obter o nome base do arquivo (sem o caminho completo)
    base_name = os.path.basename(pdf_path)
    # Remover a extensão '.pdf' do nome base
    file_name_no_ext = os.path.splitext(base_name)[0]
    # Criar o nome do arquivo de saída
    output_path = f'rel_temp_{file_name_no_ext}.txt'

    print(f'Processando o arquivo: {pdf_path}')
    # Extrair todo o texto do PDF
    texto_completo = extract_text(pdf_path)

    # Remover múltiplas quebras de linha consecutivas
    texto_limpo = re.sub(r'\n\s*\n+', '\n\n', texto_completo)

    # Remover múltiplos espaços dentro das linhas
    texto_limpo = re.sub(r'[ ]{2,}', ' ', texto_limpo)

    # Salvar o texto no arquivo de saída
    with open(output_path, 'w', encoding='utf-8') as f:
        f.write(texto_limpo)

    print(f'Extração concluída para "{pdf_path}". O texto foi salvo em "{output_path}".\n')



Processando o arquivo: /Users/alexandreloureiro/bb/mefis/estudo_risco.pdf
Extração concluída para "/Users/alexandreloureiro/bb/mefis/estudo_risco.pdf". O texto foi salvo em "rel_temp_estudo_risco.txt".

Processando o arquivo: /Users/alexandreloureiro/bb/mefis/estrategia.pdf
Extração concluída para "/Users/alexandreloureiro/bb/mefis/estrategia.pdf". O texto foi salvo em "rel_temp_estrategia.txt".

Processando o arquivo: /Users/alexandreloureiro/bb/mefis/notas/ecbb.pdf
Extração concluída para "/Users/alexandreloureiro/bb/mefis/notas/ecbb.pdf". O texto foi salvo em "rel_temp_ecbb.txt".

Processando o arquivo: /Users/alexandreloureiro/bb/mefis/notas/mrld_5.pdf
Extração concluída para "/Users/alexandreloureiro/bb/mefis/notas/mrld_5.pdf". O texto foi salvo em "rel_temp_mrld_5.txt".



In [13]:
from pdfminer.high_level import extract_text
import re
import os
import pandas as pd

# Lista de caminhos para os arquivos PDF
pdf_files = [
    '/Users/arquivo.pdf',
    '/Users/arquivo.pdf',
    '/Users/arquivo.pdf',
    '/Users/arquivo.pdf',
]

# Definir o modelo e obter o codificador apropriado
modelo = 'gpt-4'
try:
    encoding = tiktoken.encoding_for_model(modelo)
except KeyError:
    print(f'Modelo "{modelo}" não encontrado. Usando codificação padrão "cl100k_base".')
    encoding = tiktoken.get_encoding('cl100k_base')

# Lista para armazenar os dados
dados = []

for pdf_path in pdf_files:
    # Verificar se o arquivo existe
    if not os.path.isfile(pdf_path):
        print(f'Arquivo não encontrado: {pdf_path}')
        continue

    # Obter o nome base do arquivo (sem o caminho completo)
    base_name = os.path.basename(pdf_path)
    # Remover a extensão '.pdf' do nome base
    file_name_no_ext = os.path.splitext(base_name)[0]

    print(f'Processando o arquivo: {pdf_path}')
    # Extrair todo o texto do PDF
    texto_completo = extract_text(pdf_path)

    # Remover múltiplas quebras de linha consecutivas
    texto_limpo = re.sub(r'\n\s*\n+', '\n\n', texto_completo)

    # Remover múltiplos espaços dentro das linhas
    texto_limpo = re.sub(r'[ ]{2,}', ' ', texto_limpo)

    # Remover espaços em branco no início e no fim das linhas
    linhas = texto_limpo.split('\n')
    linhas = [linha.strip() for linha in linhas]

    # Remover linhas completamente vazias
    linhas = [linha for linha in linhas if linha != '']

    # Reunir as linhas em um único texto
    texto_final = '\n'.join(linhas)

    # Calcular o número de tokens usando tiktoken
    tokens = encoding.encode(texto_final)
    numero_de_tokens = len(tokens)

    # Adicionar os dados à lista
    dados.append({
        'path': pdf_path,
        'filename': base_name,
        'extracted_text': texto_final,
        'numero_de_tokens': numero_de_tokens
    })

    print(f'Extração concluída para "{pdf_path}". Número de tokens: {numero_de_tokens}\n')

# Criar o dataframe
df = pd.DataFrame(dados, columns=['path', 'filename', 'extracted_text', 'numero_de_tokens'])

# Exibir o dataframe
print(df)

# Opcional: Salvar o dataframe em um arquivo CSV ou Excel
# df.to_csv('textos_extraidos_com_tokens.csv', index=False)
# df.to_excel('textos_extraidos_com_tokens.xlsx', index=False)





Processando o arquivo: /Users/alexandreloureiro/bb/mefis/estudo_risco.pdf
Extração concluída para "/Users/alexandreloureiro/bb/mefis/estudo_risco.pdf". Número de tokens: 49846

Processando o arquivo: /Users/alexandreloureiro/bb/mefis/estrategia.pdf
Extração concluída para "/Users/alexandreloureiro/bb/mefis/estrategia.pdf". Número de tokens: 8981

Processando o arquivo: /Users/alexandreloureiro/bb/mefis/notas/ecbb.pdf
Extração concluída para "/Users/alexandreloureiro/bb/mefis/notas/ecbb.pdf". Número de tokens: 0

Processando o arquivo: /Users/alexandreloureiro/bb/mefis/notas/mrld_5.pdf
Extração concluída para "/Users/alexandreloureiro/bb/mefis/notas/mrld_5.pdf". Número de tokens: 3554

                                                path          filename  \
0  /Users/alexandreloureiro/bb/mefis/estudo_risco...  estudo_risco.pdf   
1   /Users/alexandreloureiro/bb/mefis/estrategia.pdf    estrategia.pdf   
2   /Users/alexandreloureiro/bb/mefis/notas/ecbb.pdf          ecbb.pdf   
3  /Users/

In [14]:
df

,path,filename,extracted_text,numero_de_tokens
0,/Users/alexandreloureiro/bb/mefis/estudo_risco...,estudo_risco.pdf,"Anexo 1 da DDC Diris 89044, de 17.12.2021.\n__...",49846
1,/Users/alexandreloureiro/bb/mefis/estrategia.pdf,estrategia.pdf,#interna\nFormulação\nEstratégica\n2023-2027\n...,8981
2,/Users/alexandreloureiro/bb/mefis/notas/ecbb.pdf,ecbb.pdf,,0
3,/Users/alexandreloureiro/bb/mefis/notas/mrld_5...,mrld_5.pdf,"Auditoria Interna – XXXXX\nBrasília (DF), 06.0...",3554


In [19]:
modelo = 'gpt-4'
encoding = tiktoken.encoding_for_model(modelo)

In [20]:
tokens = encoding.encode(texto_limpo)

In [21]:
c

In [22]:
print(f'O número de tokens para o modelo {modelo} é: {numero_de_tokens}')

O número de tokens para o modelo gpt-4 é: 50512


In [12]:
from pdfminer.high_level import extract_text
from pdfminer.pdfparser import PDFSyntaxError
import re
import os
from datetime import datetime

def limpar_texto(texto):
    """
    Função para limpar e formatar o texto extraído do PDF
    """
    # Remove caracteres especiais e formata o texto
    texto = re.sub(r'\f', '', texto)  # Remove form feeds
    texto = re.sub(r'[ \t]+', ' ', texto)  # Remove múltiplos espaços e tabs
    texto = re.sub(r'\n\s*\n+', '\n\n', texto)  # Remove múltiplas quebras de linha
    texto = re.sub(r'_{3,}', '_____', texto)  # Padroniza linhas de underscore
    texto = re.sub(r'\.{3,}', '...', texto)  # Padroniza reticências
    
    # Remove espaços antes de pontuação
    texto = re.sub(r'\s+([.,;!?])', r'\1', texto)
    
    # Remove linhas que contêm apenas números de página
    texto = re.sub(r'^\s*\d+\s*$', '', texto, flags=re.MULTILINE)
    
    # Remove espaços no início e fim de cada linha
    linhas = [linha.strip() for linha in texto.split('\n')]
    texto = '\n'.join(linhas)
    
    # Remove linhas vazias consecutivas mantendo no máximo uma
    texto = re.sub(r'\n{3,}', '\n\n', texto)
    
    return texto.strip()

def extrair_texto_pdf(caminho_pdf, caminho_saida):
    """
    Extrai e processa texto de um arquivo PDF
    """
    try:
        # Verifica se o arquivo existe
        if not os.path.exists(caminho_pdf):
            raise FileNotFoundError(f"O arquivo '{caminho_pdf}' não foi encontrado.")
            
        # Cria backup do arquivo de saída se ele já existir
        if os.path.exists(caminho_saida):
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            backup_path = f"{caminho_saida}.{timestamp}.bak"
            os.rename(caminho_saida, backup_path)
            print(f"Backup do arquivo anterior criado: {backup_path}")
        
        print("Extraindo texto do PDF...")
        texto_completo = extract_text(caminho_pdf)
        
        print("Limpando e formatando o texto...")
        texto_limpo = limpar_texto(texto_completo)
        
        # Adiciona informações do processamento
        cabecalho = f"""# Texto extraído do arquivo: {os.path.basename(caminho_pdf)}
# Data de extração: {datetime.now().strftime("%Y-%m-%d %H:%M:%S")}
# Processado com pdfminer.six
{'='*80}\n\n"""
        
        texto_final = cabecalho + texto_limpo
        
        # Salva o texto processado
        with open(caminho_saida, 'w', encoding='utf-8') as f:
            f.write(texto_final)
            
        # Estatísticas do processamento
        total_caracteres = len(texto_final)
        total_palavras = len(texto_final.split())
        total_linhas = len(texto_final.splitlines())
        
        print(f"\nProcessamento concluído com sucesso!")
        print(f"Arquivo salvo em: {caminho_saida}")
        print(f"\nEstatísticas:")
        print(f"- Total de caracteres: {total_caracteres:,}")
        print(f"- Total de palavras: {total_palavras:,}")
        print(f"- Total de linhas: {total_linhas:,}")
        
    except PDFSyntaxError:
        print("Erro: O arquivo PDF está corrompido ou não pode ser lido.")
    except PermissionError:
        print(f"Erro: Sem permissão para acessar ou criar o arquivo '{caminho_saida}'")
    except Exception as e:
        print(f"Erro inesperado: {str(e)}")

# Caminhos dos arquivos
pdf_path = '/Users/alexandreloureiro/bb/mefis/estudo_risco.pdf'
output_path = 'relatorio3.txt'

# Executa a extração
extrair_texto_pdf(pdf_path, output_path)

Backup do arquivo anterior criado: relatorio3.txt.20241102_182505.bak
Extraindo texto do PDF...
Limpando e formatando o texto...

Processamento concluído com sucesso!
Arquivo salvo em: relatorio3.txt

Estatísticas:
- Total de caracteres: 147,379
- Total de palavras: 19,825
- Total de linhas: 5,298


In [5]:
# Importando as bibliotecas necessárias
import PyPDF2
import re

def limpar_texto(texto):
    """
    Limpa o texto removendo linhas em branco extras e espaços desnecessários.
    
    Args:
        texto (str): Texto a ser limpo
        
    Returns:
        str: Texto limpo
    """
    # Remove caracteres especiais e espaços extras
    texto = re.sub(r'_+', '_', texto)  # Substitui múltiplos underscores por um único
    texto = re.sub(r'\s*\n\s*\n\s*', '\n\n', texto)  # Substitui múltiplas linhas em branco por uma única
    texto = re.sub(r'\s+', ' ', texto)  # Substitui múltiplos espaços por um único
    texto = texto.strip()  # Remove espaços no início e fim
    
    # Divide o texto em linhas
    linhas = texto.split('\n')
    
    # Remove linhas que só contêm espaços ou caracteres especiais
    linhas = [linha.strip() for linha in linhas if linha.strip() and not linha.strip().startswith('_' * 10)]
    
    # Junta as linhas de volta com uma quebra de linha dupla
    return '\n\n'.join(linhas)

def extrair_texto_pdf(caminho_pdf):
    """
    Extrai texto de um arquivo PDF e limpa formatação extra.
    
    Args:
        caminho_pdf (str): Caminho para o arquivo PDF
        
    Returns:
        str: Texto extraído e limpo do PDF
    """
    with open(caminho_pdf, 'rb') as arquivo:
        leitor_pdf = PyPDF2.PdfReader(arquivo)
        texto_completo = ""
        
        print(f"Total de páginas no PDF: {len(leitor_pdf.pages)}")
        
        for num_pagina, pagina in enumerate(leitor_pdf.pages, 1):
            # Extrai o texto da página
            texto_pagina = pagina.extract_text()
            
            # Limpa o texto da página
            texto_pagina_limpo = limpar_texto(texto_pagina)
            
            # Adiciona cabeçalho da página e o texto limpo
            texto_completo += f"\n=== Página {num_pagina} ===\n\n"
            texto_completo += texto_pagina_limpo + "\n"
        
        # Limpa novamente o texto completo para garantir consistência
        texto_completo = limpar_texto(texto_completo)
        return texto_completo





In [7]:
# Extrair e limpar o texto
texto_extraido = extrair_texto_pdf(caminho_do_pdf)

# Mostrar uma amostra do texto limpo
print("\nAmostra do texto limpo:")
print(texto_extraido[:500])

# Salvar o texto limpo em um arquivo
with open("texto_extraido_limpo.txt", "w", encoding="utf-8") as arquivo_texto:
    arquivo_texto.write(texto_extraido)

# Para visualizar de forma mais organizada no Jupyter
from IPython.display import display, Markdown
display(Markdown(texto_extraido))

Total de páginas no PDF: 66

Amostra do texto limpo:


=== Página 1 ===

 
Anexo 1 da DDC Diris 89044 , de 17.12.202 1. 
____________________________________________________________________________________ _ 
 
_____________________________________________________________________________________  
 
Estudo Técnico de Identificação de Riscos                                                                                                             1 #Confidencial  
  
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
  Estudo Técn




=== Página 1 ===

 
Anexo 1 da DDC Diris 89044 , de 17.12.202 1. 
____________________________________________________________________________________ _ 
 
_____________________________________________________________________________________  
 
Estudo Técnico de Identificação de Riscos                                                                                                             1 #Confidencial  
  
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
  Estudo Técnico de  
Identificação de Riscos  
 
 
 
 
 
 
Diretoria Gestão de Riscos  
Gerência Integração de Riscos  
 

=== Página 2 ===

 
Anexo 1 da DDC Diris 89044 , de 17.12.202 1. 
____________________________________________________________________________________ _ 
 
_____________________________________________________________________________________  
 
Estudo Técnico de Identificação de Riscos                                                                                                             2 #Confidencial  
©202 1. Banco do Brasil S.A. – Diretoria Gestão de Riscos.  
 
4a Edição  
 
Não é permitida a reprodução total ou parcial deste documento sem a prévia autorização 
da Diretoria de Gestão de Riscos.  
 
Equipe Responsável:  
 
Glaucio Nery Henrique   
Gerente Executiv o   
 
Mauricio Sigrist de Melo  
Gerente de Soluções    
 
Adriana Rodrigues Oliveira  
Assessora I UE  
 
Daniela Marques Teixeira  
Assessora I UE  
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 

=== Página 3 ===

 
Anexo 1 da DDC Diris 89044 , de 17.12.202 1. 
____________________________________________________________________________________ _ 
 
_____________________________________________________________________________________  
 
Estudo Técnico de Identificação de Riscos                                                                                                             3 #Confidencial  
 
 
Sumário  
1. Introdução  ................................ ................................ ................................ ................................ ................  6 
2. Conceitos Gerais  ................................ ................................ ................................ ................................ ..... 7 
2.1 RISCO  ................................ ................................ ................................ ................................ ...... 7 
2.2 EVENTO DE RISCO  ................................ ................................ ................................ ....................  8 
2.3 FATOR DE RISCO  ................................ ................................ ................................ ......................  8 
2.4 IMPACTO DO RISCO  ................................ ................................ ................................ ................  10 
2.5 PROBABILIDADE DO RISCO  ................................ ................................ ................................ ..... 10 
2.6 VISÃO CONSOLIDADA DOS CONCEITOS  ................................ ................................ ..................  10 
3. Referências do Modelo de Identificação dos Riscos  ................................ ................................ ............  12 
3.1 IDENTIFICAÇÃO DOS RISCOS  ................................ ................................ ................................ ... 12 
3.2 CLASSIFICAÇÃO  DOS RISCOS  ................................ ................................ ................................ . 12 
3.3 DEFINIÇÃO DO GRAU DE RELEVÂNCIA  DOS RISCOS  ................................ ................................  14 
3.3.1  CRITÉRIOS UTILIZADOS PELA CONSULTORIA ESPECIALIZADA ................................ ...................  15 
3.4 TRATAMENTO DOS RISCOS  ................................ ................................ ................................ ..... 17 
4. Modelo de Identificação dos Riscos do Conglomerado Prudencial Banco do Brasil (CPBB)  ..............  20 
4.1 IDENTIFICAÇÃO DOS RISCOS  ................................ ................................ ................................ ... 20 
4.1.1  TAXONOMIA CORPORATIVA DOS RISCOS  ................................ ................................ .............  21 
4.2 CLASSIFICAÇÃO  DOS RISCOS  ................................ ................................ ................................ . 21 
4.3 DEFINIÇÃO DO GRAU DE RELEVÂNCIA  DOS RISCOS  ................................ ................................  22 
4.3.1  CRITÉRIOS QUANTITATIVOS  ................................ ................................ ................................ . 23 
4.3.2  CRITÉRIOS QUALITATIVOS  ................................ ................................ ................................ ... 25 
4.3.3  MATRIZ DE RELEVÂNCIA DOS RISCOS  ................................ ................................ .................  29 
4.3.4  ANÁLISE DOS RESULTADOS E  OVERRIDE ................................ ................................ ..............  30 
4.3.5  CONJUNTO CORPORATIVO DE RISCOS RELEVANTES  ................................ ...........................  31 
4.3.6  MAPA DOS RISCOS  ................................ ................................ ................................ ..............  31 
4.4 TRATAMEN TO DOS RISCOS  ................................ ................................ ................................ ..... 31 
4.4.1  ETAPAS DA IMPLEMENTAÇÃO DA GESTÃO DOS RISCOS  ................................ .......................  31 
4.4.2  ATIVIDADES DE GESTÃO DOS RISCOS ................................ ................................ ..................  32 
4.4.2.1  APETITE E TOLERÂNCIA A RISCOS  ................................ ................................ ....................  34 
4.5 PLANEJAMENTO ESTRATÉGICO E GESTÃO DE RISCOS  ................................ ...........................  34 
5. Modelo de Identificação dos Riscos aplicado no CPBB  ................................ ................................ ....... 36 
5.1 IDENTIFICAÇÃO DOS RISCOS  ................................ ................................ ................................ ... 37 
5.1.1  DIRECIONADORES ESTRATÉGICOS  ................................ ................................ ..............................  37 
5.1.2  ENTIDADES LIGADAS BANCO DO BRASIL (ELBB)  ................................ ................................ ..... 37 
5.1.3  FORMULÁRIO DE REFERÊNCIA  ................................ ................................ ................................ ..... 38 
5.1.4  BASE DE PROCESSOS (BP)  ................................ ................................ ................................ ..........  40 
5.1.5  DIRETRIZES REGULATÓRIAS  ................................ ................................ ................................ ........  41 
5.1.6  PREOCUPAÇÕES DO MERCADO  ................................ ................................ ................................ ... 43 
5.1.7 BENCHMARK  ................................ ................................ ................................ ................................ .. 44 

=== Página 4 ===

 
Anexo 1 da DDC Diris 89044 , de 17.12.202 1. 
____________________________________________________________________________________ _ 
 
_____________________________________________________________________________________  
 
Estudo Técnico de Identificação de Riscos                                                                                                             4 #Confidencial  
5.1.8  REVISÃO DA TAXONOMIA CORPORATIVA DE RISCOS  ................................ ................................  46 
5.2 AVALIAÇÃO DOS RISCOS  ................................ ................................ ................................ ........  51 
5.2.1  CRITÉRIOS QUANTITATIVOS  ................................ ................................ ................................ . 51 
5.2.2  CRITÉRIOS QUALITATIVOS  ................................ ................................ ................................ ... 52 
5.2.3  MATRIZ DE RELEVÂNCIA DOS RISCOS  ................................ ................................ ..................  57 
5.2.4  ANÁLISE DOS RESULTADOS E  OVERRIDE  ................................ ................................ ..............  59 
5.2.5  MATRIZ DE RELEVÂNCIA DOS RISCOS APÓS OVERRIDE  ................................ ........................  60 
5.2.6  CONJUNTO CORPORATIVO DE RISCOS RELEVANTES  ................................ ...........................  61 
5.2.7  MAPA DOS RISCOS  ................................ ................................ ................................ ..............  62 
6. Bibliografia  ................................ ................................ ................................ ................................ ..............  64 
 
 
 
 
Índice de Quadros  
 
Quadro 1. Fatores de Risco  ................................ ................................ ................................ ...........................  9 
Quadro 2. Escala de Severidade  ................................ ................................ ................................ .................  15 
Quadro 3. Escala de Frequência  ................................ ................................ ................................ .................  15 
Quadro 4. Inventário de Riscos (resultado ilustrativo)  ................................ ................................ ...............  16 
Quadro 5. Exemplo de Mapa de Riscos de um Banco  ................................ ................................ ................  16 
Quadro 6. Régua Quantitativo  ................................ ................................ ................................ ....................  24 
Quadro 7. Régua e Fatores para Av. Qualitativa do Risco  ................................ ................................ ..........  26 
Quadro 8. Régua e Perdas Indiretas para Av. Qualitativa do Risco  ................................ ............................  28 
Quadro 9. Régu a Qualitativo Fator e Perda Indireta  ................................ ................................ ..................  29 
Quadro 10. Etapas da Implementação e da Revisão dos Processos de Gestão dos Riscos ........................  31 
Quadro 11. Atividades de Gestão dos Riscos  ................................ ................................ .............................  33 
Quadro 12. Inventário de Riscos X Riscos Relevantes  ................................ ................................ ................  36 
Quadro 13. ELBB X Taxonomia (Atual)  ................................ ................................ ................................ .......  37 
Quadro 14. Fatores Formulário X Taxonomia (Atual)  ................................ ................................ ................  39 
Quadro 15. Riscos BP  ................................ ................................ ................................ ................................ .. 41 
Quadro 16. Riscos Regulador X Taxonomia (Atual)  ................................ ................................ ....................  41 
Quadro 17. Agenda Bacen Competitividade X Taxonomia (Atual)  ................................ .............................  42 
Quadro 18. Pesquisa ORX – Fatores de Risco Emergentes  ................................ ................................ .........  43 
Quadro 19. Fatores de Risco do Setor Finance iro X Taxonomia (Atual)  ................................ ....................  44 
Quadro 20. Riscos Bancos X Taxonomia (Atual)  ................................ ................................ .........................  45 
Quadro 21. Resultado Qualitativo Fator (Régua conforme Quadro 7, pág. 26)  ................................ .........  53 
Quadro 22. Resultado Qualitativo Perda Indireta (Régua conforme Quadro 8, pág. 28)  ..........................  55 
Quadro 23. Resultado Impacto Qualitativo  ................................ ................................ ................................  56 
Quadro 24. Resultado Impacto Qualitativo Comparativo  ................................ ................................ ..........  57 
Quadro 25. Resultado Relevância Comparativo  ................................ ................................ .........................  59 
Quadro 26. Conjunto Corporativo de Riscos Relevantes  ................................ ................................ ...........  61 
 
 
 

=== Página 5 ===

 
Anexo 1 da DDC Diris 89044 , de 17.12.202 1. 
____________________________________________________________________________________ _ 
 
_____________________________________________________________________________________  
 
Estudo Técnico de Identificação de Riscos                                                                                                             5 #Confidencial  
Índice de Figuras  
 
Figura 1 . Visão consolidada de conceitos  ................................ ................................ ................................ ... 11 
Figura 2 . Exemplo Ilustrativo de Mapa de Avaliação dos Riscos  ................................ ................................  14 
Figura 3 . Matriz de Relevância de Riscos (resultado ilustrativo)  ................................ ................................  17 
Figura 4 . Matriz de Riscos e os 4T s de Gestão  ................................ ................................ ............................  19 
Figura 5.  Processo de identificação dos riscos  ................................ ................................ ...........................  20 
Figura 6.  Processo de Avaliação dos Riscos  ................................ ................................ ................................  23 
Figura 7.  Modelo de Avaliação dos Riscos  ................................ ................................ ................................ . 23 
Figura 8.  Modelo da Matriz de Relevância dos Riscos  ................................ ................................ ...............  30 
Figura 9.  Apetite e Tolerância a Riscos  ................................ ................................ ................................ .......  34 
Figura 10.  Gestão da Estratégia e Riscos  ................................ ................................ ................................ .... 35 
Figura  11.  Taxonomia Corporativa de Riscos  ................................ ................................ .............................  50 
Figura 12.  Matriz de Relevância dos Riscos  ................................ ................................ ................................  58 
Figura 13.  Matriz de Relevância dos Riscos após override ................................ ................................ .........  60 
Figura 13.  Mapa dos Riscos  ................................ ................................ ................................ ........................  62 
 
 
Índice de Tabelas  
 
Tabela 1. Resultado Quantitativo  ................................ ................................ ................................ ................  51 
Tabela 2. Resultado Quanti tativo Comparativo  ................................ ................................ ..........................  51 
 
 
 
  

=== Página 6 ===

 
Anexo 1 da DDC Diris 89044 , de 17.12.202 1. 
____________________________________________________________________________________ _ 
 
_____________________________________________________________________________________  
 
Estudo Técnico de Identificação de Riscos                                                                                                             6 #Confidencial  
1. Introdução  
 
Em um contexto de rápida transformação, advinda do acesso facilitado à informação e 
tecnologia, faz -se necessário que as empresas sejam capazes de se reinventar e 
aprimorar  continuamente seus processos, destacando -se no ambiente competitivo, de 
forma a entregar valor ao cliente e manter a sustentabilidade dos seus negócios.  
 
A mudança constante na estratégia de negócio requer alinhamento, na mesma via, das 
atividades necessá rias para atingimento dos objetivos definidos. A estratégia e sua 
operacionalização estão sujeitas a  riscos que precisam ser identificados para que seja 
possível avaliar seu impacto, realizar gestão adequada e contribuir para o alcance d os 
objetivos  projet ados.  
 
No processo de identificação dos riscos, são considerados insumos externos, tais como 
diretrizes regulatórias, preocupações relevantes da indústria financeira mundial e 
benchmark  com outras instituições. Como insumos internos, são avaliados os  riscos 
incorridos pelas entidades ligadas, os fatores de risco divulgados no Formulário de 
Referên cia do BB e, principalmente, os direcionadores estratégicos. Os riscos 
identificados têm sua relevância definida com base em critérios quantitativos e 
qualitativos .  
 
Para os riscos relevantes, são estabelecidas etapas da implementação de estrutura e 
processos de gestão, com as atividades mínimas a serem consideradas para tratamento 
dos riscos.  
 
Este estudo tem por objetivo a identificação dos riscos incorridos pelo Co nglomerado 
Prudencial Banco do Brasil (CPBB) e definição dos riscos relevantes, para os quais deve 
haver processo de gestão estruturado. É composto, além deste capítulo introdutório:   
 
a) do capítulo 2, que traz conceitos gerais de gestão de riscos  e formam a base para 
o desenvolvimento e implementação do mod elo de identificação dos riscos;  
 
b) do capítulo 3, que apresenta o referencial teórico para a identificação e a avaliação 
da relevância dos riscos;  
 
c) do capítulo 4, que demonstra o modelo de identificação dos riscos e avaliação da 
relevância  dos riscos do CPBB ; 
 
d) do capítulo 5, onde são demonstrados os detalhes do processo de identificação 
dos riscos, a avaliação da relevância e as conclusões a serem aprov adas pela 
instância competente no Banco.  
 
 
 
 
 
 
 
 

=== Página 7 ===

 
Anexo 1 da DDC Diris 89044 , de 17.12.202 1. 
____________________________________________________________________________________ _ 
 
_____________________________________________________________________________________  
 
Estudo Técnico de Identificação de Riscos                                                                                                             7 #Confidencial  
2. Conceitos Gerais  
 
2.1 Risco  
 
O livro Ecological Economics: Concepts and Methods1 traz o conceito de risco como um 
evento futuro identificado, ao qual é possível associar uma distribuição de probabilidades 
de ocorrência. A incerteza é apresentada como um evento futuro identificado, ao qual 
não é possível associar uma distribuição de pr obabilidades de ocorrência. Ainda, define 
ignorância como o conjunto de eventos futuros que, no momento da análise, não poderão 
sequer ser identificados, muito menos quantificados.  
 
A Associação Brasileira de Norma s Técnica s (ABNT)2 define risco como efei to da 
incerteza nos objetivos. Um efeito é um desvio em relação ao esperado. Pode ser 
positivo, negativo ou ambos, e pode abordar, criar ou resultar em oportunidades e 
ameaças. Objetivos podem possuir diferentes aspectos e categorias, e podem ser 
aplicados  em diferentes níveis. A incerteza3 é o estado, mesmo parcial, de deficiência de 
informações relacionadas a um evento, sua compreensão, conhecimento, consequência 
ou probabilidade. O risco é normalmente  expresso em termos de  fontes de risco,  evento s 
poten ciais, suas consequências e suas  probabilidade s. 
 
Considerando a necessidade de identificação e gestão dos riscos, a definição trazida 
pela ABNT torna -se mais alinhada aos processos e objetivo s pretendido s, dado que o 
risco provém da incerteza da ocorrência de um evento sobre o objetivo, sendo essa 
incerteza avaliada, conforme informações disponíveis, em termos quantitativos ou 
qualitativos.  
 
Assim, risco pode ser definido como a possibilidade de um resultado realizado 
(observado) ser diferente do resultado esperado. A gestão de riscos4 contempla  
atividades coordenadas para dirigir e controlar uma organização no que se refere a 
riscos.  
 
A empresa de Consultoria Oliver Wyman5 apresenta algumas definições de riscos 
emergentes encontradas no mercado, sendo uma delas: “Riscos emergentes e 
oportunidades são definidos como possíveis eventos que podem ocorrer, mas não são 
completamente compreendidos e tem potencial para significante mente impactar a 
instituição no futuro”.  
 
Analisando o conceito de riscos emergentes apresentado, assim como os conceitos de 
incerteza e risco da ABNT, conclui -se que risco emergente pode ser definido como a 
possibilidade de ocorrência de eventos cujas inf ormações ainda não são suficientemente 
compreendidas, sendo sua avaliação realizada com base em dados essencialmente 
qualitativos, que apontem perspectivas futuras de impacto significativo nos resultados.  
 
1 M. Faber, R. Manstetten e J. Proops , Ecological Economics: Concepts and Methods , 1996, pp. 209 -211. 
2 ABNT NBR ISO Guia 31000 de 03/2018, Gestão de Riscos – Diretrizes, item 3.1, p. 1.  
3ABNT NBR ISO Guia 31000 de 11/2009, Gestão de Riscos – Princípios e Diretrizes, item 2.1, p. 1.  
4ABNT NBR ISO Guia 31000 de 03/2018, Gestão de Riscos –Diretrizes, item 3.2, p. 1.  
5 Relatório técnico referente à identificação e avaliação dos riscos corporativos (relevantes e emergentes), Ponto de Homologaç ão 
nº 05 de 06/2019.  

=== Página 8 ===

 
Anexo 1 da DDC Diris 89044 , de 17.12.202 1. 
____________________________________________________________________________________ _ 
 
_____________________________________________________________________________________  
 
Estudo Técnico de Identificação de Riscos                                                                                                             8 #Confidencial  
2.2 Evento de Risco  
 
O Guia Aplicado para Modelagem de Processos6, que utiliza a Notação de Modelagem 
de Processos de Negócio (BPMN), conceitua evento como algo que ocorre durante o 
curso de um processo, afetando o seu fluxo, e que geralmente possui uma causa ou um 
impacto.  
 
A ABNT7 define event o como a ocorrência ou mudança de um conjunto  específico de 
circunstâncias, podendo consistir em uma ou mais ocorrências e ter várias causas  e 
várias consequências . Um evento pode também ser algo que é esperado, mas não 
acontece, ou algo que não é esperado , mas acontece. Um evento pode ser uma fonte 
de risco.  
 
Em alinhamento à prática de processos e visão negocial, podemos definir evento de risco 
como a ocorrência observada  durante a execução de um objetivo definido, podendo 
causar impacto inesperado no resultado.     
 
2.3 Fator de Risco  
 
A Comissão de Valores Mobiliários (CVM)8 orienta que, na elaboração do Formulário de 
Referência, devem ser expostos, em ordem de relevância, quaisquer fatores de risco que 
possam influenciar a decisão de investimento, em especial aqueles relacionados com o 
emissor e seu controlador, direto ou indireto, ou grupo de controle, seus acionistas, suas 
controladas e coligadas, seus fornecedores, seus clientes, com os setores da economia 
nos quais o emissor atue e sua respectiva regulação, com os países estrangeiros onde 
o emissor atua e a questões socioambientais. Essa lista de fatores é exemplificativa, 
podendo deixar de ser utilizados ou incluídos ou tros que sejam aplicáveis ao emissor.  
 
Com relação aos riscos relacionados a seus acionistas, o emissor deverá apresentar os 
riscos aos quais a Companhia está exposta em razão de seus acionistas, ou seja, 
aqueles em que a fonte do risco é o acionista.  
 
Nos comentários do Formulário de Referência, deverão ser explicitados os fatores 
internos ou externos ao emissor que embasam a opinião emitida, podendo ser também 
comentadas as eventuais medidas adotadas pelo emissor para reduzir a exposição aos 
riscos.  
 
O Instituto Brasileiro de Governança Corporativa (IBGC)9 demonstra, dentre algumas 
formas de categorização dos riscos, exemplo relacionado à origem dos eventos 
(externos ou internos). Os riscos externos são ocorrências associadas ao ambiente 
macroeconômico,  político, social, natural ou setorial em que a organização opera. Os 
riscos internos são eventos originados na própria estrutura da organização, pelos seus 
processos, seu quadro de pessoal ou de seu ambiente de tecnologia.  
 
6 UGP/Gepro/BPCC, Guia Apl icado para Modelagem de Processos, 02/2019, p. 50.  
7 ABNT NBR ISO Guia 31000 de 03/2018, Gestão de Riscos – Diretrizes, item 3.5, p. 2.  
8 Ofício Circular CVM/SEP/nº 02/2020, item 10.2.4, pp. 180 e 181.  
9 IBGC, Guia de Orientação para Gerenciamento de Riscos Corporativos, 2007, p. 18.  

=== Página 9 ===

 
Anexo 1 da DDC Diris 89044 , de 17.12.202 1. 
____________________________________________________________________________________ _ 
 
_____________________________________________________________________________________  
 
Estudo Técnico de Identificação de Riscos                                                                                                             9 #Confidencial  
O Conselho Monetário Nacional ( CMN)10 define risco operacional como a possibilidade 
da ocorrência de perdas resultantes de eventos externos ou de falha, deficiência ou 
inadequação de processos internos, pessoas ou sistemas, os quais se convencionou 
denominar de fatores de risco operacional no  BB e são conceituados como situações 
e/ou circunstâncias que podem levar ao aumento da probabilidade de ocorrência de um 
risco.  
 
A ABNT11 define que fonte de risco é um elemento que, individualmente ou combinado, 
tem o potencial intrínseco para dar orige m ao risco.  
 
Observa -se que fator de risco pode ser entendido como a causa, externa ou interna, de 
um evento de risco. Dessa forma, segue quadro com fatores de risco por referência 
pesquisada, bem como o modelo consolidado dos fatores a serem considerados  na 
gestão dos riscos  do CPBB : 
Quadro 1. Fatores de Risco  
 
CVM  IBGC  CMN  Modelo consolidado  
Internos  
Emissor  Processos  Processos  Processos  
Pessoas  Pessoas  Pessoas  
Ambiente de TI  Sistemas  Sistemas  
Externos  
Controlador  - - Controlador /Acionista  Acionistas  - - 
Controladas e coligadas  - - Controladas /Coligadas  
Fornecedores  - Fornecedores e 
terceirizados ( 1) Terceiros  
Clientes  - Clientes (1)  Ambiente Social  
Ambiente Cultural  
Ambiente Concorrencial  
Setores da economia  Ambiente 
macroeconômico  - Ambiente 
Político/Econômico  Político  
Regulação dos setores  Setorial  Ambiente regulatório (1)  Ambiente Legal/ Regulatório  
Países estrangeiros  - - Países Estrangeiros  
Questões socioambientais  Social  Ambiente social (1)  Fenômenos da 
Natureza/Climáticos  Natural  Desastres naturais e 
catástrofes (1)  
(1) Detalhamento dado pela IN 410.  
Fonte: Elaboração própria (Diris/ Geric ) baseada nas fontes citadas.  
 
O fator “Fornecedores”  da CVM  ou “Fornecedores e terceirizados”  do CMN  podem ser 
entendidos, no sentido amplo, como “Terceiros” tendo em vista abarcar demais 
relacionamentos não societários. No fator “Clientes”, para melhor classificação do 
conteúdo, previu -se a divisão em Ambientes Social, Cultural e Concorrencial.  
  
 
 
 
10 Resolução CMN nº 4.557 de 23/02/2017, Capítulo III, Seção VI, Art. 32, p. 17 e 18.  
11 ABNT NBR ISO Guia 310 00 de 03/2018, Gestão de Riscos – Diretrizes, item 3.4, p. 2.  

=== Página 10 ===

 
Anexo 1 da DDC Diris 89044 , de 17.12.202 1. 
____________________________________________________________________________________ _ 
 
_____________________________________________________________________________________  
 
Estudo Técnico de Identificação de Riscos                                                                                                             10 #Confidencial  
Expo sição ao Risco  
 
A ABNT define exposição como a medida em que uma organização e/ou stakeholder  
está sujeita a um evento de risco.  
 
Para o IBGC12, o grau de exposição é o efeito potencial do risco sobre a organização.  
 
Os conceitos apresentados estão bem ali nhados e apresentam a exposição aos riscos 
como a medida de vulnerabilidade da organização e/ou objetivos definidos frente a um 
evento de risco.  
 
2.4 Impacto do Risco  
 
Em relação ao impacto, a ABNT13 traz conceito correlato de consequência, referindo -se 
ao resultado de um evento sobre os objetivos, podendo ser expressa de forma qualitativa 
ou quantitativa.  
 
O IBGC14 também considera o impacto em termos qualitativos (intangível) e quantitativos 
(desemp enho econômico -financeiro).  
 
Dessa forma, impacto pode ser entendido como o resultado de um evento de risco sobre 
os objetivos definidos.  
 
2.5 Probabilidade do Risco  
 
A ABNT15 refere -se à probabilidade como a chance de algo acontecer, não importando 
se defini da, medida ou determinada , ainda que  objetiva ou subjetivamente, qualitativa ou 
quantitativamente, e se descrita utilizando -se termos gerais ou matemáticos.  
 
2.6 Visão Consolidada dos Conceitos  
 
A figura a  seguir demonstra a relação entre os conceitos apresentados:  
 
12 IBGC, Guia de Orientação para Gerenciamento de Riscos Corporativos, 2007, p. 20.  
13 ABNT NBR ISO Guia 31000 de 11/2009, Gestão de Riscos – Princípios e Diretrizes, item 2.18, p. 5.  
14 IBGC, Guia  de Orientação para Gerenciamento de Riscos Corporativos, 2007, p. 20.  
15 ABNT NBR ISO 31000 de 03/2018, Gestão de Riscos – Diretrizes, item 3.7, p. 2.  

=== Página 11 ===

 
Anexo 1 da DDC Diris 89044 , de 17.12.202 1. 
____________________________________________________________________________________ _ 
 
_____________________________________________________________________________________  
 
Estudo Técnico de Identificação de Riscos                                                                                                             11 #Confidencial  
 
Figura 1. Visão consolidada de conceitos  
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
Possibilidade de um resultado realizado (observado) ser diferente do resultado esperado.Ocorrência 
observada durante a 
realização de um 
objetivo definido, 
podendo causar 
impacto inesperado 
no resultado.    Causa, externa ou 
interna, de um 
evento de risco.Medida de 
vulnerabili -
dade da 
organização 
e/ou 
objetivos 
definidos 
frente a um 
evento de 
risco.Resultado
EsperadoObjetivos 
estratégicos
Exposição 
ao Evento
RiscoFator de Risco Evento de RiscoProcessos
Fraqueza X Força
Ameaça X Oportunidade
Resultado 
de um 
evento de 
risco sobre 
os objetivos 
definidos.Impacto do 
Evento
Chance de 
algo 
acontecer, 
não 
importando 
se definida, 
medida ou 
determinada 
qualitativa ou 
quantitativa -
mente . Probabilida -
de do Evento

=== Página 12 ===

 
Anexo 1 da DDC Diris 89044 , de 17.12.202 1. 
____________________________________________________________________________________ _ 
 
_____________________________________________________________________________________  
 
Estudo Técnico de Identificação de Riscos                                                                                                             12 #Confidencial  
3. Referências do Modelo de Identificação dos Riscos  
 
3.1 Identificação dos Riscos  
 
A ABNT16 explica que o propósito da identificação de riscos é encontrar, reconhecer e 
descrever riscos que possam ajudar ou impedir que uma organização alcance seus 
objetivos. Informações pertinentes, apropriadas e atualizadas são importantes na 
identificação de riscos.  A identificação de riscos envolve a identificação de suas fontes  
tangíveis  ou intangíveis  de risco , eventos, consequências e impactos nos objetivos . 
 
Segundo o IBGC17, a identificação dos riscos consiste na definição do conjunto de 
eventos, externos ou internos, que possam impactar os objeti vos estratégicos da 
organização, inclusive os relacionados aos ativos intangíveis. Os ativos intangíveis 
podem ser entendidos como os ativos e métodos responsáveis pela diferença entre o 
valor de mercado e o valor contábil da organização.  
 
A integração do planejamento estratégico com informações de risco proporciona 
alternativas de escolha aos tomadores de decisão, tornando possível o atingimento dos 
objetivos.  
 
Assim, o modelo de identificação dos riscos envolve a descrição dos fatores, eventos e 
potencia l impacto, tendo em vista a estratégia estabelecida.  
 
3.2 Classificação  dos Riscos  
 
Uma forma de classificar os riscos, proposta pelo IBGC18, pauta -se pela natureza dos 
riscos (estratégica, operacional ou financeira), em função das áreas da organização qu e 
são afetadas pelos eventos.  
 
Segundo o Instituto, os riscos estratégicos estão associados à tomada de decisão da alta 
administração e podem gerar perda substancial no valor econômico da organização.  
 
Os riscos financeiros (liquidez, mercado  e crédito) são aqueles associados à exposição 
das operações financeiras da organização, ou seja, é o risco de que os fluxos de caixa 
não sejam administrados efetivamente para maximizar a geração de caixa operacional, 
gerenciar os riscos e retornos específ icos das transações financeiras e captar e aplicar 
recursos financeiros de acordo com as políticas estabelecidas.  
 
Os riscos operacionais estão associados à possibilidade de ocorrência de perdas (de 
produção, ativos, clientes, receitas) resultantes de falh as, deficiências ou inadequação 
de processos internos, pessoas e sistemas, assim como de eventos externos como 
catástrofes naturais, fraudes, greves e atos terroristas. Esses riscos geralmente 
acarretam redução, degradação ou interrupção, total ou parcial,  das atividades, com 
 
16 ABNT NBR ISO 31000 de 03/2018, Gestão de Riscos – Diretrizes, item 6.4.2, p. 12.  
17 IBGC, Guia de Orient ação para Gerenciamento de Riscos Corporativos, 2007, p. 16.  
18 IBGC, Guia de Orientação para Gerenciamento de Riscos Corporativos, 2007, pp. 18 -19. 

=== Página 13 ===

 
Anexo 1 da DDC Diris 89044 , de 17.12.202 1. 
____________________________________________________________________________________ _ 
 
_____________________________________________________________________________________  
 
Estudo Técnico de Identificação de Riscos                                                                                                             13 #Confidencial  
impacto negativo na reputação, além do potencial geração de passivos contratuais, 
regulatórios e ambientais.  
 
Segundo Jorion19, os riscos podem ser agrupados e vinculados ao negócio ( business 
risk) e não vinculados ao negócio ( non busine ss risks). Business risk são aqueles riscos 
que a organização assume com o objetivo de criar uma vantagem competitiva e adicionar 
valor a os seus acionistas . Non busines s risk se refere aos riscos financeiros, 
relacionados a possíveis perdas decorrentes da atuação no  mercado financeiro, como  
os risco s de crédito, mercado  e operacional , e outros riscos, como de reputação , políticos 
e regulatórios.   
 
De acordo com o  Bacen20, os riscos podem ser divididos em  riscos de origem financeira 
(risco de mercado, crédito e liquidez) e riscos de origem não financeira (risco operacional, 
estratégico, legal etc.) , com diferentes dimensões de impacto, como financeiro, 
reputacional e estratégic o. 
 
Quanto à classificação de Paul Hopkin21, os riscos são subdivididos em estratégicos, 
táticos, operacionais e de compliance .  
 
Os riscos estratégicos têm o potencial de inviabilizar o atingimento da estratégia, sendo, 
portanto, considerados de maior impacto para o sucesso da estratégia que os riscos 
táticos e operacionais.  
 
Os riscos táticos estão associados a projetos, programas  e outros aprimoramentos 
realizados para atingimento da estratégia.  
 
Os riscos operacionais estão relacionados à habilidade da organização em manter 
processos efetivos e eficientes, visando a continuidade e monitoramento das operações 
rotineiras da empres a.  
 
Segundo o autor, o risco de compliance  é considerado uma categoria adicional aos riscos 
de estratégia, tático e operacional, devendo ser tratado em cada um dos níveis de riscos.  
 
O BIS22 separou os riscos da seguinte forma:  
a) risco analítico e de políti cas; 
b) risco financeiro : mercado, crédito e liquidez;  
c) risco operacional: sistema de TI, recursos humanos, controle operacional, 
projetos, interrupção de negócios,  conformidade  e práticas negociais , segurança  
e legal.  
 
 
 
19 Phillipe Jorion , Value at Risk: The New Benchmark for Managing Financial Risk , 3ª Edição, pp. 517 -518. 
20 Banco Central do Brasil , Gestão Integrada de Riscos , 13.09.2017 . 
21 Paul Hopkin, Fundamentals of Risk Management , 3º Edição, pp. 152 -154. 
22 Bank for International Settlements, Issues in the Governance of Central Banks - A report from the Central Bank Governance 
Group , May 2009, pp. 152.  

=== Página 14 ===

 
Anexo 1 da DDC Diris 89044 , de 17.12.202 1. 
____________________________________________________________________________________ _ 
 
_____________________________________________________________________________________  
 
Estudo Técnico de Identificação de Riscos                                                                                                             14 #Confidencial  
3.3 Definição do Grau de Relevância  dos Riscos  
 
A ABNT23 define nível de risco em função de sua magnitude, expressa em termos da 
combinação das consequências e probabilidades.  O nível de risco está present e na 
análise de riscos24, cujo  propósito é compreender a natureza do risco e  suas 
características.  
 
O IBGC25 esclarece que o modelo de gerenciamento de riscos corporativo deve buscar 
quantificar as incertezas envolvidas na fase de planejamento e projetar os resultados da 
organização em cenários alternativos de preços, condições macroeconômicas e 
operacionais. As sociando -se probabilidades aos cenários gerados, é possível quantificar 
o risco e estimar a probabilidade de que qualquer métrica de desempenho fique abaixo 
das metas orçadas em cada período.  
 
Para se definir o tratamento que será dado a determinado risco,  o IBGC26 define que o 
primeiro passo consiste em determinar o seu efeito potencial, ou seja, o grau de 
exposição da organização àquele risco. Esse grau leva em consideração dois aspectos: 
a probabilidade de ocorrência e o seu impacto (em geral medido pelo  impacto no 
desempenho econômico -financeiro do período). Deve -se incorporar também o impacto 
“intangível” à análise. A figura a seguir demonstra como pode ser feita a avaliação dos 
riscos corporativos:  
 
 
Figura 2. Exemplo Ilustrativ o de Mapa de Avaliação dos Riscos  
 
O Gráfico A, Exposição Financeira, representa o espaço no qual são expostos os eventos 
de riscos identificados em função do nível da probabilidade de ocorrência (eixo 
horizontal) e do valor do impacto do evento (eixo vert ical). Os tons denotam o grau de 
importância que se deve dar a cada um dos eventos em função da região que ocupam 
 
23 ABNT NBR ISO Guia 31000 de 11/2009, Gestão de Riscos – Princípios e Diretrizes, item 2.23, p. 6.  
24 ABNT NBR ISO 31000 de 03/2018, Gestão de Riscos –Diretrizes, item 6.4.3, p. 13.  
25 IBGC, Guia de Orientação para Gerenciamento de Riscos Corporativos, 2007, cap 2.3 Mensuração dos Riscos.  
26 IBGC, Guia de Orientação para Gerenciamento de Riscos Corporativos, 2007, cap 2. 2 Avaliação dos Riscos.  


=== Página 15 ===

 
Anexo 1 da DDC Diris 89044 , de 17.12.202 1. 
____________________________________________________________________________________ _ 
 
_____________________________________________________________________________________  
 
Estudo Técnico de Identificação de Riscos                                                                                                             15 #Confidencial  
no gráfico (um evento de alta probabilidade e alto impacto dispõe -se na região escura, 
devendo ser monitorado com muito cuidado).  
 
No Gráfico B, Exposição Final, a exposição financeira mensurada no eixo vertical já é o 
resultado do impacto multiplicado pela probabilidade (Gráfico A). A partir de então, 
incorpora -se o impacto intangível de cada um dos eventos. O evento 1, que numa 
primeira anális e foi classificado como de média importância (região intermediária no 
Gráfico A), caso tenha um impacto intangível alto, passa a se enquadrar na região escura 
no Gráfico B, pois a Exposição Final (Exposição Financeira + Impacto Intangível) é 
considerada al ta. 
 
A elaboração de um Mapa de Avaliação dos Riscos, tal como ilustrado pelo IBGC na 
figura, é uma etapa fundamental na priorização do gerenciamento de riscos e na 
definição do tratamento que deve ser dado a cada um dos riscos identificados.  
 
3.3.1  Critérios utilizados pela Consultoria Especializada  
 
A Ernst&Young27 apresenta a elaboração de mapa de riscos para avaliação da 
relevância dos riscos. Primeiramente, sugere -se a construção de escala para classificar 
a magnitude da severidade do risco e ou tra para classificar a frequência do risco. As 
escalas devem ser construídas com base nos dados de perda e frequência do risco com 
maior impacto para o banco e atualizadas anualmente.  
 
O mapa de riscos consiste no inventário de riscos, categorizado por me io das escalas 
de classificação, conforme exemplo a seguir:  
 
Quadro 2. Escala de Severidade  
 
Severidade (R$ mil)  
Classificação  De Até 
Extrema  50.000  - 
Alta 5.000  50.000  
Média  500 5.000  
Baixa  0,5 500 
Irrisória  - 0,5 
 
Quadro 3. Escala de Frequência  
 
Frequência de eventos por ano  
Classificação  De Até 
Muito Frequente  100.000  - 
Frequente  1.000  100.000  
Provável  10 1.000  
Ocasional  0,1 10 
Remoto  0,001  0,1 
 
 
 
27 Projeto de Gerenciamento de Capital e Icaap, Ponto de Homologação nº 15 de 10/2013.  

=== Página 16 ===

 
Anexo 1 da DDC Diris 89044 , de 17.12.202 1. 
____________________________________________________________________________________ _ 
 
_____________________________________________________________________________________  
 
Estudo Técnico de Identificação de Riscos                                                                                                             16 #Confidencial  
Nas escalas representadas nos quadros acima, a coluna de classificação corresponde à 
classificação da severidade e frequência, que pode ser desde irrisória até extrema 
(escala de severidade), e de remoto a muito frequente (escala de frequência). As colunas  
de “De” e “Até” correspondem aos limites de perda em milhares de reais (escala de 
severidade) e frequência de eventos por ano (escala de frequência). Os valores 
apresentados foram utilizados como exemplo e devem ser calibrados de acordo com a 
instituição financeira.  
 
A partir dos dados de perdas e frequência e da experiência dos executivos, a área de 
risco incorpora ao inventário de riscos as informações de severidade e frequência dos 
riscos identificados.  
Quadro 4. Inventário de R iscos (resultado ilustrativo)  
 
Lista de Riscos  Severidade  Frequência  
Crédito  60.000.000  40.000  
Concentração  2.500.000  800 
Operacional  20.000.000  80.000  
Legal  Média  Provável  
Conjuntura  Média  Ocasional  
Risco País  Baixa  Provável  
Reputação  Baixa  Provável  
Compliance  400.000  700 
Mercado  300.000  400 
Liquidez  Alta Frequente  
Taxa de Juros do Banking 
Book  200.000  300 
Contraparte  100.000  400 
Modelo  100.000  600 
Participações  5.000.000  40.000  
Socioambiental  Irrisória  Remoto  
 
Os riscos Legal, Conjuntura, País, Reputação, Liquidez e Socioambiental podem ser 
avaliados a partir de métodos qualitativos, tais como workshops  com executivos. O 
restante dos riscos é avaliado por meio de modelos quantitativos, através de histórico de 
perdas e utilização de modelos internos.  
 
A identificação dos riscos relevantes é realizada a partir da junção do inventário de riscos 
com as escalas apresentadas, resultando no mapa de riscos demonstrado abaixo:  
 
Quadro 5. Exemplo de Mapa de Riscos de um Banco  
 
 
Lista de Riscos  Classificação 
da Severidade  Classificação 
da Frequência  
Crédito  Extrema  Frequente  
Concentração  Média  Provável  
Operacional  Alta Frequente  
Legal  Média  Provável  
Conjuntura  Média  Ocasional  
Risco País  Baixa  Provável  
Reputação  Baixa  Provável  
Compliance  Baixa  Provável  
Mercado  Baixa  Provável  
Liquidez  Alta Frequente  

=== Página 17 ===

 
Anexo 1 da DDC Diris 89044 , de 17.12.202 1. 
____________________________________________________________________________________ _ 
 
_____________________________________________________________________________________  
 
Estudo Técnico de Identificação de Riscos                                                                                                             17 #Confidencial  
Taxa de Juros do Banking  Book  Baixa  Provável  
Contraparte  Baixa  Provável  
Modelo  Baixa  Provável  
Participações  Alta Frequente  
Socioambiental  Irrisória  Remoto  
 
A partir do mapa de riscos, é construída a matriz de relevância de riscos, conforme 
demostrado na figura:  
 
 
Figura 3. Matriz de Relevância de Riscos (resultado ilustrativo)  
 
Os riscos localizados na região vermelha são aqueles que possuem classificação de 
severidade igual à média, alta ou extrema e classificação de frequência igual à provável, 
frequente ou muito frequente e são, portanto, considerados relevantes.  
 
3.4 Tratamento dos Riscos  
 
Segundo a ABNT28, o propósito do tratamento de riscos é selecionar e implementar 
opções para abordar riscos que envolve um processo iterativo de:  
a) formular e selecionar opções para tratamento do risco;  
b) planejar e implementar o tratamento do risco;  
c) avaliar a eficácia deste tratamento;  
d) decidir se o risco remanescente é aceitável;  
e) se não for aceitável, realizar tratamento adicional.  
 
Ainda de acordo com a Associação, a s opções de tratamento de riscos envolvem  um ou 
mais dos seguintes:  
a) evitar o risco ao decidir não iniciar ou continuar com a atividade que dá origem ao 
risco;  
b) assumir ou aumentar o risco  de maneira a perseguir  uma oportunidade;  
c) remover  a fonte de risco;  
 
28 ABNT NBR ISO 31000 de 03/2018, Gestão de Riscos – Diretrizes, item 6.5, p. 14.  


=== Página 18 ===

 
Anexo 1 da DDC Diris 89044 , de 17.12.202 1. 
____________________________________________________________________________________ _ 
 
_____________________________________________________________________________________  
 
Estudo Técnico de Identificação de Riscos                                                                                                             18 #Confidencial  
d) mudar a probabilidade;  
e) mudar as  consequências;  
f) compartilhar o risco (por exemplo, por meio de contratos, compra de seguros) ; 
g) reter o risco por decisão fundamentada . 
 
Após o tratamento do risco, resta o risco remanescente, denominado risco residual ou 
retido29. 
 
O IBGC30 informa que a al ta administração poderá determinar seu posicionamento frente 
aos riscos, considerando seus efeitos, grau de aversão e resposta, complementada por 
uma análise de custo -benefício. As possíveis respostas aos riscos são:  
a) evitar o risco: decisão de não se envol ver ou agir de forma a se retirar de uma 
situação de risco;  
b) aceitar o risco: neste caso, apresentam -se quatro alternativas:  
i) reter: manter o risco no nível atual de impacto e probabilidade;  
ii) reduzir: ações são tomadas para minimizar a probabilidade e/ou o impacto 
do risco;  
iii) transferir/compartilhar: atividades que visam reduzir o impacto e/ou a 
probabilidade de ocorrência do risco através da transferência ou, em alguns 
casos, do compartilhamento de uma parte do risco;  
iv) explorar: aumentar o grau de exposição ao risco na medida em que isto 
possibilita vantagens competitivas.  
 
As decisões sobre evitar, reter, reduzir, transferir ou explorar riscos estão baseadas na 
avaliação do impacto dessas possíveis decisões sobre os in dicadores de desempenho 
escolhidos e sobre a imagem da organização vis -à-vis os custos de se estabelecerem 
controles internos.  
 
Um dos objetivos do gerenciamento de riscos corporativos é buscar um nível confortável 
e balanceado de retenção, redução, explo ração e transferência de riscos, adequado a 
seu apetite definido estrategicamente, envolvendo os objetivos, os riscos respectivos e 
os controles internos.  
 
Paul Hopkin31 utiliza abordagem semelhante de avaliação dos riscos, em que são 
considerados o impact o e probabilidade de sua ocorrência. Os riscos avaliados são 
tratados conforme os 4Ts de Gestão, reproduzidos na figura a seguir:  
a) transferir o risco para terceiro;  
b) tirar (eliminar) a atividade que gera o risco;  
c) tolerar o risco e seu impacto;  
d) tratar o risco  para reduzir a probabilidade e impacto ou exposição.  
 
29 ABNT NBR ISO Guia 31000 de 11/2009, Gestão de Riscos – Princípios e Diretrizes, item 2.27, p. 6.  
30 IBGC, Guia de Orientação para Gerenciamento de Riscos Corporativos, cap 2.4  Tratamento dos Riscos.  
31 Paul Hopkin, Fundamentals of Risk Management , 3º Edição, cap 15 Risk likelihood and impact.  

=== Página 19 ===

 
Anexo 1 da DDC Diris 89044 , de 17.12.202 1. 
____________________________________________________________________________________ _ 
 
_____________________________________________________________________________________  
 
Estudo Técnico de Identificação de Riscos                                                                                                             19 #Confidencial  
 
Figura 4. Matriz de Riscos e os 4Ts de Gestão  
 
Além das possibilidades de se evitar, reter, reduzir e transferir o risco, ainda que com 
nomenclaturas diferentes, mas com significado comum às fontes pesquisadas, a ABNT 
e IBGC trazem a opção de aumento do grau de exposição ao risco a fim de buscar 
vantagens competitivas.  
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 


=== Página 20 ===

 
Anexo 1 da DDC Diris 89044 , de 17.12.202 1. 
____________________________________________________________________________________ _ 
 
_____________________________________________________________________________________  
 
Estudo Técnico de Identificação de Riscos                                                                                                             20 #Confidencial  
4. Modelo de Identificação dos Riscos do C onglomerado Prudencial 
Banco do Brasil (CPBB)  
 
A estrutura de gerenciamento de riscos, que inclui o processo de identificação dos riscos, 
deve ser unificada para as instituições integrantes de um mesmo Conglomerado 
Prudencial32. 
 
4.1 Identificação dos Riscos  
 
Focando as atividades do CPBB , o processo de identificação dos riscos considera os 
direcionadores estratégicos do Banco, os riscos incorridos pelas Entidades Ligadas 
(ELBB), bem como os fatores de risco divulgados no Formulário de Referência do BB.  
 
A Bas e de Processos (BP) é o insumo que propicia a visão da primeira linha de defesa 
na identificação dos riscos corporativos. Os processos objeto de escopo anual são 
mapeados com o auxílio da Dicoi e os riscos identificados são registrados na BP, 
conforme Bibl ioteca de Riscos constante na ferramenta.  
 
Além disso, as diretrizes regulatórias  e de supervisão , práticas de mercado e benchmark  
com outras instituições são utilizados de modo a avaliar os riscos declarados por essas 
fontes e sua aplicação no CPBB.  
 
O processo e insumos utilizados para a identificação dos riscos incorridos pelo CPBB 
estão ilustrados na figura a seguir:  
 
 
Figura 5.  Processo de identificação dos riscos   
 
 
 
 
 
 
32 Resolução CMN nº 4.557, de 23.02.2017, Capítulo I - Do Escopo, Art. 2º, § 2º.   
Diretrizes 
Regulatórias 
e de 
Supervisão
Atos normativos 
do Bacen
Normas ou 
orientações de 
reguladores no 
exteriorPreocupa -
ções do 
Mercado
Relatórios de 
consultorias
Publicações 
especializadasBenchmark
Riscos 
declarados por 
concorrentes 
brasileiros
Riscos 
declarados por 
IF's no exterior de 
porte semelhante 
ao BBEntidades 
Ligadas 
Banco do 
Brasil (ELBB)
Riscos incorridos 
pelas ELBB, 
considerando a 
classificação:
 CP
 CICP
 PICP
 Demais 
Participações
 Patrocinada, 
Administrada e 
FundaçãoFormulário de 
Referência
Fatores de risco, 
constantes no 
Formulário de 
Referência BB, 
considerados 
mais relevantes e 
que possam 
impactar os 
negócios e os 
resultados das 
operações do 
Banco.Direciona -
dores 
Estratégicos
Riscos que 
possam impactar 
o atingimento dos 
objetivos 
constantes na 
Estratégia 
Corporativa
Insumos Externos Insumos InternosBase de 
Processos 
(BP)
Riscos
identificados
pelos gestores de 
primeira linha de 
defesa em seus 
processos

=== Página 21 ===

 
Anexo 1 da DDC Diris 89044 , de 17.12.202 1. 
____________________________________________________________________________________ _ 
 
_____________________________________________________________________________________  
 
Estudo Técnico de Identificação de Riscos                                                                                                             21 #Confidencial  
4.1.1  Taxonomia Corporativa dos Riscos  
 
Os riscos são caracterizados por seus eventos, fatores e impactos, constando na 
Taxonomia Corporativa de Riscos, resultado desse processo de identificação de riscos.  
 
Qualquer alteração ou identificação de novos riscos, fatores e impactos, pelos gestores 
de primeira linha de defesa, é avaliada em conjunto com a Diris, responsável pela 
Taxonomia Corporativa de Riscos, atualizando -a caso pertinente.  
 
Dentre os insumos utilizados, a ferramenta Radar Regulatório, sob gestão da Dicoi, 
auxilia na supervisão do monitoramento regulatório, onde os gestores identificam e 
avaliam as normas capturadas e o estabelecimento dos planos de ação para a sua 
internalização. Assim, a prospecção realizada na ferramenta pode captar informações 
em normas externas q ue auxiliam o processo de identificação de riscos. As publicações 
na página Economia em Foco também auxiliam no mapeamento das perspectivas  
relacionadas à conjuntura de riscos mundial.  
 
Os fatos supervenientes nas condições macroeconômicas e estratégicas, as mudanças 
nos negócios das entidades ligadas, as atualizações extraordinárias dos fatores de risco 
do Banco, a identificação de riscos em processos durante o ciclo anual de avaliação da 
Dicoi, bem como o foco dado pelos reguladores, consultorias e entida des especializadas 
nas diversas publicações relacionadas a riscos, são subsídios para alteração de eventos, 
fatores e impactos da Taxonomia Corporativa de Riscos, a partir de sua constatação, 
proporcionado dinâmica ao processo. Dessa forma, está prevista  constante atualização 
do instrumento.  Deve rá ser feita revisão anual caso não haja mudanças significativas no 
período.  A decisão sobre revisões extraordinárias está a cargo do Comitê Diris, 
fundamentado pela gerência responsável pelo estudo.  
 
A Taxonomia Co rporativa de Riscos  contempla três documentos:  
a) Dicionário Corporativo dos Fatores de Riscos, organizado em quatro níveis 
hierárquicos, iniciando pelo N1 (pessoas, processos, sistemas e eventos 
externos), subdividindo -se até o N4;  
b) Dicionário Corporativo dos Eventos de Risco, estruturado em quatro níveis, a 
partir dos riscos (N1) e das categorias (N 2) que o compõem, ramificando -se até o 
N4, de forma a tornar mais claro o comportamento do risco no nível de possíveis 
eventos; e  
c) Dicion ário Corporativo dos Impactos de Risco, cuja estrutura está classificada  em 
três níveis, onde o N1 (perdas diretas e indiretas) se subdivide até o N3 , trazendo 
as possibilidades de impacto dos riscos.  
 
4.2 Classificação  dos Riscos  
 
A classificação utilizada pelo IBGC  e BIS  (item 3.2) está mais coerente à atividade do 
Banco, quando considerada a definição dada aos riscos estratégicos, financeiros e 
operacionais, em função da natureza e área afetada pelos eventos.  Também foi 
considerada  a divisão dos grupamentos de riscos declarado s pelo Bacen como de 
origem “financeira” e “não financeira”  (que inclu i o risco operacional ).   
 

=== Página 22 ===

 
Anexo 1 da DDC Diris 89044 , de 17.12.202 1. 
____________________________________________________________________________________ _ 
 
_____________________________________________________________________________________  
 
Estudo Técnico de Identificação de Riscos                                                                                                             22 #Confidencial  
Assim, foi definida a classificação dos riscos em função de sua natureza: estratégica, 
financeira, n ão financeira  e reputacional.  
 
O risco estratégico  está associado à tomada de decisão da alta administração ou 
utilização de modelos inadequados para definição e implementação da estratégia, 
desencadeando outros riscos e gerando impacto substancial na instituição.  
 
Os riscos financeiros  são aqueles associados à exposição das operações financeiras 
da instituição, ou seja, é o risco de que os fluxos de caixa não sejam administrados 
efetivamente para maximizar a geração de caixa operacional, gerenciar os riscos e 
retornos específicos das transações financeiras e c aptar e aplicar recursos financeiros 
de acordo com as políticas estabelecidas.  
 
Os riscos não financeiros  decorrem da realização das atividades e utilização dos 
recursos necessários à entrega do produto/serviço ao cliente. Esses riscos podem 
acarretar redu ção, degradação ou interrupção das atividades, além do potencial geração 
de passivos contratuais, regulatórios e ambientais.  
 
O risco reputacional  é decorrente da não efetividade da gestão dos riscos estratégicos, 
financeiros e não financeiros e podem retr oalimentar o processo de entrega do 
produto/serviço ao cliente, ocasionando perdas para a instituição.  
 
4.3 Definição do Grau de Relevância  dos Riscos  
 
Conforme exposto  no item 3.3, para a definição do grau de relevância do risco, a 
Ernst&Young  considera o impacto financeiro do risco, quando é possível essa 
mensuração, e quando não há dados quantitativos disponíveis, o r isco é classificado 
segundo escala baseada em critérios qualitativos. Nessa avaliação, a relevância do risco 
é baseada em aspecto quantitativo ou qualitativo, não se agregando os critérios. O 
modelo que o IBGC propõe torna -se mais consistente ao processo d e definição do grau 
de relevância dos riscos do Banco, pois considera os aspectos tangíveis e intangíveis de 
um mesmo risco. Avalia -se primeiramente o aspecto tangível do risco, medido em função 
da probabilidade de ocorrência e o seu impacto financeiro. À essa avaliação, incorpora -
se o impacto intangível, resultando na exposição final ao risco.  
 
Com a Taxonomia Corporativa de Riscos definida, é realizada a revisão da relevância 
dos riscos e categorias, observando seus fatores e impactos, conforme segue:  

=== Página 23 ===

 
Anexo 1 da DDC Diris 89044 , de 17.12.202 1. 
____________________________________________________________________________________ _ 
 
_____________________________________________________________________________________  
 
Estudo Técnico de Identificação de Riscos                                                                                                             23 #Confidencial  
 
Figura 6.  Processo de Avaliação dos Riscos  
 
O modelo de avaliação da relevância considera os critérios quantitativos (perdas diretas) 
e qualitativos (perdas indiretas), sendo o critério qualitativo caracterizado pelos fatores e 
impactos indiretos da Taxonomia Corporativa de Riscos. O resultado agregado dos 
critérios é a matriz de relevância dos riscos. O esquema geral consta na figura abaixo, e 
está detalhado nos próximos itens:  
 
 
Figura 7.  Modelo de Aval iação dos Riscos  
 
4.3.1  Critérios Quantitativos  
 
Para aqueles riscos passíveis de mensuração, optou -se por adotar como critério 
quantitativo os limites estabelecidos na Declaração de Apetite e Tolerância a Riscos 
Critérios 
Quantitativos
Limite de capital 
da Declaração de 
Apetite e 
Tolerância a 
Riscos (RAS)
ou
Volume de Perda 
(VaR) em relação 
ao Capital 
PrincipalEscala de 
Criticidade 
Quantitativa
Nível de 
exposição 
considerando 
limites de capital 
ou volume de 
perdas em 
relação ao capital 
principalCritérios 
Qualitativos
Avaliação do 
grau de 
relacionamento 
dos riscos com 
seus fatores 
(causas)
Avaliação dos 
impactos 
indiretos gerados 
pelos riscosEscala de 
Criticidade 
Qualitativa
Nível de 
exposição 
considerando 
aspectos para 
gestão 
preventiva dos 
riscos (fatores) e 
detectiva 
(impactos)Matriz de 
Relevância
Nível de risco 
considerando 
critérios 
quantitativos e 
qualitativos
Escala 
CriticidadeLimite de Capital 
da RAS ou VaR 
sobre o Capital 
PrincipalEscala 
CriticidadeFatores de 
RiscoEscala 
CriticidadePerdas 
IndiretasEscala 
CriticidadeImpacto 
Qualitativo
Muito Baixo X ≤ 1,0% Muito Baixo 51,5 a 77,2 Muito Baixo 9,0 a 13,4 Muito Baixo 60,5 a 90,7
Baixo 1,0% < X <= 3,0% Baixo 77,3 a 102,9 Baixo 13,5 a 17,9 Baixo 90,8 a 120,9
Médio 3,0% < X <= 5,0% Médio 103,0 a 128,7 Médio 18,0 a 22,4 Médio 121,0 a 151,2
Alto 5,0% < X<= 10% Alto 128,8 a 154,4 Alto 22,5 a 26,9 Alto 151,3 a 181,4
Muito Alto X > 10% Muito Alto 154,5 a 257,5 Muito Alto 27,0 a 45 Muito Alto 181,5 a 302,5
Perdas Diretas Perdas Indiretas% VaR=PE+PI / CP
+ =
Regua QL Fator Regua QL Per IndCritério Quantitativo
RAS – Limites 2021
%CN1 - IRRBB
%CP- demais riscos
Regua QT Gestão 
Detectiva da 
Perda IndiretaGestão 
Preventiva do 
Fator de RiscoCritério Qualitativo
Xou, quando não houver limite:
Matriz

=== Página 24 ===

 
Anexo 1 da DDC Diris 89044 , de 17.12.202 1. 
____________________________________________________________________________________ _ 
 
_____________________________________________________________________________________  
 
Estudo Técnico de Identificação de Riscos                                                                                                             24 #Confidencial  
(RAS)33, em percentuais do Capital Nível 1 (CN1) para o IRRBB e do Capital Principal 
(CP) par a os demais riscos.  
 
Quando não houver limite estabelecido na RAS, é utilizado o  Valor em Risco (VaR)  em 
relação ao Capital Principal, que incorpora os conceitos de probabilidade e impacto 
financeiro dos riscos.  
 
Observa -se que a relação do valor em risco frente  ao capital constitui -se em indicador 
da perda potencial que o risco avaliado pode causar para o Banco. Dentre as formas de 
mensurar a perda potencial, cabe destacar o conceito de Valor em Risco ou Value at 
Risk (VaR), o qual pode ser definido como “ a maior (ou pior) perda potencial dentro de 
determinado período de tempo e intervalo de confiança ”34. 
 
A escolha do Capital Principal ocorre por representar o tipo de capital com a mais alta 
capacidade para absorção de perdas sob o andamento normal d as operações da 
instituição financeira.  
 
Considerando as características dos riscos incorridos pelo Banco, considera -se que os 
riscos cujo limite na RAS ou VaR sobre o capital seja maior do que 1% devam ter 
processos de gestão estabelecidos. De forma a dis criminar o grau de relevância dos 
riscos, foi definida a escala abaixo:  
Quadro 6. Régua Quantitativo  
 
 
 
33 NTE Diris 87121 , de 26.10.2021 , aprovada pelo CA em 05.11.2021 . 
34 Philippe Jorion, Value at Risk , 3º Edição,.  
Escala CriticidadeLimite de Capital da RAS ou 
VaR sobre o Capital Principal
Muito Baixo X ≤ 1,0% 
Baixo 1,0% < X <= 3,0%
Médio 3,0% < X <= 5,0%
Alto 5,0% < X<= 10%
Muito Alto X > 10% 
Tipo N1Impacto – 
Quantitativo
Modelo Cálculo
Perdas DiretasRAS – Limites  2022:
%CN1 - IRRBB
%CP- demais riscos
Conforme modelo de cáculo de cada risco
ou, quando não houver limite:
%VaR=(PE+PI)/CP, onde:
VaR=Valor em Risco
PE=Perda Esperada (provisões)
PI=Perda Inesperada (maior valore entre CR=Capital 
Regulatório ou CE=Capital Econômico)
CP=Capital Principal

=== Página 25 ===

 
Anexo 1 da DDC Diris 89044 , de 17.12.202 1. 
____________________________________________________________________________________ _ 
 
_____________________________________________________________________________________  
 
Estudo Técnico de Identificação de Riscos                                                                                                             25 #Confidencial  
O VaR é a soma de perdas esperadas e inesperadas. Nas perdas esperadas, são 
consideradas as provisões para os riscos que têm esses dados calculados. Com relação 
à perda inesperada, utiliza -se o capital regulatório ou econômico, ou o máximo entre o 
capital regulatório e econômico, conforme disponibilidade de informações.  
 
4.3.2  Critérios Qualitativos  
 
Ainda que o risco tenha sido avaliado sob o aspecto quantitativo ou que não seja possível 
essa mensuração em virtude da ausência de metodologia ou base de dados, é 
necessária sua análise utilizando critérios qualitativos. Tal avaliação é realizada p elo 
gestor dos riscos  em segunda linha de defesa  (Diris) , que possu i conhecimento do 
escopo dos riscos e considera seus diversos atributos.   
 
Um dos critérios qualitativos utilizados para avaliação da relevância dos riscos são os 
fatores que podem causar os risco s, descritos no N3 do Dicionário de Fatores da 
Taxonomia, que demonstram nível de abrangência e qualificação suficientes das causas 
dos riscos.  
 
A avaliação dos fatores possibilita conhecer as origens dos riscos, podendo ser 
estabelecidos processos de ges tão preventivos, de modo a minimizar sua ocorrência. O 
critério leva em conta diferentes pesos definidos para os quesitos ( qualificadores ) 
avaliados, bem como probabilidades de relação com a manifestação do risco. Para os 
qualificadores  relacionados a even tos internos (processos, pessoas e sistemas), o peso 
é menor em função da viabilidade de gestão, aumentando conforme sua complexidade. 
Os qualificadores relacionados a processos são mais fáceis de serem ajustados, 
bastando alinhamento e adequação interna d as áreas. Os qualificadores relacionados a 
pessoas exigem mudança de cultura, o que leva tempo para adaptação. Os 
qualificadores relacionados a sistemas, além de demandarem médio e longo prazo para 
ajustes, têm alto custo de implementação. Os pesos dos qualificadores  relacionados a 
eventos externos são maiores, pois tornam as atividades do Banco mais vulneráveis, 
independente dos processos de gestão estabelecidos.   
 
À lista de qualificadores são elencados exemplos  de fatores que podem comprometer o 
alcance da ECBB, traduzidos das  discussões ocorridas na análise de mercados para 
formulação da estratégia corporativa 2022/2026, publicações externas e da Gerência 
Inteligência de Mercado .  
 
A régua e fatores estabelecid os para avaliação dos riscos e categorias em relação às 
suas possíveis causas consta no quadro a seguir:  
 
 
 
 
 
 
 
 
 
 

=== Página 26 ===

 
Anexo 1 da DDC Diris 89044 , de 17.12.202 1. 
____________________________________________________________________________________ _ 
 
_____________________________________________________________________________________  
 
Estudo Técnico de Identificação de Riscos                                                                                                             26 #Confidencial  
Quadro 7. Régua  e Fatores para Av.  Qualitativ a do Risco  
 
Escala Criticidade  Fatores de Risco  
Muito Baixo  51,5 a 77,2 
Baixo  77,3 a 102,9  
Médio  103,0 a 128,7  
Alto 128,8 a 154,4  
Muito Alto  154,5 a 257,5  
 
Fator de Risco - Qualitativo - Gestão Preventiva   
Qual é o grau de 
exposição do 
risco à causa 
(fator):  
Muito Baixo  
Baixo  
Médio  
Alto 
Muito Alto  
 
 
 
 
           
Categoria 
N1 Qualificador N3  Exemplos de fatores que 
podem comprometer a ECBB   Peso  1 2 3 4 5 
Processos  Definição Incorreta da 
Governança  Morosidade no processo de 
decisão e execução    0,5 0,5 1,0 1,5 2,0 2,5 
Desenho/ Execução 
Inadequada do 
Processo  Operação em silos das áreas, 
informações e soluções    0,5 0,5 1,0 1,5 2,0 2,5 
Estrutura Inadequada 
ou Insuficiente  Foco em atividades 
operacionais e não estratégicas    0,5 0,5 1,0 1,5 2,0 2,5 
Relacion amento com 
Terceiros Inadequado  Incapacidade operacional de 
atendimento    0,5 0,5 1,0 1,5 2,0 2,5 
Pessoas  Conduta Inadequada, 
Antiética ou Negligente  Ausência de mobilização ampla 
(senso de dono)    1,0 1,0 2,0 3,0 4,0 5,0 
Habilida des e 
Competências 
Insuficientes  Desqualificação para o mindset  
digital    1,0 1,0 2,0 3,0 4,0 5,0 
Condiçõe s de Trabalho 
e Qualidade de Vida 
Inadequadas  Falta de flexibilidade, autonomia 
e compensação financeira    1,0 1,0 2,0 3,0 4,0 5,0 
Sistemas  Hardware, Rede e 
Telecomunicações 
Deficientes  Obsoletismo da base de TI    1,5 1,5 3,0 4,5 6,0 7,5 
Desenvol vimento/Manut
enção Inadequada de 
Aplicações  Inadequação das ferramentas 
de análises de dados    1,5 1,5 3,0 4,5 6,0 7,5 
Seguranç a Lógica 
Deficiente  Fragilidade dos mecanismos de 
segurança de dados    1,5 1,5 3,0 4,5 6,0 7,5 
Eventos 
Externos  Fenômenos 
Inesperados da 
Natureza/Climáticos  Demanda por negócios 
relacionados à redução de 
emissões    2,0 2,0 4,0 6,0 8,0 10,0 
Surgi mento de epidemias e 
variantes    2,0 2,0 4,0 6,0 8,0 10,0 
Ambiente 
Regulatório/Legal 
Adverso  Adaptação onerosa à reforma 
fiscal/tributária    2,0 2,0 4,0 6,0 8,0 10,0 
Aumen to da pressão regulatória 
advinda de novas normas    2,0 2,0 4,0 6,0 8,0 10,0 
Ambien te 
Político/Social/Econômi
co Instável  Aumento da inflação/taxas de 
juros (Selic)    2,0 2,0 4,0 6,0 8,0 10,0 
Desva lorização do real    2,0 2,0 4,0 6,0 8,0 10,0 
Introdução de transações com 
moedas digitais (Criptomoedas)    2,0 2,0 4,0 6,0 8,0 10,0 
Ambiente Cultural 
Transformado  Intensificação das relações 
virtuais ( Bots)   2,0 2,0 4,0 6,0 8,0 10,0 
Demanda maior por 
atendimento personalizado    2,0 2,0 4,0 6,0 8,0 10,0 

=== Página 27 ===

 
Anexo 1 da DDC Diris 89044 , de 17.12.202 1. 
____________________________________________________________________________________ _ 
 
_____________________________________________________________________________________  
 
Estudo Técnico de Identificação de Riscos                                                                                                             27 #Confidencial  
Aumen to da influência digital de 
consumo ( Finfluencers )    2,0 2,0 4,0 6,0 8,0 10,0 
Ampli ação da prestação de 
serviços (consultoria financeira)    2,0 2,0 4,0 6,0 8,0 10,0 
Provimento maior de soluções 
de investimento    2,0 2,0 4,0 6,0 8,0 10,0 
Ambien te Concorrencial 
Altamente Competitivo  Isenção na cobrança de tarifas 
e taxas    2,0 2,0 4,0 6,0 8,0 10,0 
Crescimento do PIX e carteiras 
digitais ( e-Wallets )   2,0 2,0 4,0 6,0 8,0 10,0 
Compartilhamento de dados 
(Open Banking )   2,0 2,0 4,0 6,0 8,0 10,0 
Uso de Plataforma/Ecossistema 
digital/ Marketplace    2,0 2,0 4,0 6,0 8,0 10,0 
Adesõ es a programas de 
recompensas ( Cashback )   2,0 2,0 4,0 6,0 8,0 10,0 
Aumento da atuação de 
neobanks, fintechs e bigtechs    2,0 2,0 4,0 6,0 8,0 10,0 
Atuaçã o Insatisfatória 
de Fornecedores e 
Parceiros  Ampliação da relação com 
Fornecedores/Prestadores    2,0 2,0 4,0 6,0 8,0 10,0 
Ampli ação da relação com 
Startups (Agtechs, Fintech, 
Govtech)    2,0 2,0 4,0 6,0 8,0 10,0 
Decisõ es Adversas do 
Controlador/Acionista  Ingerência governamental na 
atuação e negócios do Banco    2,0 2,0 4,0 6,0 8,0 10,0 
 
A avaliação final dos riscos e categorias em relação aos seus fatores é estabelecida 
conforme escala de criticidade dos fatores de risco.  
 
A distribuição da escala de criticidade dos fatores de risco possui intervalo igual  nas 
criticidades de “Muito Baixo” a “Alto” . Acima desses intervalos, a pontuação foi 
direcionada para a relevância muito alta, de modo que o resultado seja conservador 
quanto a exposição aos riscos.   
 
Para avaliação dos riscos e categorias em relação às suas perdas indiretas, foi definida 
régua considerando o N2 do Dicionário de Impactos da Taxonomia, conforme quadro a 
seguir:  
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 

=== Página 28 ===

 
Anexo 1 da DDC Diris 89044 , de 17.12.202 1. 
____________________________________________________________________________________ _ 
 
_____________________________________________________________________________________  
 
Estudo Técnico de Identificação de Riscos                                                                                                             28 #Confidencial  
Quadro 8. Régua e Perda s Indireta s para Av. Qualitativa do Risco  
 
Escala Criticidade  Perdas Indiretas  
Muito Baixo  9,0 a 13,4  
Baixo  13,5 a 17,9  
Médio  18,0 a 22,4  
Alto 22,5 a 26,9  
Muito Alto  27,0 a 45  
 
Perda Indireta - Qualitativo - Gestão Detectiva   
Qual é o impacto do 
risco na Forma (N2):  
Muito Baixo  
Baixo  
Médio  
Alto 
Muito Alto  
 
 
 
 
          
Tipo N1  Forma N2  
 Peso  1 2 3 4 5 
Perdas 
Indiretas  Sanções Administrativas   1,0 1,0 2,0 3,0 4,0 5,0 
Perda de Dados e Informações   1,0 1,0 2,0 3,0 4,0 5,0 
Descontinuidade dos Negócios   1,0 1,0 2,0 3,0 4,0 5,0 
Fardo ou Ônus Operacional   1,0 1,0 2,0 3,0 4,0 5,0 
Evasão / Insatisfação de Clientes   1,5 1,5 3,0 4,5 6,0 7,5 
Dano à Imagem e Reputação   1,5 1,5 3,0 4,5 6,0 7,5 
Alcance da Estratégia   2,0 2,0 4,0 6,0 8,0 10,0 
 
A avaliação das perdas indiretas permite o estabelecimento de controles visando diminuir 
sua reincidência. O critério estabelece diferentes pesos para os quesitos (forma de 
perda) avaliados, conforme seu alcance nos processos do Banco, bem como a dimensão 
do seu impacto. As perdas indiretas relacionadas  a sanções, perda de dados, 
descontinuidade  e fardo operacional têm peso menor pois , a princípio, são perdas 
localizadas, exigindo controle específico ou restrito.  As perdas que têm relação com 
clientes (evas ão / insatisfação, dano à imagem) possuem peso intermediário pois tratam 
de perdas de negócios e rentabilidade , exigindo tratamento junto às carteiras . As perdas 
que têm conexão com a estratégia têm peso  maior  pois dessa advém os direcionadores, 
indicadore s e atividades d e todo o Banco.  
 
A avaliação final dos riscos e categorias em relação às suas perdas indiretas é 
estabelecida conforme escala de perdas indiretas.  
 
A distribuição da escala de criticidade d as perdas indiretas  possui intervalo igual  nas 
criticidades de “Muito Baixo” a “Alto” . Acima desses intervalos, a pontuação foi 
direcionada para a relevância muito alta, de modo que o resultado seja conservador 
quanto a exposição aos riscos.   
 
A junção das réguas qualitativ as de fatores de risco e perdas indiretas resulta no 
resultado da avaliação qualitativa dos riscos  e categorias : 
 
 
 
 

=== Página 29 ===

 
Anexo 1 da DDC Diris 89044 , de 17.12.202 1. 
____________________________________________________________________________________ _ 
 
_____________________________________________________________________________________  
 
Estudo Técnico de Identificação de Riscos                                                                                                             29 #Confidencial  
Quadro 9. Régua Qualitativo Fator e Perda Indireta  
  
Escala 
Criticidade  Fatores de 
Risco  
+ Escala 
Criticidade  Perdas 
Indiretas  
= Escala 
Criticidade  Impacto 
Qualitativo  
Muito Baixo  51,5 a 77,2  Muito Baixo  9,0 a 13,4  Muito Baixo  60,5 a 90,7  
Baixo  77,3 a 102,9  Baixo  13,5 a 17,9  Baixo  90,8 a 120,9  
Médio  103,0 a 128,7  Médio  18,0 a 22,4  Médio  121,0 a 151,2  
Alto 128,8 a 154,4  Alto 22,5 a 26,9  Alto 151,3 a 181,4  
Muito Alto  154,5 a 257,5  Muito Alto  27,0 a 45  Muito Alto  181,5 a 302,5  
 
A avaliação dos riscos é resultado da média da avaliação d e suas  categorias. Nesta 
avaliação, não cabe exclusão de outlier  tendo em vista a forma de distribuição dos 
valores na escala e a não observância de valores significativamente discrepantes nas 
avaliações realizadas.  
 
As avaliações são realizadas pelo gestor dos riscos  em segunda linha de defesa ( Diris). 
Nas próximas revisões, poderão ser acionados os gestores dos riscos em primeira linha 
no intuito de uniformizar o entendimento sobre o grau de relevância gerado.  
 
4.3.3  Matriz de Relevância dos Riscos  
 
A matriz de relevânci a é o instrumento que consolida os resultados das avaliações 
quantitativa e qualitativa, equilibrando -os de modo a demonstrar o grau de relevância 
final mais apurado para o risco. A figura demonstra modelo da matriz de relevância dos 
riscos:  

=== Página 30 ===

 
Anexo 1 da DDC Diris 89044 , de 17.12.202 1. 
____________________________________________________________________________________ _ 
 
_____________________________________________________________________________________  
 
Estudo Técnico de Identificação de Riscos                                                                                                             30 #Confidencial  
 
Figura 8.  Modelo da Matriz de Relevância dos Riscos  
 
A matriz da figura pondera a avaliação quantitativa com a qualitativa e vice -versa, 
mantendo certo grau de conservadorismo. Quando os resultados das duas avaliações 
são iguais, mantém -se esse como  relevância final. Nos casos em que as avaliações 
quantitati va e qualitativa são diferentes, busca -se o resultado intermediário ou 
imediatamente superior entre as avaliações plotadas na matriz. Com isso, evita -se a 
definição de grau de relevância final baseada apenas em uma das avaliações com 
resultado mais alto.  
 
No caso dos riscos não mensuráveis, não há ponderação quantitativa e, portanto, sua 
exposição é refletida apenas pela percepção dos gestores na avaliação qualitativa . 
 
4.3.4  Análise dos resultados e  override35 
 
Reconhecendo que os critérios quantitativos e qualitativos podem não capturar 
adequadamente todas as características dos diversos riscos incorridos pelo Banco, dada 
 
35 Sobreposição ao resultado da metodologia a partir de análise qualificada e justificada.  


=== Página 31 ===

 
Anexo 1 da DDC Diris 89044 , de 17.12.202 1. 
____________________________________________________________________________________ _ 
 
_____________________________________________________________________________________  
 
Estudo Técnico de Identificação de Riscos                                                                                                             31 #Confidencial  
sua natureza ou, entre outros aspectos, o grau de informações disponíveis, é feita, 
adicion almente, análise mais aprofundada dos riscos com base em informações externas 
ou a partir da opinião de áreas especialistas.  
 
4.3.5  Conjunto Corporativo de Riscos Relevantes  
 
Com a aplicação da matriz e override , chega -se ao conjunto corporativo de riscos 
relevantes, onde são apresentados os graus de relevância de cada risco, apurados por 
meio dos critérios quantitativo e qualitativo.   
 
4.3.6  Mapa d os Riscos  
 
O processo de identificação dos riscos envolve a descrição de seus fatores, eventos e 
potenciai s impactos, considerando a estratégia estabelecida. A partir dos conceitos 
constantes no capítulo 2, é definido mapa de riscos demonstrando a relação existente 
entre fatores de risco, impactos dos riscos, bem como os riscos e categorias constantes 
na Taxonomia Corporativa de Riscos (item 4.1.1 ).  
   
4.4 Tratamento dos Riscos  
 
4.4.1  Etapas da Implementação da Gestão dos Riscos  
 
Caso um novo risco seja identificado e tenha sua relevância final estabelecida entre 
Baixa e Muito Alta, deverão ser determinados seus gestores, processos e estrutura de 
gerenciamento. Para os riscos que foram considerados relevantes em revisões 
anteriores  e que já possuem gestores, processos e estrutura definidos, podem ser 
necessárias alterações posteriores em função de evolução no processo de gestão. Os 
riscos de relevância muito baixa, embora não mereçam tratamento específico de gestão, 
devem ser observ ados, de forma que seja possível identificar possíveis elevações nas 
exposições. O quadro abaixo demonstra as etapas da implementação da gestão dos 
riscos:  
Quadro 10. Etapas da Implementação e da Revisão dos Processos de Gestão dos Riscos  
 
Etapa  Objetivo  Prazo  Responsável  
1 Identificar e definir a relevância 
dos riscos do CPBB.  A partir da constatação de 
alterações de cenários conforme 
definido no item 4.1.  Diris  
2 Definir(alterar) responsável(eis) 
pela gestão do(s) risco(s) 
considerado(s) relevante(s).  Até 120 dias após finalização da 
etapa 1(novo risco) ou de 
processo de reestruturação.  Direo  
3 Definir(alterar)  framework de 
gestão do(s) risco(s) a partir de 
estudo que considere práticas de 
mercado e a dimensão e 
exposição ao(s) risco(s).  Até 180 dias após finalização da 
etapa 2 ou quando houver 
necessidade.  Área(s) responsável(eis) pela 
gestão do(s) risco(s) com 
interveniência da Diris.  
4 Avaliar(revisar) processos e a 
estrutura de gestão do(s) risco(s).  Até 180 dias após finalização da 
etapa 3 ou quando houver 
necessidade.  Direo com interveniência da Diris 
e da(s) área(s) definida(s) como 
responsável(eis) pela gestão do 
risco.  
5 Implementar processos 
necessários para a gestão, 
compatíveis com a dimensão e a 
exposição ao(s) risco(s).  Após finalização da etapa 4, 
podendo ser estabelecido plano 
de ação para implementação das 
atividades de gestão do(s) 
risco(s).  Área(s) responsável(eis) pela 
gestão do(s) risco(s), em conjunto 
com o( s) interveniente(s) dos 
processos e fluxos estabelecidos.  

=== Página 32 ===

 
Anexo 1 da DDC Diris 89044 , de 17.12.202 1. 
____________________________________________________________________________________ _ 
 
_____________________________________________________________________________________  
 
Estudo Técnico de Identificação de Riscos                                                                                                             32 #Confidencial  
Nas etapas de 2 a 4, poderã o ser abertas ações Sigas para os respectivos responsáveis, 
com a finalidade de acompanhamento, pela Diris, dos objetivos definidos e prazos para 
implementação dos processos de gestão dos riscos.   
 
4.4.2  Atividades de Gestão dos Riscos  
 
Nas etapas 4 e 5 do processo  de implementação da gestão dos riscos, devem ser 
estabelecidos processos de gestão compatíveis com a dimensão e a exposição aos 
riscos englobando as seguintes atividades elencadas pelo Bacen36: identificar, mensurar, 
avaliar, mitigar, controlar, monitorar  e reportar os riscos.  A atividade “aprimorar” foi 
acrescentada devido à necessidade de aprimoramento contínuo dos processos de 
gestão de riscos, em linha com os princípios do Pilar II de Basileia.  O quadro seguinte 
apresenta a s atividades de gestão dos ri scos obsevando -se o Modelo Referencial de 
Linhas de Defesa37: 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
36 Resolução CMN nº 4.557, de 23.02.2017, Capítulo III – Da Estrutura de Gerenciamento de Riscos, Art. 6º.  
37 IN 224 -1.5 Estrutura Organizacional – Banco do Brasil, Gerenciamento de Riscos, Controles Internos e Compliance , versão 109. 

=== Página 33 ===

 
Anexo 1 da DDC Diris 89044 , de 17.12.202 1. 
____________________________________________________________________________________ _ 
 
_____________________________________________________________________________________  
 
Estudo Técnico de Identificação de Riscos                                                                                                             33 #Confidencial  
Quadro 11. Atividades de Gestão dos Riscos  
 
Identificar Mensurar Avaliar Mitigar Controlar Monitorar Reportar AprimorarAtividadeIdentificar os 
riscos, fatores e 
impactos a que o 
Banco está sujeito 
no nível 
institucional, com o 
auxílio dos 
gestores 
corporativos de 
riscos.Dimensionar 
quantitativa ou 
qualitativamente, a 
exposição aos 
riscos no nível 
institucional, com o 
auxílio dos 
gestores 
corporativos de 
riscos.Qualificar o 
resultado da 
mensuração dos 
riscos.Estabelecer 
diretrizes para a 
gestão dos riscos.
Disseminar a 
cultura de gestão 
dos riscosEstabelecer 
parâmetros para 
revisão dos riscos 
corporativos e sua 
exposição.Observar 
mudanças no 
ambiente externo 
e interno 
relacionadas aos 
riscos e seu grau 
de exposição.Comunicar, à Alta 
Administração e 
aos gestores 
corporativos de 
riscos, o resultado 
da avaliação dos 
riscos 
corporativos.Verificar a 
efetividade do 
modelo de 
identificação e 
avaliação dos 
riscos 
corporativos.ex. InstrumentoTaxonomia 
Corporativa de 
RiscosMétrica 
quanti/quali de 
exposiçãoEscala de 
criticidade
Matriz de riscos 
relevantesPolíticas
Modelo Referncial 
de Linhas de 
defesa (MRLD)Plano de 
Monitoramento 
Contínuo do 
ModeloFatores de risco 
Impactos de risco
Indicadores de 
capitalCorreio
IntranetEstudo Técnico de 
Identificação de 
RiscosAtividadeIdentificar os 
eventos, fatores e 
impactos do risco 
objeto de seu 
escopo de gestão, 
com o auxílio do 
gestor específico 
do risco e 
assessoria da 
estratégia de 
gestão de riscos.Dimensionar 
quantitativa ou 
qualitativamente, a 
exposição 
agregada aos 
eventos do risco 
objeto de seu 
escopo de gestão.Qualificar o 
resultado da 
mensuração 
agregada dos 
eventos do risco 
objeto de seu 
escopo de gestão.Estabelecer 
normas e 
procedimentos 
para a gestão do 
risco.
Emitir 
recomendação ao 
gestor específico 
do risco.Estabelecer 
indicadores e 
limites de 
exposição aos 
riscos e capital, 
agregando 
informações do 
gestor específico 
do risco.Registrar o 
comportamento e 
garantir a 
manutenção da 
exposição aos 
riscos conforme 
limites 
estabelecidos, 
acionando o 
gestor específico 
do risco.Comunicar à Alta 
Administração e 
ao gestor 
específico de 
risco, o 
comportamento da 
exposição 
agregada aos 
eventos do risco 
ou resultado do 
monitoramento.Verificar a 
efetividade do 
modelo de gestão 
de riscos 
corporativos e 
atuação do gestor 
específico de 
risco.ex. InstrumentoTaxonomia 
Corporativa de 
Riscos
Propostas e 
pareceres em 
DDCsMétrica 
quanti/quali de 
exposição
Teste de estresse
Cálculo de capital 
(regulatório/econô
mico)Escala de 
criticidade/Matriz
Análise de 
Cenários
Alocação de 
capitalInstrução 
normativa
Recomendação 
Técnica de Risco 
(RTR)Declaração de 
Apetite e 
Tolerância a 
Riscos (RAS) - 
Limites globaisIndicadores e 
Limites globais
Estados de alerta
Planos de 
Contingência/ 
Continuidade/ 
RecuperaçãoGovernança
Painel de Risco
Relatório
Ficha de 
ExtrapolaçãoManuais 
Metodologias
ProcessosAtividadeIdentificar os 
eventos, fatores e 
impactos do risco 
a que seu 
processo chave 
está sujeito, com o 
auxílio dos 
tomadores de 
risco e assessoria 
do gestor 
corporativo de 
risco.Dimensionar 
quantitativa ou 
qualitativamente, o 
efeito da 
exposição aos 
eventos do risco a 
que seu processo 
chave está sujeito.Analisar o impacto 
potencial dos 
eventos do risco 
em seu processo 
chave.
Definir parâmetros 
de respostas ao 
risco (“evitar”, 
“manter”, "reduzir" 
ou “explorar”).Estabelecer 
normas e 
procedimentos 
para a gestão dos 
eventos do risco 
em seu processo 
chave.Estabelecer 
indicadores e 
limites de 
exposição aos 
eventos do risco, 
agregando 
informações dos 
tomadores de 
risco.Registrar o 
comportamento e 
garantir a 
manutenção da 
exposição aos 
eventos do risco 
conforme limites 
estabelecidos, 
acionando os 
tomadores de 
risco.Comunicar ao 
gestor corporativo 
e tomadores de 
risco, o 
comportamento da 
exposição aos 
eventos do risco 
ou resultado do 
monitoramento.Verificar a 
adequação do seu 
processo chave 
de gestão do risco 
e atuação dos 
tomadores de 
risco.ex. InstrumentoTaxonomia 
Corporativa de 
Riscos
Propostas e 
pareceres em 
DDCsMétrica 
quanti/quali de 
exposiçãoEscala de 
criticidade/Matriz
Relação 
risco X controle X 
retornoInstrução 
normativaDeclaração de 
Apetite e 
Tolerância a 
Riscos (RAS) - 
Limites 
específicosIndicadores e 
Limites 
específicos
Estados de alerta
Planos de 
Contingência/ 
Continuidade/ 
RecuperaçãoGovernança
Painel 
RelatórioManuais 
Metodologias
ProcessosAtividadeIdentificar os 
eventos, fatores e 
impactos dos 
riscos a que seus 
processos estão 
sujeitos, com 
assessoria dos 
gestores 
específicos de 
riscos.Dimensionar 
quantitativa ou 
qualitativamente, o 
efeito da 
exposição aos 
eventos dos 
riscos a que seus 
processos estão 
sujeitos.Analisar o impacto 
potencial dos 
eventos dos 
riscos em seus 
processos.
Selecionar a 
resposta ao
risco (“evitar”, 
“manter”, "reduzir" 
ou “explorar”).Estabelecer 
normas e 
procedimentos 
para a gestão dos 
eventos dos 
riscos em seus 
processos.Estabelecer 
indicadores e 
limites de 
exposição aos 
eventos dos 
riscos em seus 
processos.Garantir a 
manutenção da 
exposição aos 
eventos dos 
riscos em seus 
processos 
conforme limites 
estabelecidos.Comunicar ao 
gestor específico 
e áreas de 
interesse, o 
comportamento da 
exposição aos 
eventos dos 
riscos em seus 
processos ou 
resultado do 
monitoramento.Verificar a 
adequação dos 
seus processos 
de gestão de 
riscos.ex. InstrumentoTaxonomia 
Corporativa de 
Riscos
Propostas e 
pareceres em 
DDCsMétrica 
quanti/quali de 
exposição Escala de 
criticidade/Matriz 
Relação 
risco X controle X 
retornoInstrução 
normativaDeclaração de 
Apetite e 
Tolerância a 
Riscos (RAS) - 
Limites 
operacionaisIndicadores e 
Limites 
operacionais 
Estados de alerta
Planos de 
Contingência/ 
Continuidade/ 
RecuperaçãoGovernança
Painel 
RelatórioManuais 
Metodologias
ProcessosEstratégia Gestão de Riscos Gestor Corporativo de Risco1º Linha de Defesa
Gestor Específico de Risco1º Linha de Defesa
Tomador de Risco2º Linha de DefesaAssessoraSubsidia
AssessoraSubsidia
AssessoraSubsidia

=== Página 34 ===

 
Anexo 1 da DDC Diris 89044 , de 17.12.202 1. 
____________________________________________________________________________________ _ 
 
_____________________________________________________________________________________  
 
Estudo Técnico de Identificação de Riscos                                                                                                             34 #Confidencial  
4.4.2.1  Apetite e Tolerância a Riscos  
 
A atividade de mensuração quantitativa e qualitativa do risco possibilita determinar o 
apetite aos riscos que, conforme  conceitos do IBGC constantes no item 3.4, é insumo 
para as decisões sobre evitar, reduzir , manter  ou explorar os riscos, levando em 
consideração os objetivos e custo dos controles a serem estabelecidos.  
 
A figura a seguir demonstra o esquema a partir do qual pode -se definir a estratégia de 
atuação  em relação aos riscos relevantes incorridos pelo Banco, cuja aplicação fica à 
critério dos gestores dos riscos.  
 
Nível de Exposição  Muito Alto  
 Reduzir  Reduzir  Reduzir  Evitar  Evitar  
Alto 
 Manter  Reduzir  Reduzir  Reduzir  Evitar  
Médio  
 Manter  Manter  Reduzir  Reduzir  Reduzir  
Baixo  
 Explorar  Manter  Manter  Reduzir  Reduzir  
Muito 
Baixo  
 Explorar  Explorar  Manter  Manter  Reduzir          
   Muito Alta  Alta Média  Baixa  Muito 
Baixa  
   Estrutura de Gestão - Robustez  
 
Figura 9.  Apetite e Tolerância a  Riscos  
 
4.5 Planejamento Estratégico e Gestão de Riscos  
 
Tendo em vista a importância da participação da Diris no planejamento estratégico, tanto 
na identificação e definição da relevância dos riscos, quanto na análise de cenários, 
apetite a riscos e gestão dos riscos estratégicos, foi realizado mapeamento do processo 
de construção e gestão da estratégia corporativa, identificando quais fases devem ter a 
participação da Diris. O processo demonstra, de forma seque ncial, as atividades 
realizadas e respectivos responsáveis, bem como sinaliza quais são consideradas 
fornecedoras de insumos e quais são recebedoras de informações, conforme figura a 
seguir : 
 
 

=== Página 35 ===

 
Anexo 1 da DDC Diris 89044 , de 17.12.202 1. 
____________________________________________________________________________________ _ 
 
_____________________________________________________________________________________  
 
Estudo Técnico de Identificação de Riscos                                                                                                             35 #Confidencial  
 
Figura 10.  Gestão da Estratégia e Riscos  
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
DiagnósticoFormulação de 
CenáriosGestão do 
Portfólio 
EstratégicoPlanejamento 
das UE's
Apresentação de 
Tendências e 
IncertezasMatriz SWOT
 Cenário Base 
(mais provável) e 
Cenários 
Alternativos 
(“Adverso” e 
“Otimista”, com 
menores 
probabilidades)Análise dos 
MercadosSimulação 
dos 
GuidancesDireciona-
mentos 
EstratégicosMapa 
Estratégico
Ambições do 
MercadoPlano 
DiretorAprovação 
Formal 
ECBB/PDIniciativas 
Estratégicas,
Programas, 
Projetos e Ações 
EstratégicasObjetivos e 
Resultados 
Chave (OKR)
Identificação de 
Riscos 
Corporativos 
(Geric/Gerec)Análise de 
Cenários 
(Gefic/Getes)Avaliação do 
Risco de 
Estratégia 
(Geric/Gerec)Assessoria 
sobre Riscos
(conf risco 
identificado)Planejamento 
das UE's
(Geric/Gerec)
Elaboração de 
Briefing  de 
Fatores de Risco 
para a Estratégia 
CorporativaCenários de 
Estresse (baixa 
probabilidade e 
alta severidade)
Projeção de capital 
em estresseAcompanha-
mento das 
discussõesPareceres NT
Participação em 
Grupos de 
TrabalhoCoordenação do 
tema alocação de 
capital e 
eficiênciaGestão da Estratégia - Direo Gestão de Riscos - DirisFormulação Estratégica
Formulação da RAS (Geric/Gecre)
Formulação de indicadores da Declaração de Apetite e Tolerância a 
Riscos (RAS)

=== Página 36 ===

 
Anexo 1 da DDC Diris 89044 , de 17.12.202 1. 
____________________________________________________________________________________ _ 
 
_____________________________________________________________________________________  
 
Estudo Técnico de Identificação de Riscos                                                                                                             36 #Confidencial  
5. Modelo de Identificação dos Riscos aplicado no CPBB  
 
No quadro seguinte, consta o resultado da revisão  de 2020  do Inventário de Riscos e do 
Conjunto Corporativo de Riscos Relevantes do Conglomerado Prudencial Banco do 
Brasil  (CPBB)38:  
Quadro 12. Inventário de Riscos X Riscos Relevantes  
 
Risco  Relevância  Categoria  Relevância  
Estratégia  Muito Alta  Sistêmico  Alta 
Conjuntura  Alta 
Corporativo  Muito Alta  
Crédito  Muito Alta  Tomador  Muito Alta  
Concentração de Crédito  Alta 
Contraparte  Média  
Convenente  Baixa  
País Baixa  
Soberano  Baixa  
Transferência  Baixa  
EFPPS  Muito Alta  Valor Justo dos Ativos  Baixa  
Passivo Atuarial  Baixa  
IRRBB  Muito Alta  Descasamento  Baixa  
Opcionalidades  Baixa  
Base  Baixa  
Mercado  Baixa  Cambial  Baixa  
Taxa de Juros da Carteira de Negociação  Baixa  
Preços de Mercadorias ( Commodities ) Baixa  
Preços de Ações da Carteira de Negociação  Baixa  
Liquidez  Média  Mercado  Média  
Fluxo de Caixa  Média  
Contágio  Média  Relac Entidades Não Consol  Média  
Fornecedor  Alta     
Socioambiental  Média  Social  Média  
Ambiental  Média  
Climático  Média  Físico  Média  
Transição  Média  
Responsabilidade  Média  
Legal  Média      
Conformidade  Média      
Operacional  Alta Práticas Inadequadas  Alta 
Problemas Trabalhistas  Média  
Fraudes e Roubos Externos  Muito Alta  
Falhas em Processos  Média  
Interrupção das Atividades  Média  
Danos a Ativos e Pessoas  Baixa  
Fraudes e Roubos Internos  Média  
Falhas e Sistemas e Tecnologia  Alta 
Conduta  Média  Trabalho e Demais Partes  Média  
Clientes e Usuários  Média  
TI Muito Alta      
Cibernético  Muito Alta      
Modelo  Média  Desenvolvimento  Baixa  
Implementação  Média  
Uso Média  
Reputação  Média  Percepção Mercado  Alta 
Percepção Órgão Regulador  Média  
 
 
38 NTE Diris 69536 , de 09.12.20 20, aprovada pelo CEGRC em 1 5.12.20 20. 

=== Página 37 ===

 
Anexo 1 da DDC Diris 89044 , de 17.12.202 1. 
____________________________________________________________________________________ _ 
 
_____________________________________________________________________________________  
 
Estudo Técnico de Identificação de Riscos                                                                                                             37 #Confidencial  
5.1 Identificação dos Riscos  
 
5.1.1  Direcionadores Estratégicos  
 
Diversos temas constituem -se em objeto de discussão na formulação da estratégia, 
sendo necessário avaliar os riscos que possam impactar o atingimento dos objetivos 
traçados, assim como possibilitar sua gestão proativa, viabilizando a realiza ção dos 
indicadores de negócios.  
 
Para essa análise, fo ram consideradas as informações geradas na discussão da revisão 
da estratégia 202 2-2026, relacionadas  à análise de mercados, publicações externas e 
da Gerência Inteligência de Mercado , que podem ser traduz idas nos fatores  (causas) 
dos riscos incorridos pelo Banco.  As informações foram utilizadas na avaliação 
qualitativa conforme modelo constante no item 4.3.2 . Os fatores oriundos da revisão da 
estratégia são vinculados aos fatores da taxonomia, não havendo necessidade de 
inclusão de novos fatores.  
 
5.1.2  Entidades Ligadas Banco do Brasil (ELBB)  
 
A estrutura unificada para gerenciamento de riscos do CPBB deve considerar os riscos 
associados ao Conglomerado e a cada instituição individualmente, bem como identificar 
e acompanhar os riscos associados às demais entidades controladas por seus 
integrante s ou das quais estes participem39. 
 
Com base no resultado parcial do ciclo de avaliação da gestão de riscos das ELBB 
202140, foram realizadas considerações sobre a aplicação dos riscos declarados pelas 
empresas no CPBB, conforme quadro seguinte:  
 
Quadro 13. ELBB X Taxonomia  (Atual)  
 
Risco  Qt empresas que 
declaram  Considerações sobre a aplicação do risco no 
CPBB  
Estratégia  19 risco relevante  
Governança  1 escopo da categoria "tomada de decisões" do risco 
de estratégia  
Crédito  19 risco relevante  
Atuarial  1 risco relevante  
RBAN  1 risco relevante  
Mercado  18 risco relevante  
Liquidez  18 risco relevante  
Solvência  1 escopo do risco de liquidez  
Contágio  4 risco relevante  
Subscrição  4 escopo do risco de contágio  
Fornecedores  1 risco relevante  
Continuidade do Negócio  1 considerado Impacto de Risco  
Socioambiental  18 risco relevante  
 
39 Resolução CMN nº 4.557, de 23.02.2017, Capítulo VI - Do Gerenciamento de Riscos e do Gerenciamento de Capital de 
Conglomerado Prudencial, Art. 53º.  
40 Disponibilizado pelo gestor corporativo do risco de contágio . 

=== Página 38 ===

 
Anexo 1 da DDC Diris 89044 , de 17.12.202 1. 
____________________________________________________________________________________ _ 
 
_____________________________________________________________________________________  
 
Estudo Técnico de Identificação de Riscos                                                                                                             38 #Confidencial  
Legal  9 risco relevante  
Contencioso Judicial  1 escopo do risco legal  
Compliance  18 risco relevante  
Regulatório  1 escopo do risco de co nformidade  
Operacional  19 risco relevante  
Lav Dinheiro, Financ Terrorismo e 
Corrupção  2 escopo do risco operacional  
Corrupção  2 escopo do risco de conduta  
Ético  1 escopo do risco de conduta  
Cibernético  6 risco relevante  
TI 3 risco relevante  
Segurança da Informação  3 escopo dos riscos de TI e segurança  
Modelo  4 risco relevante  
Reputação  18 risco relevante  
Emergente  1 termo utilizado para classificação de riscos  
Financeiro  1 termo utilizado para classificação de riscos  
 
Conforme demonstrado, o s riscos mapeados nas empresas  estão contemplados no 
escopo dos riscos do CPBB , não necessitando de novas inclusões .   
 
5.1.3  Formulário de Referência  
 
A revisão dos riscos considerou o Formulário de Referência, período -base 20 2141, 
apresentado à Comissão de Valores Mobiliários (CVM). O documento contempla, sob a 
ótica dos gestores específicos de riscos , com assessoria dos gestores  corporativos  de 
riscos, os fatores de risco considerados mais relevante s e que podem impactar os 
negócios e os resultados das operações do Banco.  
 
Os fatores de risco do Formulário de Referência são classificados em matéria s conforme  
norma CVM explanada no item 2.3 deste estudo. A Taxonomia Corporativa  de Riscos 
organiza os fatores de risco em quatro níveis hierárquicos, subdividindo -se até a 
condição geradora do risco.  Buscando integração das informações constantes nos dois 
instrumentos , foram avaliados os fatores do Formu lário de Referência e vinculados aos 
fatores  constantes na Taxonomia Corporativa de Riscos. Para isso, utilizou -se o tema 
principal do s fatores de risco  do Formulário e o s qualificador es do dicionário de fatores  
da Taxonomia , observando  a similaridade  e nível  de conteúdo. Nesta avaliação, não foi 
identificada necessidade de inclusão de novos fatores de risco na Taxonomia 
provenientes da revisão dos fatores de risco do Formulário de Referê ncia. O quadro 
seguinte demonstra as vinculações realizadas : 
 
 
 
 
 
 
 
 
41 NTE Diris 75515 , de 12.02.2021 , aprovada pelo CEGRC em 23.02.2021, NTE Diris 76369, de 14.04.2021, aprovada pelo 
CEGRC em 20.04.2021 e NTE Diris 79390, de 18.05.2021, aprovada pelo CEGRC em 20.05.2021 . 

=== Página 39 ===

 
Anexo 1 da DDC Diris 89044 , de 17.12.202 1. 
____________________________________________________________________________________ _ 
 
_____________________________________________________________________________________  
 
Estudo Técnico de Identificação de Riscos                                                                                                             39 #Confidencial  
Quadro 14. Fatores Formulário  X Taxonomia (Atu al) 
 
Formulário de Referência  Taxonomia Corporativa de Riscos  Qtd. Matéria CVM  Código¹  Tema  Qualificador do Dicionário de Fatores  
a) Banco do 
Brasil  FRA101  Aumento de provisões ou do passivo 
atuarial  Ambiente Regulatório/Legal Adverso  
16 FRA105  Falhas ou violações em processos críticos  Execução Inadequada dos Processos  
FRA106  Lavagem de dinheiro e financiamento do 
terrorismo  Inadequação de Conformidade ao Processo  
FRA107  Modelos utilizados no gerenciamento dos 
riscos  Desenho Inadequado do Processo  
FRA110  Corrupção no Brasil ou em qualquer outra 
jurisdição  Conduta Inadequada - Fraude em Benefício 
de Terceiros  
FRA111  Danos à reputação do Banco  Todos os qualificadores de todos os fatores  
FRA112  Incidentes de segurança cibernética  Atos Criminosos e de Vandalismo  
FRA113  Registros de fundos e programas 
governamentais  Execução Inadequada dos Processos  
FRA115  Lei Geral de Proteção de Dados (LGPD)  Ambiente Regulatório/Legal Adverso  
FRA116  Falhas na infraestrutura e sistemas de TI  Todos os qualificadores do fator Sistemas  
FRA117  Rebaixamento dos rating s do Banco  Ambiente Político/Social/Econômico Instável  
FRA118  Registro contábil de créditos tributários  Execução Inadequada dos Processos  
FRA119  Pandemia do Coronavírus (COVID -19) Fenômenos Inesperados da 
Natureza/Climáticos  
FRA123  Apólices de seguro contratadas pelo 
Banco  Desenho Inadequado do Processo  
FRA124  Direitos de propriedade intelectual  Desenho Inadequado do Processo  
FRA125  Queda adicional no rating de crédito do 
Brasil  Ambiente Político/Social/Econômico Instável  
b) Controlador  FRB203  Equalização do crédito rural pelo Tesouro 
Nacional  Decisões Adversas do Controlador/Acionista  
3 FRB204  Políticas adversas do acionista 
controlador  Decisões Adversas do Controlador/Acionista  
FRB205  Dividendos ou juros sobre o capital 
próprio  Decisões Adversas do Controlador/Acionista  
c) Acionistas  FRC301  Diluição da participação acionária  Decisões Adversas do Controlador/Acionista  1 
d) Controladas e 
Coligadas  FRD406  Participação em controladas e coligadas  Relacionamento com Entidades Não 
Consolidadas no C onglomerado Prudencial  
3 FRD409  Participações e constituição de parcerias  Atuação Insatisfatória de Fornecedores e 
Parceiros  
FRD410  Investimento realizado no Banco 
Patagônia  Relacionamento com Entidades Não 
Consolidadas no C onglomerado Prudencial  
e) Fornecedores  FRE501  Critérios na aquisição de bens e serviços  Atuação Insatisfatória de Fornecedores e 
Parceiros  2 
FRE502  Compartilhamento de canais  Atuação Insatisfatória de Fornecedores e 
Parceiros  
f) Clientes  FRF601  Garantias vinculadas a empréstimos e 
financiamento  Ambiente Regulatório/Legal Adverso  
10 FRF603  Cumprimento de obrigações da 
contraparte  Ambiente Político/Social/Econômico Instável  
FRF605   Perdas reais de crédito  Ambiente Político/Social/Econômico Instável  
FRF606  Retomada da economia e capacidade de 
pagamento  Ambiente Político/Social/Econômico Instável  
FRF607  Concentração da carteira de crédito  Ambiente Político/Social/Econômico Instável  
FRF608  Recuperação de empréstimos 
inadimplidos  Ambiente Regulatório/Legal Adverso  
FRF609  Consolidação dos imóveis garantidos por 
alienação  Ambiente Regulatório/Legal Adverso  
FRF610  Incorporação de imóveis ao patrimônio do 
Banco  Inadequação de Conformidade ao Processo  
FRF611  Qualidade da carteira de garantia de 
recebíveis  Ambiente Político/Social/Econômico Instável  
FRF613  Gestão de risco e análise de crédito  Desenho Inadequado do Processo  
g) Setores da 
economia nos 
quais o emissor 
atue FRG701  Condições econômicas domésticas ou 
globais  Ambiente Político/Social/Econômico Instável  
13 FRG702   Restrições de liquidez no Brasil  Ambiente Político/Social/Econômico Instável  
FRG703  Mudanças promovidas pelo Bacen na 
selic Ambiente Político/Social/Econômico Instável  
FRG704  Exposição à dívida do Governo Federal  Ambiente Político/Social/Econômico Instável  

=== Página 40 ===

 
Anexo 1 da DDC Diris 89044 , de 17.12.202 1. 
____________________________________________________________________________________ _ 
 
_____________________________________________________________________________________  
 
Estudo Técnico de Identificação de Riscos                                                                                                             40 #Confidencial  
FRG705  Crescimento do ambiente competitivo  Ambiente Concorrencial Altamente 
Competitivo  
FRG707  Instabilidade cambial  Ambiente Político/Social/Econômico Instável  
FRG709  Variações nas taxas de juros, câmbio, 
preços dos TVM  Ambiente Político/Social/Econômico Instável  
FRG710  Futura elevação substancial da inflação  Ambiente Político/Social/Econômico Instável  
FRG711  Eventos adversos com origem em outros 
países  Países Estrangeiros  
FRG712  Ações de controle governamentais  Ambiente Político/Social/Econômico Instável  
FRG714  Controle de capitais pelo governo 
brasileiro  Ambiente Político/Social/Econômico Instável  
FRG715  Concentração de fontes de captação de 
recursos  Ambiente Político/Social/Econômico Instável  
FRG716  Manutenção de nível de liquidez  Ambiente Político/Social/Econômico Instável  
h) Regulação dos 
setores em que o 
emissor atue  FRH804  Mudanças na legislação fiscal e 
previdenciária  Ambiente Regulatório/Legal Adverso  
11 FRH807  Exigibilidade para habitação e agricultura  Ambiente Regulatório/Legal Adverso  
FRH808  Controle acionário da empresa pela União  Ambiente Regulatório/Legal Adverso  
FRH812  Regulamentação das instituições 
financeiras  Ambiente Regulatório/Legal Adverso  
FRH814  Investimento de estrangeiros em bancos 
brasileiros  Ambiente Regulatório/Legal Adverso  
FRH815  Cobrança de crédito consignado  Ambiente Regulatório/Legal Adverso  
FRH817  Alterações legais/regulatórias  Ambiente Regulatório/Legal Adverso  
FRH819  Processos judiciais, administrativos ou 
arbitrais  Ambiente Regulatório/Legal Adverso  
FRH822  Implementação do Acordo da Basileia III  Ambiente Regulatório/Legal Adverso  
FRH823  Sistema Financeiro Aberto (Open 
Banking)  Ambiente Concorrencial Altamente 
Competitivo  
FRH825  Alterações regulatórias relevantes  Ambiente Regulatório/Legal Adverso  
i) Países 
estrangeiros 
onde o emissor 
atue FRI901  Mudanças na regulamentação bancária 
local  Países Estrangeiros  
3 FRI903  U.S. Dodd -Frank Wall Street Reform  Países Estrangeiros  
FRI904  Mudanças nos requerimentos de nível de 
capital  Países Estrangeiros  
j) Questões 
socioambientais  FRJ1001  Crédito e mudanças climáticas  Fenômenos Inesperados da 
Natureza/Climáticos  2 
FRJ1002  Crédito e impactos socioambientais  Fenômenos Inesperados da 
Natureza/Climáticos  
k) Ações 
Ordinárias ou 
ADSs  FRK1101 
a 
FRK1110  ADSs (American Depositary Share) do 
Banco  Decisões Adversas do Controlador/Acionista  9 
TOTAL  73 
¹Numeração sequencial definida conforme matéria dos fatores de risco para facilitar sua localização e atualização. O código n ão 
é publicado no formulário de referência, somente consta nos documentos internos.  
 
5.1.4  Base de Processos (BP)  
 
A Dicoi  assessora as áreas no mapeamento dos riscos nos processos , sempre que 
necessário, e em especial a partir dos temas priorizados definidos no seu Escopo de 
Atuação .  
 
Os riscos e controles são identificados e avaliados nos processos, ancorados no nível 3 
da Base de Processos (BP), e norteados pela Taxonomia Corporativa de Riscos e 
Dicionário de Controles Internos. Dessa forma, a Dicoi parte das categorias de risco 
definidas como relevantes para o Conglomerado, identifica o potencial de materialização 
desses riscos nos processos corporativos e avalia a efetividade dos controles 
implementados para sua mitigação.  O quadro a seguir demonstra o número de vezes 

=== Página 41 ===

 
Anexo 1 da DDC Diris 89044 , de 17.12.202 1. 
____________________________________________________________________________________ _ 
 
_____________________________________________________________________________________  
 
Estudo Técnico de Identificação de Riscos                                                                                                             41 #Confidencial  
que os riscos  relevantes  foram identifica dos nos diversos processos objeto do escopo 
anual Dicoi42:    
 
Quadro 15. Riscos BP  
 
Risco  Categoria  Qtd. 
Estratégia  Corporativo  94 
Fornecedor    18 
Legal    18 
Conformidade    157 
Operacional  Fraudes e Roubos Externos  8 
Falhas em Processos  117 
Danos a Ativos e Pessoas  7 
Falhas e Sistemas e Tecnologia  11 
Conduta  Trabalho e Demais Partes  101 
Clientes e Usuários  183 
Modelo    117 
Reputação  Percepção Mercado  5 
Percepção Órgão Regulador  3 
  
O quadro mostra a predominância de riscos não financeiros nos processos, podendo ser 
explicada pela recente inclusão dos demais riscos da Taxonomia Corporativa de Riscos 
na Biblioteca  de R iscos da BP . 
 
5.1.5  Diretrizes Regulatórias  
 
O Banco Central do Brasil (Bacen) aponta o conjunto mínimo de riscos a serem 
considerados pelos bancos43, cuja abrangência de aplicação é comparada à Taxonomia 
Corporativa de Riscos , conforme demonstrado no quadro seguinte:  
 
Quadro 16. Riscos Regulador X Taxonomia  (Atual)  
 
Conjunto de 
Riscos Bacen  Considerações em 
relação à Taxonomia  A definição do Bacen inclui:  Considerações em relação à 
Taxonomia  
Estratégia  risco relevante      
Crédito  risco relevante  Tomador  categoria do risco de crédito  
Concentração  categoria do risco de crédito  
Contraparte  categoria do risco de crédito  
País/Soberano  categorias do risco de crédito  
Transferência  categoria do risco de crédito  
Garantia Prestada  escopo das categorias de crédito  
Compromisso de Crédito  escopo das categorias de crédito  
Fundos de Pensão  escopo do risco 
relevante  Atuarial      
IRRBB  risco relevante      
Mercado  risco relevante  Cambial  categoria do risco de mercado  
 
42 Disponibilizado pel a Dicoi, referente a data -base outubro/ 2021. 
43  Resolução CMN nº 4.557, de 23.02.2017, Carta Circular Bacen nº 3.907, de  10.09.2018, Resolução CMN nº 4.595, de 28.08.2017, 
Resoluç ões CMN n° 4.943 e 4.945 , de 15.09.2021 . 

=== Página 42 ===

 
Anexo 1 da DDC Diris 89044 , de 17.12.202 1. 
____________________________________________________________________________________ _ 
 
_____________________________________________________________________________________  
 
Estudo Técnico de Identificação de Riscos                                                                                                             42 #Confidencial  
Taxas de Juros  categoria do risco de mercado  
Preços de Mercadorias 
(Commodities ) categoria do risco de mercado  
Preços de Ações  categoria do risco de mercado  
Liquidez  risco relevante  Mercado  categoria do risco de liquidez  
Fluxo de Caixa  categoria do risco de liquidez  
Contágio  risco relevante      
Seguros, 
Previdência e 
Capitalização  escopo do risco 
relevante Contágio      
Social, ambiental e 
climático  riscos relevantes 
Socioambiental e 
Climático      
Operacional  risco relevante  Legal  risco relevante  
Conformidade  risco relevante      
Reputação  risco relevante      
Fronteira  aspecto a ser 
considerado na gestão 
dos riscos      
 
Conforme observado, os riscos  do Bacen são considerados na Taxonomia Corporativa 
de Riscos  do Banco.   
 
Além do exposto nas normas, o supervisor bancário possui em sua agenda de trabalho 
a dimensão Competitividade , onde  busca a adequada precificação dos serviços/produtos 
bancários por meio de instrumentos de acesso competitivo aos mercados.  Para o Bacen, 
as diversas inovações, impulsionadas por tecnologia, incentivam a competição.  Tais 
ações podem resultar na necessidade de incremento de processo s de gestão de riscos 
relevantes . As ações  com impacto no Banco e riscos relacionados  constam no quadro 
seguinte : 
Quadro 17. Agenda Bacen Competitividade X Taxonomia (Atual)  
 
  Agenda Bacen Competitividade  
Risco  Categoria  Cadastro 
positivo  Pagamentos 
instantâneos 
(pix)  Open Banking  Risco 
Cibernético  
Contágio  Relacionamento 
Entidades não 
Consolidadas  
Fornecimento, 
pelas instituições 
financeiras e 
demais 
instituições 
autorizadas a 
funcionar pelo 
Banco Central do 
Brasil, das 
informações de 
adimplemento de 
pessoas naturais 
e de pessoas 
jurídicas, para 
formação de 
histórico de 
crédito.  O Sistema de 
Pagamentos 
Instantâneos 
(SPI) é a 
infraestrutura 
centralizada de 
liquidação bruta 
em tempo real de 
pagamentos 
instantâne os que 
resultam em 
transferências de 
fundos entre seus 
participantes 
titulares de Conta 
Pagamentos 
Instantâneos 
(Conta PI) no 
Banco Central do 
Brasil.  Implementação do 
Sistema 
Financeiro Aberto 
para 
compartilhamento 
padronizado de 
dados e serviços 
por meio  de 
abertura e 
integração de 
sistemas entre as 
instituições 
financeiras, 
instituições de 
pagamento e 
demais 
instituições 
autorizadas a 
funcionar pelo 
Banco Central do 
Brasil.  Implementação 
da Política de 
segurança 
cibernética e 
requisitos para 
a contrataçã o 
de serviços de 
processamento 
e 
armazenamento 
de dados e de 
computação em 
nuvem.  Fornecedor    
Legal    
Conformidade    
Operacional  Falhas em 
Negócios  
Fraudes e 
Roubos Externos  
Falhas em 
Processos  
Interrupção das 
Atividades  
Fraudes e 
Roubos Internos  
Falhas em 
Sistemas  
Conduta  Ambiente de 
Trabalho e 
Relacionamento 
Clientes/Usuários  
TI Falha em 
Sistemas ou 

=== Página 43 ===

 
Anexo 1 da DDC Diris 89044 , de 17.12.202 1. 
____________________________________________________________________________________ _ 
 
_____________________________________________________________________________________  
 
Estudo Técnico de Identificação de Riscos                                                                                                             43 #Confidencial  
Infraestrutura de 
TI 
Cibernético    
Modelo  Desenvolvimento, 
Implementação e 
Uso 
Reputação  Percepção de 
Mercado e de 
Órgãos 
Reguladores  
 
5.1.6  Preocupações do Mercado  
 
Nos processos de gestão dos riscos, há de se considerar as principais preocupações da 
indústria bancária. Nesse sentido, é apresentada  a pesquisa realizada  pela ORX44, com 
informações dos membros da organização  sobre as principais preocupações  ou fatores 
de risco emergentes  da indústria agrupadas por tema e classificadas em rating , conforme 
quadro a seguir:  
Quadro 18. Pesquisa ORX – Fatores de Risco Emergentes  
 
Tema  Rating  
1-insignificante a  
5-extremamente 
significante  Preocupações do setor  
Digitalização  3.9 • Ritmo da transformação digital  
• Escassez de habilidades  
• Mudança de expectativas de clientes  
• Crime cibernético avançado  
• Tecnologias emergentes  
• Gestão de dados  
• Inovação do concorrente e do me rcado  
• Sistemas legados  
• Estratégias digitais e mais amplas  
• Dependência de terceirização. Ex. provedores de nuvem  
• Capacidade de manter -se atualizado com as mudanças regulamentares  
Resiliência 
Operacional  3.8 • Conformidade regulatória  
• Dependência de sistemas de TI  
• Dependência de terceiros  
• Trabalho remoto  
• Digitalização rápida  
• Processo interconectado  
• Incerteza macroeconômica  
• Risco sistêmico  
• Capacidade de gestão de risco e recuperação de desastres  
• Eventos climáticos pertur badores  
• Ataques cibernéticos disruptivos  
Risco de 
Terceiros  3.7 • Maior dependência na terceirização  
• Segurança cibernética de terceiros  
• Falta de controle de terceiros  
• Mudança regulatória  
• Resiliência operacional de terceiros  
• Aumento e complexidade da cadeia de suprimentos  
• (Sistêmico) risco de concentração  
• Gestão de dados e privacidade  
 
44 ORX , Operational Risk Horizon 2021, Emerging Risk in Focus, Figure 13: Key risk themes  (adaptação).  

=== Página 44 ===

 
Anexo 1 da DDC Diris 89044 , de 17.12.202 1. 
____________________________________________________________________________________ _ 
 
_____________________________________________________________________________________  
 
Estudo Técnico de Identificação de Riscos                                                                                                             44 #Confidencial  
ESG e Risco 
Climático  3.6 • Aumento do foco regulatório  
• Pressão das partes interessadas  
• Crescente interesse público  
• Clima e eventos meteorológicos adversos  
• Impacto físico de longo prazo das mudanças climáticas  
• Incerteza geopolítica  
• Transição para economias de baixo carbono  
• Alto nível de incerteza  
• Alto nível de complexidade  
Risco de 
Concentração  3.2 • Volatilidade geopolítica  
• Dependência de terceiros incluindo provedores de nuvem  
• Mudança regulatória  
• Falha em diversificar base de fornecedores  
• Falha na escolha de fornecedor. Ex. tecnologias emergentes  
• Falta de disponibilidade do fornecedor em especial region almente  
• Complexidade da cadeia de suprimentos e aumento da terceirização  
• Falta de controle de terceiros  
Risco Sistêmico  3.1 • Maior regulamentação  
• Preocupações com a infraestrutura regional. Ex. suprimento de energia  
• Tensão geopolítica  
• Concentração de terceiros  
• Concentração da quarta parte  
• Complexidade da cadeia de suprimentos e falta de controle  
• Eventos climáticos perturbadores  
• Nível crescente de complexidade do financiamento  
 
Os fatores descritos pela ORX podem ser associados aos  seguintes qualificadores dos  
fatores  de risco  da Taxonomia Corporativa de Riscos,  não sendo identificados novos 
temas a serem incluídos no instrumento : 
 
Quadro 19. Fatores de Risco do Setor Financeiro X Taxonomia  (Atual)  
 
Qualificadores dos Fatores de Risco da Taxonomia  
• Ambiente Concorrencial Altamente Competitivo  
• Habilidades e Competências Insuficientes  
• Ambiente Cultural Transformado  
• Atos Criminosos e de Vandalismo  
• Segurança da Informação Deficiente  
• Desenvolvimento e Manutenção Inadequados de Aplicações  
• Definição Incorreta da Governança  
• Atuação Insatisfatória de Fornecedores e Parceiros  
• Ambiente Regulatório/Legal Adverso  
• Desenho  Inadequado do Processo  
• Ambiente Político/Social/Econômico Instável  
• Fenômenos Inesperados da Natureza/Climáticos  
• Estrutura Inadequada ou Insuficiente  
 
5.1.7  Benchmark  
 
A análise a seguir é realizada de forma a verificar se há algum risco declarado por outros 
bancos e não considerado pelo Banco, avaliando a pertinência de inclusão n a Taxonomia 
Corporativa de Riscos.  
 
Para obter o conjunto de riscos dos bancos , foram pesquisados os relatórios públicos de 
concorrentes no Brasil e maiores ban cos em ativos representantes de outros países . O 
quadro a seguir apresenta o levantamento de riscos declarados pelos bancos 
selecionados, que foram comparados aos riscos d a Taxonomia : 
 
 

=== Página 45 ===

 
Anexo 1 da DDC Diris 89044 , de 17.12.202 1. 
____________________________________________________________________________________ _ 
 
_____________________________________________________________________________________  
 
Estudo Técnico de Identificação de Riscos                                                                                                             45 #Confidencial  
Quadro 20. Riscos Bancos X Taxonomia  (Atual)  
 
 
 
Para o Bradesco, o risco de contágio é representado pela possibilidade de perda 
financeira decorrentes dos relacionamentos (contratuais ou não) da Organização com 
empresas controladas, coligadas, estruturas paralelas, controladoras, fundos de 
Caixa 
Rconômica 
FederalBanco 
Bradesco SABanco do 
Brasil SAItaú Unibanco 
Holding S.ABanco 
SantanderHSBC 
HoldingsBNP ParibasJ.P. M organ 
Chase & CoM itsubishi UFJ 
Finalcial GroupIndustrial & 
Commercial 
Bank of China
Brasil Brasil Brasil Brasil Espanha Reino Unido França EUA Japão China
Relatório de 
Gerenciament
o de Riscos - 
Pilar 3 - 1 T21Relatório de 
Gerenciament
o de Riscos - 
Pilar 3 - 2T21Formulário de 
Referência 
2021Relatório 
Gerenciament
o Riscos e 
Capital - Pilar 3 
- 2T21Estrutura 
Gerenciament
o Riscos e 
Capital - ago/21Relatório 
Estratégico 
2020Gestão 
integrada de 
riscos e capital 
- ago/21Reporte Anual 
2020Relatório 
Gerenciament
o Riscos e 
Capital - Pilar 3 
- 1 T21Estrutura 
Gerenciament
o Riscos e 
Capital - dez/20
Negócio/ 
Estratégia✓ ✓ ✓ ✓ ✓ ✓ ✓ 7
Crédito ✓ ✓ ✓ ✓ ✓ ✓ ✓ ✓ ✓ ✓ 10
Concentração ✓Considerado 
em CréditoCategoria de 
CréditoConsiderado 
em Crédito1
Contraparte ✓Considerado 
em CréditoCategoria de 
Crédito✓Considerado 
em CréditoConsiderado 
em Crédito✓ 3
País/ 
TransferênciaConsiderado 
em CréditoCategoria de 
Crédito✓ ✓ 2
Mercado ✓ ✓ ✓ ✓ ✓ ✓ ✓ ✓ ✓ ✓ 10
IRRBB ✓ ✓ ✓ ✓Considerado 
em M ercado✓ ✓ ✓ 7
Liquidez ✓ ✓ ✓ ✓ ✓ ✓ ✓ ✓ 8
Contágio ✓ ✓ ✓ 3
FornecedorConsiderado 
em Contágio✓Considerado 
em Contágio✓Considerado 
em Informação2
Atuarial ✓ ✓ 2
Seguros, 
Previdência e 
CapitalizaçãoEscopo 
Contágio✓ 1
Subscrição ✓Escopo 
ContágioConsiderado 
em Seg/Prev1
Fundos de 
pensãoConsiderado 
em SubscriçãoEscopo 
Atuarial0
Socioambiental ✓ ✓ ✓ ✓ ✓ ✓ ✓Considerado 
em 
Operacional✓ 8
ClimáticoConsiderado 
em Socio 
ambiental✓Considerado 
em Socio 
ambiental✓ 2
LegalConsiderado 
em 
Operacional✓Considerado 
em 
OperacionalConsiderado 
em 
Operacional✓Considerado 
em 
OperacionalConsiderado 
em 
Operacional✓ 3
Conformidade ✓ ✓ ✓ ✓ ✓ ✓Considerado 
em 
Operacional✓ 7
Operacional ✓ ✓ ✓ ✓ ✓ ✓ ✓ ✓ ✓ 9
Conduta ✓Considerado 
em 
Operacional✓ ✓Considerado 
em 
Operacional3
ResiliênciaEscopo 
Conduta✓ 1
InformaçãoEscopo 
Segurança ✓ 1
Cibernético ✓ ✓ ✓ ✓Considerado 
em Informação4
TI ✓ ✓ ✓Considerado 
em Informação✓ 4
Segurança ✓ ✓Considerado 
em Informação✓ 3
Continuidade 
de negóciosEscopo 
OperacionalConsiderado 
em Informação✓ 1
Lavagem de 
DinheiroEscopo 
Segurança ✓ ✓ 2
Modelo ✓ ✓ ✓ ✓ ✓Considerado 
em 
OperacionalConsiderado 
em 
Operacional5
Reputação ✓ ✓ ✓ ✓ ✓ ✓ ✓ ✓ 8
Total por 
Instituição14 13 19 14 10 14 7 4 9 14RiscoTotal 
por 
risco


=== Página 46 ===

 
Anexo 1 da DDC Diris 89044 , de 17.12.202 1. 
____________________________________________________________________________________ _ 
 
_____________________________________________________________________________________  
 
Estudo Técnico de Identificação de Riscos                                                                                                             46 #Confidencial  
investiment os, fundações, fornecedores e parceiros não consolidados no Conglomerado 
Prudencial.  
 
Faz parte do escopo da governança de risco de contágio do Itaú os públicos de Partes 
Relacionadas, composto principalmente por controladores, entidades controladas e 
coligadas, fundações, investimentos em entidades não consolidadas, fornecedores 
de produtos e serviços críticos, cessionárias, compradoras e vendedoras de ativos 
relevantes, terceiros com produtos distribuídos pelo Itaú Unibanco e terceiros para 
os quais o Ita ú Unibanco distribui produtos, além de toda a análise das Unidades 
Internacionais.  
 
A Caixa Econômica Fed eral trata o risco de contágio como sendo a possibilidade de 
ocorrência de perdas que demandem suporte financeiro para as entidades 
integrantes do Conglomerado, decorrentes de seus relacionamentos, contratuais ou 
não. O relacionamento pode ser estabelecido não somente por meio de participação 
de capital, mas por outros mecanismos, como parceria ou patrocínio, consultoria e 
assessoria, participação na gestão ou operação, relações comerciais ou financeiras.  
 
No Banco do Brasil, o risco de contágio é decorrente da possibilidade de perdas para 
as entidades integrantes do Conglomerado Prudencial, incluindo a instituição 
financeira líder, dec orrentes de seus relacionamentos (contratuais ou não) com 
entidades não consolidadas.  
 
Observa -se também que alguns riscos não financeiros são considerados no escopo do 
risco operacional em outros bancos.  
 
Não obstante  a forma de classificação dos riscos , verifica -se alinhamento entre o 
Taxonomia de Riscos  do Banco  e os riscos declarados pelos bancos selecionados.   
 
5.1.8  Revisão d a Taxonomia Corporativa de Riscos   
 
A partir das análises constantes nos itens anteriores, conclui -se que os riscos e fatores 
da Taxonomia Corporativa de Riscos est ão alinhad os aos identificados em processos 
internos e declarados pelo mercado .  
 
Para aprimoramento do instrumento, foram realizadas as seguintes alterações na 
Taxonomia:  
a) alteração do conceito e inclusão de categoria no risco de contágio, de forma a 
especificar a sua forma de materialização , buscando alinhamento com o BCBS45.  
A materi alização do risco se dá na ocorrência de suporte financeiro, de modo a se 
evitar possível impacto na reputação do Banco decorrente, por exemplo, de um  
cenário de insolvência regulatória . O risco de contágio é definido como a 
possibilidade de perdas para as  entidades integrantes do Conglomerado 
Prudencial Banco do Brasil (CPBB), incluindo a instituição financeira (IF) líder, 
 
45 Basel Committee on Banking Supervision,  Identification and management of step -in-risk, October 2017, p.4.  

=== Página 47 ===

 
Anexo 1 da DDC Diris 89044 , de 17.12.202 1. 
____________________________________________________________________________________ _ 
 
_____________________________________________________________________________________  
 
Estudo Técnico de Identificação de Riscos                                                                                                             47 #Confidencial  
decorrentes da prestação de suporte financeiro para entidades não consolidadas, 
com relacionamentos contratuais ou não . O risco passa a  ter duas categorias:   
i. relacionamentos contratuais ou societários: referem -se aos 
relacionamentos com pessoas jurídicas (entidades ou sociedades 
empresariais) nas quais o C PBB possui participação societária, ou 
atua como patrocinador ou administrador. São  classificadas como 
Entidade Ligadas ao Banco do Brasil (ELBBs) e as Simples 
Participação. As ELBBs são compostas pelas Subsidiárias Integrais, 
Controladas, Coligadas, Administradas, Patrocinadas e Fundações ; 
ii. demais relacionamentos: referem -se aos demais r elacionamentos 
do CP BB não aderente ao conceito registrado acima, como 
parcerias, fornecedores, fundos de investimento, entre outros . 
b) revisão  da denominação  do risco de fornecedor para risco de terceiros , de forma 
a manter alinhamento com o BCBS46 e os par es de mercado . O risco passa a ter 
escopo mais abrangente, tal qual o risco de contágio, abarcando, por exemplo, 
correspondentes bancários e parcerias,  porém com o objetivo de  se evitar perda 
operacional.  O risco de terceiros é definido como a possibilidade de perdas 
decorrentes do relacionamento do Banco com terceiros ; 
c) inclusão do risco social, ambiental e climático, subdivido nas  categorias social, 
ambiental e climático, de forma a demonstrar as evoluçõe s e os destaque s deste 
tema no Banco e nos anteciparmos às normas que aprimoram as regras de 
gerenciamento deste risco, que estarão em vigor a partir de 202247.  Assim, o s 
risco s socioambiental e climático passa m a compor o escopo do risco social, 
ambienta l e climático , sem prejuízo dos encaminhamentos  aprovados em nota 
técnica48.  O risco social, ambiental e climático é definido como a possibilidade de 
ocorrência de perdas para  a instituição ocasionadas por eventos associados aos 
seus temas. O referido risco é formado por três categorias  com os seguintes 
conceitos :   
i. risco social: possibilidade de ocorrência de perdas para a instituição 
ocasionadas por eventos associados à violação de direitos e 
garantias fundamentais ou a atos lesivos a interesse comum ; 
ii. risco ambiental: possibilidade de ocorrência de perdas para a 
instituição ocasionadas por eventos associados à degradação do 
meio ambiente, incluindo o uso excessivo de recursos naturais ; e 
iii. risco climático: em suas vertentes de risco de transição e de risco 
físico, como:  
• risco climático de transição: possibilidade de ocorrência de 
perdas para a instituição ocasionadas por eventos 
associados ao processo de transição para uma economia de 
baixo carbono, em que a emissão de gases do efeito estufa 
 
46 Basel Committee on Banking Supervision , Revisions to the Principles for the Sound Management of Operational  Risk, March  
2021, p.15.  
47 Resoluç ões CMN n° 4.943 e 4.945 , de 15.09.2021 . 
48 NTE Direo 85370, de 08.11.2021, aprovada pelo CA Direo em 10.11.2021.   

=== Página 48 ===

 
Anexo 1 da DDC Diris 89044 , de 17.12.202 1. 
____________________________________________________________________________________ _ 
 
_____________________________________________________________________________________  
 
Estudo Técnico de Identificação de Riscos                                                                                                             48 #Confidencial  
é reduzida ou compensada e os mecanismos naturais de 
captura desses gases são preservados; e  
• risco climático físico: possibilidade de ocorrência de perdas 
para a instituição ocasionadas por eventos associados a 
intempéries frequentes e severas ou a alterações ambientais 
de longo prazo,  que possam ser relacionadas a mudanças 
em padrões climáticos.  
d) revisão da forma de apresentação dos riscos que compõem a  natureza não 
financeira, conforme e studo do risco operacional: taxonomia e limites para a 
RAS49. O risco operacional passa a contemplar as categorias gerenciais de risco 
de terceiros, risco legal, risco de conformidade, risco de segurança, risco de 
modelo, risco de conduta, risco cibernético e risco de TI , com o objetivo de 
melhorar a eficiência na ges tão dos riscos não financeiros, tendo em vista a 
convergência de instrumentos de gestão como taxonomia e base de perdas . As 
categorias regulatórias do risco operacional (práticas inadequadas, práticas 
trabalhistas, fraudes e roubos externos, falhas de proc essos, interrupção das 
atividades, danos a ativos e pessoas, fraudes e roubos internos, falhas de 
sistemas e tecnologia) continuarão sendo monitoradas e reportadas. Está em 
andamento a  associação dos eventos de perdas das categorias gerenciais aos 
eventos de perdas das categorias regulatórias ;  
e) clarificação do escopo d o risco de segurança , definido como a p ossibilidade de 
perdas decorrentes de falhas nos processos de segurança do Banco, causando 
danos ao patrimônio, lesões pessoais ou apropriação de ativos físicos ou 
financeiros tanto do Banco quanto de terceiros . Essa definição inclui segurança 
física da informação;  
f) clarificação do escopo d o risco cibernético, definido como a p ossibilidade de 
perdas decorrentes de ataques cibernéticos contra a infraestrutur a de TI ou 
sistemas corporativos, afetando a integridade, confidencialidade e 
disponibilidade . Essa definição inclui segurança lógica da informação.  
g) tratamento dos 13 eventos remanescentes vinculados diretamente ao risco 
operacional, conforme segue:  
i. inclusão de 2 eventos no risco de terceiros;  
ii. inclusão de 2 eventos no risco de conduta;  
iii. inclusão de 2 eventos no risco de conformidade;  
iv. exclusão de 6 eventos em função da não existência de perda 
associada;  
v. exclusão de 1 evento em função da existência de event o semelhante 
no risco de terceiros.  
h) adequação dos níveis de riscos, de forma a padronizar  a estrutura da Taxonomia 
de Riscos Corporativ os – Dicionário de Eventos de Risco , sendo  os níveis 
apresentados  da seguinte forma:  
 
49 NTE Dir is 86329, de 20.10.2021, aprovada pelo CA Dir is em 10.11.2021.  

=== Página 49 ===

 
Anexo 1 da DDC Diris 89044 , de 17.12.202 1. 
____________________________________________________________________________________ _ 
 
_____________________________________________________________________________________  
 
Estudo Técnico de Identificação de Riscos                                                                                                             49 #Confidencial  
vi. Nível 1 (N1): risco;  
vii. Nível 2 (N2):  categoria  de risco ; 
viii. Nível 3 (N3): subcategoria  de risco ; 
ix. Nível 4 (N4): evento  de risco  ou, caso não haja , o campo é 
preenchido com “nihil.    
b) adequação dos níveis d e fatores de risco , de forma a padronizar  a estrutura da 
Taxonomia de Riscos Corporativ os – Dicionário de Fatores de Risco , sendo os 
níveis apresentados  da seguinte forma:  
i. Nível 1 (N1): fator de risco;  
ii. Nível 2 (N2):  categoria de fator de risco;  
iii. Nível 3 (N3): subfator de risco;  
Nível 4 (N4): tipo de fator de risco.  
c) adequação dos níveis de impacto de risco, de forma a padronizar a estrutura da 
Taxonomia de Riscos Corporativos  – Dicionário de Impactos , sendo os níveis 
apresentados da seguinte forma:  
i. Nível 1 (N1): impacto de risco;  
ii. Nível 2 (N2):  categoria de impacto de risco;  
iii. Nível 3 (N3): s ubcategoria de impacto de risco;  
 
A Taxonomia Corporativa de Riscos passa a apresentar a seguinte estrutura:   

=== Página 50 ===

 
Anexo 1 da DDC Diris 89044 , de 17.12.202 1. 
____________________________________________________________________________________ _ 
 
_____________________________________________________________________________________  
 
Estudo Técnico de Identificação de Riscos                                                                                                             50 #Confidencial  
 
Figura 11.  Taxonomia Corporativa de  Riscos  
Nível 1 (Fator) Nível 2 (Categoria do fator)Nível 3 
(Subfa-
tor)Nível 4 
(Tipo)Nível 1 (Risco) Nível 2 (Categoria do Risco)Nível 3 
(Subca-
tegoria)Nível 4 
(Evento)Nível 1
(Impacto)Nível 2 (Categoria)Nível 3 
(Subca-
tegoria)
Conduta Inadequada 5 23 Mudanças nos ambientes de negócio 7 0 Perda de Bens Móveis e Imóveis 3
Erro Humano - Falha em Serviço 2 16 Tomada de decisões estratégicas inadequadas 3 0 Multas e Multas Cominatórias (diárias) 3
Gestão Ambiente de Trabalho e Recursos Humanos 3 7 Execução da estratégia 5 0 Perda de Recursos e Valores 2
Modelagem Inadequada do Processo 4 19 Tomador 0 0 Perda de Valores ou Redução de Ativos 3
Gerenciamento Inadequado do Processo 2 11 Concentração de Crédito 2 7 Ressarcimento / Indenização 3
Comunicação Deficiente 2 4 Contraparte 0 0 Responsabilização Legal 3
Equipamentos, Infraestruturas de Redes e Comunicações 2 9 Convenente 0 0 Passivo Ambiental 2
Software Deficiente 2 9 País 0 0 Redução de Receitas Operacionais 2
Segurança Lógica Deficiente 1 4 Soberano 0 0 Redução Receitas de Intermediação Financeira 6
Eventos Imprevistos ou de Força Maior 2 9 Tranferência 0 0 Aumento de Despesas de Intermediação Financeira 3
Ambiente Externo Adverso 5 15 Valor Justo dos Ativos 2 0 Redução do Patrimônio Líquido 1
Agentes Externos 5 14 Passivo Atuarial 2 0 Perda Financeira 2
4 12 35 140 Descasamento 0 0 Dano à Imagem e Reputação 1
Opcionalidades 0 0 Evasão / Insatisfação de Clientes 3
Base 0 0 Fardo ou Ônus Operacional 3
Cambial 0 0 Descontinuidade dos Negócios 3
Taxa de Juros da Carteira de Negociação 0 0 Perda de Dados e Informações 2
Preços de Mercadorias 0 0 Sanções Administrativas 4
Preços de Ações da Carteira de Negociação 0 0 2 18 49
Mercado 0 0
Fluxo de Caixa 0 0
Relacionamentos societários 1 5 ,
Demais relacionamentos 5 9
Social 1 2
Ambiental 1 2
Climático 2 7
Terceiros 5 24
Legal 3 7
Conformidade 9 32
Modelo 6 12
Segurança 16 56
Conduta 39 99
Tecnologia da Informação 4 13
Cibernético 2 4
Percepção de Mercado 0 0
Percepção de Órgãos Reguladores 0 0
10 36 115 279Pessoas Estratégia
Processos
Sistemas
Eventos 
Externos AtuarialTaxonomia Corporativa de Riscos
Dicionário de Fatores de Risco Dicionário de Eventos de Risco Dicionário de Impactos de Rsico
Causa, externa ou interna, de um evento de risco.Ocorrência observada durante a realização de um objetivo definido, podendo causar impacto 
inesperado no resultado. Resultado de um evento de risco sobre os objetivos definidos.
Perdas 
IndiretasPerdas 
Diretas
ReputaçãoCrédito
Taxa de Juros da 
Carteira Bancária 
(IRRBB)
Mercado
Liquidez
Contágio
Risco Social, 
Ambiental e 
Climático
Operacional

=== Página 51 ===

 
Anexo 1 da DDC Diris 89044, de 17.12.2021.  
__________________________________________________________________________________________  
_________________________________________________________________________________________  
 
Estudo Técnico de Identificação de Riscos                                                                                                             51 #Confidencial  
5.2 Avaliação dos Riscos  
 
5.2.1  Critérios Quantitativos  
  
A tabela a seguir apresenta a avaliação quantitativa dos riscos a que o CP BB está exposto:  
 
Tabela 1. Resultado Quantitativ o 
 
Risco
   Categoria
Crédito 81,760
   Tomador 79,430
   Concentração de Crédito 1,260
   Contraparte 1,070
IRRBB 15,000
Mercado 2,000
   Cambial 0,900
   Taxa de Juros Cart. Neg. 0,974
   Preços de Mercadorias 0,125
   Preço de Ações Cart. Neg. 0,001
Socioal e Ambiental 0,110
Operacional 17,520
Atuarial 4,422Capital Econômico (CE) de R$ 5,31 bi, data-base 09/2021¹, dividido pelo 
CP de R$ 120,09 bi utilizado na projeção da RAS 2022.
Contágio 2,115Capital Regulatório (CR) de R$ 2,54 bi, data-base 09/2021¹, dividido pelo 
CP de R$ 120,09 bi utilizado na projeção da RAS 2022.Impacto – Quantitativo - Perdas Diretas
RAS - Limites 2022, baseados no Capital Principal (CP) de R$ 120,09 bi e 
Capital Nível 1 (CN1) de R$ 148,46, projetados para 01/2023.Modelo de Cálculo%CN1 - IRRBB
%CP - demais riscos
¹ Considerado o capital de 09/2021 como o projetado para 01/2023.
 
 
Escala Criticidade  Limite de Capital da RAS ou 
VaR sobre o Capital Principal  
Muito Baixo  X ≤ 1,0%  
Baixo  1,0% < X <= 3,0%  
Médio  3,0% < X <= 5,0%  
Alto 5,0% < X<= 10%  
Muito Alto  X > 10%  
 
Em relação  à avaliação realizada  no ano anterior, houve alteração da  relevância dos seguintes 
riscos /categorias :  
Tabela 2. Resultado Quantitativo Comparativo  
 
Impacto – Quantitativo - Perdas Diretas  
Risco  
2020  2021  
   Categoria  
Mercado      
   Cambial  1,20 0,90 
   Taxa de Juros Cart. Neg.  1,80 0,97 
Operacional  9,30 17,52  
Atuarial  10,50  4,42 
 

=== Página 52 ===

 
Anexo 1 da DDC Diris 89044, de 17.12.2021.  
__________________________________________________________________________________________  
_________________________________________________________________________________________  
 
Estudo Técnico de Identificação de Riscos                                                                                                             52 #Confidencial  
Os eventos e respectivas perdas registradas no Painel de Perdas Operacionais para a s 
categorias regulatórias do risco operacional serão  associad as aos eventos das  categorias 
gerenciais do risco operacional, de modo a se obter a exposição quantitativa para os riscos 
não financeiros. Dessa forma, nesta revisão não foi possível avaliar quan titativamente as 
categorias do risco operacional em função de necessidade de enquadramento mais assertivo 
das perdas. Quando o trabalho de vinculação das perdas for finalizado pelos gestores, poderá 
ser considerado na revisão deste estudo.  
 
Com relação ao risco de estratégia, em que pese a existência de indicador na RAS, este não 
tem como medida o v alor em risco  em relação ao capital principal, tal qual os demais riscos e 
metodologia aplicada a esta avaliação quantitativa. O indicador mede a variação entre o 
capital projetado e o capital observado, não sendo possível a utilização da régua estabelecida  
neste estudo .  
 
5.2.2  Critérios Qualitativos  
 
Aplicando o critério qualitativo  que demonstra a exposição dos riscos aos fatores mapeados 
na metodologia , observa -se o seguinte resultado:  
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 

=== Página 53 ===

 
Anexo 1 da DDC Diris 89044, de 17.12.2021.  
__________________________________________________________________________________________  
_________________________________________________________________________________________  
 
Estudo Técnico de Identificação de Riscos                                                                                                             53 #Confidencial  
Quadro 21. Resultado Qualitativo Fator  (Régua conforme Quadro 7, pág. 26)  
 
 
Ambientes de Negócios
Tomada de Decisões
Execução
Tomador
Concentração
Contraparte
Convenente
País
Soberano
Transferência
Valor Justo dos Ativos
Passivo Atuarial
Descasamento 
Opcionalidades
Base
Cambial
TJ Carteira Negociação
Preços de Mercadorias
Preços Ações Cart Neg
Mercado
Fluxo de Caixa
Relacionamentos 
Societários
Demais Relacionamentos
Social
Ambiental
Climático
Terceiros
Legal
Conformidade
Modelo
Segurança
Conduta
TI
Cibernético
Percepção Mercado
Percepção Órgão Regulador
Categoria 
N1Qualificador N3Exemplos de fatores que podem 
comprometer a ECBBPeso
Definição Incorreta da 
GovernançaMorosidade no processo de decisão e 
execução0,5 2,0 2,5 2,5 1,5 1,5 1,5 1,0 1,0 1,0 1,0 0,5 1,5 1,5 1,0 1,5 1,5 1,5 1,5 1,5 1,0 0,5 1,5 1,5 1,5 1,5 2,5 2,5 2,0 2,0 2,5 1,5 2,0 2,5 2,5 2,0 2,0 59
Desenho/Execução Inadequada 
do ProcessoOperação em silos das áreas, 
informações e soluções0,5 1,0 2,0 2,0 1,5 1,5 1,5 1,0 1,0 1,0 1,0 1,5 1,5 1,5 1,5 1,5 1,5 1,5 1,5 1,5 1,0 1,0 1,5 1,5 1,0 1,0 1,5 1,5 2,0 1,5 1,5 1,0 1,5 1,5 2,0 1,0 2,0 51
Estrutura Inadequada ou 
InsuficienteFoco em atividades operacionais e não 
estratégicas0,5 1,0 2,5 2,5 1,5 1,5 1,5 1,0 1,0 1,0 1,0 2,0 2,0 2,0 2,0 2,0 2,0 2,0 2,0 2,0 1,0 1,0 2,0 2,0 2,0 2,0 2,0 2,0 2,0 1,5 2,0 1,5 1,5 2,5 2,5 1,5 1,5 63
Relacionamento com Terceiros 
InadequadoIncapacidade operacional de atendimento 0,5 1,0 2,5 2,5 2,0 2,0 2,0 1,5 1,5 1,5 1,5 0,5 0,5 0,5 0,5 0,5 0,5 0,5 0,5 0,5 0,5 0,5 1,5 1,5 2,0 2,0 2,0 2,5 2,0 2,0 1,5 1,5 1,5 2,0 2,0 2,5 2,5 53
Conduta Inadequada, Antiética 
ou NegligenteAusência de mobilização ampla (senso de 
dono)1,0 2,0 5,0 5,0 4,0 3,0 3,0 3,0 2,0 2,0 2,0 1,0 1,0 1,0 1,0 1,0 1,0 1,0 1,0 1,0 1,0 1,0 3,0 3,0 4,0 4,0 4,0 2,0 4,0 4,0 3,0 3,0 5,0 3,0 3,0 4,0 4,0 95
Habilidades e Competências 
InsuficientesDesqualificação para o mindset  digital 1,0 3,0 4,0 4,0 4,0 3,0 3,0 3,0 3,0 3,0 3,0 2,0 2,0 2,0 2,0 2,0 2,0 2,0 2,0 2,0 2,0 2,0 2,0 2,0 3,0 3,0 3,0 3,0 3,0 2,0 5,0 3,0 5,0 5,0 5,0 3,0 3,0 105
Condições de Trabalho e 
Qualidade de Vida InadequadasFalta de flexibilidade, autonomia e 
compensação financeira1,0 3,0 3,0 4,0 4,0 3,0 3,0 3,0 3,0 3,0 3,0 4,0 4,0 3,0 3,0 3,0 3,0 3,0 3,0 3,0 3,0 3,0 2,0 2,0 4,0 3,0 3,0 3,0 3,0 2,0 4,0 3,0 5,0 4,0 4,0 3,0 3,0 115
Hardware, Rede e 
Telecomunicações DeficientesObsoletismo da base de TI 1,5 7,5 6,0 7,5 7,5 4,5 4,5 3,0 3,0 3,0 3,0 6,0 6,0 4,5 4,5 4,5 4,5 4,5 4,5 4,5 4,5 4,5 3,0 3,0 4,5 4,5 4,5 7,5 3,0 3,0 6,0 6,0 4,5 7,5 7,5 7,5 7,5 182
Desenvolvimento/Manutenção 
Inadequada de AplicaçõesInadequação das ferramentas de análises 
de dados1,5 4,5 4,5 7,5 7,5 6,0 6,0 4,5 4,5 4,5 4,5 6,0 6,0 4,5 4,5 4,5 4,5 4,5 4,5 4,5 4,5 4,5 4,5 4,5 4,5 4,5 4,5 4,5 4,5 4,5 7,5 6,0 4,5 7,5 7,5 7,5 7,5 191
Segurança Lógica DeficienteFragilidade dos mecanismos de 
segurança de dados1,5 4,5 6,0 6,0 6,0 4,5 4,5 3,0 3,0 3,0 3,0 4,5 4,5 4,5 4,5 4,5 4,5 4,5 4,5 4,5 4,5 4,5 6,0 6,0 4,5 4,5 4,5 7,5 4,5 4,5 4,5 6,0 4,5 7,5 7,5 7,5 7,5 180
Demanda por negócios relacionados à 
redução de emissões2,0 6,0 6,0 6,0 8,0 6,0 6,0 4,0 4,0 4,0 4,0 6,0 6,0 6,0 6,0 6,0 6,0 6,0 6,0 6,0 6,0 6,0 2,0 2,0 6,0 6,0 10,0 6,0 8,0 6,0 2,0 4,0 4,0 4,0 4,0 4,0 4,0 192
Surgimento de epidemias e variantes 2,0 6,0 6,0 6,0 8,0 6,0 6,0 4,0 4,0 4,0 4,0 6,0 6,0 4,0 4,0 4,0 4,0 4,0 4,0 4,0 4,0 4,0 2,0 2,0 6,0 6,0 6,0 6,0 8,0 8,0 2,0 6,0 10,0 8,0 8,0 4,0 4,0 188
Adaptação onerosa à reforma 
fiscal/tributária2,0 10,0 10,0 4,0 8,0 8,0 8,0 6,0 6,0 6,0 6,0 6,0 6,0 6,0 6,0 6,0 6,0 6,0 6,0 6,0 6,0 6,0 6,0 6,0 6,0 2,0 2,0 4,0 10,0 10,0 2,0 2,0 4,0 4,0 8,0 8,0 8,0 220
Aumento da pressão regulatória advinda 
de novas normas2,0 10,0 10,0 4,0 8,0 8,0 8,0 6,0 6,0 6,0 6,0 10,0 10,0 10,0 10,0 10,0 10,0 10,0 10,0 10,0 10,0 10,0 4,0 4,0 8,0 8,0 8,0 8,0 8,0 10,0 4,0 2,0 4,0 8,0 8,0 8,0 8,0 282
Aumento da inflação/taxas de juros (Selic) 2,0 10,0 10,0 4,0 8,0 8,0 8,0 6,0 6,0 6,0 6,0 10,0 10,0 10,0 10,0 10,0 10,0 10,0 10,0 10,0 10,0 10,0 4,0 4,0 6,0 2,0 2,0 6,0 6,0 4,0 2,0 4,0 4,0 4,0 4,0 6,0 6,0 246
Desvalorização do real 2,0 10,0 10,0 4,0 8,0 8,0 8,0 6,0 6,0 6,0 6,0 10,0 10,0 10,0 10,0 10,0 10,0 10,0 10,0 10,0 10,0 10,0 2,0 2,0 6,0 2,0 2,0 6,0 4,0 4,0 2,0 4,0 4,0 6,0 6,0 6,0 6,0 244
Introdução de transações com moedas 
digitais (Criptomoedas)2,0 10,0 10,0 4,0 8,0 8,0 8,0 6,0 6,0 6,0 6,0 2,0 2,0 2,0 2,0 2,0 2,0 2,0 2,0 2,0 6,0 6,0 2,0 2,0 6,0 2,0 2,0 6,0 6,0 6,0 2,0 6,0 4,0 6,0 6,0 6,0 6,0 170
Intensificação das relações virtuais ( Bots )2,0 6,0 6,0 6,0 6,0 4,0 4,0 4,0 4,0 4,0 4,0 2,0 2,0 2,0 2,0 2,0 2,0 2,0 2,0 2,0 2,0 2,0 4,0 4,0 6,0 6,0 2,0 10,0 6,0 4,0 6,0 8,0 8,0 6,0 8,0 6,0 6,0 160
Demanda maior por atendimento 
personalizado2,0 6,0 6,0 6,0 6,0 4,0 4,0 4,0 4,0 4,0 4,0 2,0 2,0 2,0 2,0 2,0 2,0 2,0 2,0 2,0 2,0 2,0 4,0 4,0 4,0 4,0 4,0 4,0 4,0 4,0 10,0 6,0 8,0 6,0 6,0 6,0 6,0 150
Aumento da influência digital de consumo 
(Finfluencers ) 2,0 6,0 6,0 6,0 6,0 4,0 4,0 4,0 4,0 4,0 4,0 2,0 2,0 2,0 2,0 2,0 2,0 2,0 2,0 2,0 2,0 2,0 2,0 2,0 2,0 2,0 2,0 6,0 4,0 2,0 2,0 6,0 4,0 2,0 4,0 6,0 6,0 122
Ampliação da prestação de serviços 
(consultoria financeira)2,0 6,0 6,0 6,0 6,0 4,0 4,0 4,0 4,0 4,0 4,0 2,0 2,0 2,0 2,0 2,0 2,0 2,0 2,0 2,0 2,0 2,0 4,0 4,0 2,0 2,0 6,0 2,0 4,0 4,0 8,0 6,0 6,0 2,0 2,0 6,0 6,0 134
Provimento maior de soluções de 
investimento 2,0 6,0 6,0 6,0 4,0 4,0 4,0 4,0 4,0 4,0 4,0 2,0 2,0 2,0 2,0 2,0 2,0 2,0 2,0 2,0 2,0 2,0 4,0 4,0 8,0 8,0 8,0 2,0 4,0 4,0 6,0 6,0 4,0 6,0 4,0 2,0 2,0 140
Isenção na cobrança de tarifas e taxas 2,0 8,0 8,0 4,0 6,0 4,0 4,0 4,0 2,0 2,0 2,0 2,0 2,0 2,0 2,0 2,0 2,0 2,0 2,0 2,0 2,0 2,0 6,0 6,0 2,0 2,0 2,0 4,0 4,0 4,0 2,0 2,0 4,0 2,0 2,0 6,0 6,0 120
Crescimento do PIX e carteiras digitais ( e-
Wallets )2,0 8,0 8,0 4,0 8,0 6,0 6,0 4,0 4,0 4,0 4,0 2,0 2,0 2,0 2,0 2,0 2,0 2,0 2,0 2,0 4,0 4,0 4,0 4,0 2,0 2,0 2,0 2,0 4,0 4,0 2,0 8,0 4,0 6,0 8,0 4,0 4,0 142
Compartilhamento de dados ( Open 
Banking )2,0 10,0 10,0 4,0 8,0 6,0 6,0 4,0 4,0 4,0 4,0 2,0 2,0 2,0 2,0 2,0 2,0 2,0 2,0 2,0 4,0 4,0 6,0 6,0 4,0 4,0 2,0 10,0 6,0 6,0 6,0 8,0 6,0 6,0 8,0 8,0 8,0 180
Uso de Plataforma/Ecossistema 
digital/ Marketplace2,0 8,0 8,0 4,0 8,0 6,0 6,0 4,0 2,0 2,0 2,0 2,0 2,0 2,0 2,0 2,0 2,0 2,0 2,0 2,0 4,0 4,0 6,0 6,0 4,0 4,0 2,0 10,0 4,0 4,0 6,0 6,0 4,0 6,0 8,0 8,0 8,0 162
Adesões a programas de recompensas 
(Cashback )2,0 8,0 8,0 4,0 8,0 6,0 6,0 4,0 2,0 2,0 2,0 2,0 2,0 2,0 2,0 2,0 2,0 2,0 2,0 2,0 2,0 2,0 4,0 4,0 2,0 2,0 2,0 8,0 4,0 4,0 2,0 6,0 4,0 2,0 2,0 2,0 2,0 122
Aumento da atuação de neobanks, 
fintechs e bigtechs 2,0 10,0 10,0 4,0 8,0 6,0 6,0 4,0 2,0 2,0 2,0 2,0 2,0 2,0 2,0 2,0 2,0 2,0 2,0 2,0 4,0 4,0 6,0 6,0 2,0 2,0 2,0 8,0 6,0 6,0 8,0 6,0 4,0 6,0 4,0 6,0 6,0 158
Ampliação da relação com 
Fornecedores/Prestadores2,0 6,0 8,0 4,0 4,0 4,0 4,0 4,0 2,0 2,0 2,0 6,0 6,0 6,0 6,0 6,0 6,0 6,0 6,0 6,0 2,0 2,0 8,0 8,0 8,0 8,0 8,0 10,0 6,0 6,0 6,0 6,0 6,0 8,0 8,0 8,0 8,0 210
Ampliação da relação com Startups 
(Agtechs, Fintech, Govtech)2,0 8,0 8,0 4,0 6,0 6,0 4,0 4,0 2,0 2,0 2,0 2,0 2,0 2,0 2,0 2,0 2,0 2,0 2,0 2,0 2,0 2,0 4,0 4,0 6,0 6,0 6,0 10,0 6,0 6,0 6,0 6,0 6,0 8,0 8,0 8,0 8,0 166
Decisões Adversas do 
Controlador/AcionistaIngerência governamental na atuação e 
negócios do Banco2,0 8,0 10,0 4,0 6,0 4,0 4,0 4,0 2,0 2,0 2,0 10,0 10,0 8,0 8,0 8,0 8,0 8,0 8,0 8,0 8,0 8,0 8,0 8,0 6,0 6,0 8,0 6,0 8,0 6,0 4,0 4,0 6,0 4,0 4,0 10,0 8,0 234
195,5 208,0 141,5 185,5 150,5 148,5 118,0 103,0 103,0 103,0 118,0 119,0 111,0 110,5 111,0 111,0 111,0 111,0 111,0 117,0 116,5 119,0 119,0 133,0 116,0 119,5 170,0 150,0 139,0 127,5 144,5 143,0 153,0 163,5 167,5 166,5
Fonte: Avaliação realizada pelo gestor dos riscos em segunda linha de defesa (Diris), data-base dezembro/2021.Fator de Risco - Qualitativo - Gestão Preventiva
Qual é o grau de exposição do risco à causa (fator) abaixo:
(1) Muito Baixo
(2) Baixo
(3) Médio
(4) Alto
(5) Muito Alto
Pessoas
Sistemas
Eventos 
ExternosFenômenos Inesperados da 
Natureza/Climáticos
Ambiente Regulatório/Legal 
Adverso
Ambiente 
Político/Social/Econômico 
Instável
Ambiente Cultural Transformado
Ambiente Concorrencial 
Altamente Competitivo
Atuação Insatisfatória de 
Fornecedores e ParceirosProcessosEstratégia Crédito Atuarial IRRBB Mercado
Total Categoria
Total Risco 181,7 130,2 118,5
Total Fator
122,8 148,8 167,0 110,8 111,0 116,8 119,0Liquidez Contági oSocial, Ambiental, 
ClimáticoOperaci onal Reputação

=== Página 54 ===

 
Anexo 1 da DDC Diris 89044, de 17.12.2021.  
__________________________________________________________________________________________  
_________________________________________________________________________________________  
 
Estudo Técnico de Identificação de Riscos                                                                                                             54 #Confidencial  
 
O quadro demonstra que fatores relacionados aos ambientes regulatório/legal e 
político/econômico/social,  podem ocasionar, em maior ou menor grau, todos os riscos. 
Também há destaque para fatores relacionados à atuação insatisfatória de fornecedores e 
parce iros, bem como decisões adversas do controlador/acionistas do Banco. O foco na gestão 
preventiva desses fatores pode diminuir a ocorrência dos riscos de forma geral.  
 
Com isso, é observado que aqueles riscos que têm grande probabilidade de estar associados 
a determinados fatores podem representar os riscos fronteiriços nesse grupo específico. A 
maior relação entre os riscos é percebida nos riscos não financeiros. Os fatores vincula dos à 
sistemas de TI , por exemplo, têm grande possibilidade de  causar os riscos de terceiros, 
segurança, TI e cibernético. Como a caracterização dos riscos decorre de seus fatores, 
eventos e impactos, conforme explicado no item 2.6 deste estudo, percebe -se uma fronteira 
que torna complexa a especificação dos limites de atuação na gestão de cada risco.   
 
Cada categoria de risco tem seus fatores mapeados e avaliados, au xiliando no direcionamento 
e eficácia da gestão. A média das categorias de cada risco representa o resultado da avaliação 
do risco. O s risco s que evidencia m maior exposição para o Banco, sob o critério avaliado, são 
o de estratégia,  crédito, operacional e reputação . 
 
Considerando que os fatores de risco do Formulário de Referência foram vinculados aos 
fatores da Taxonomia, conforme item 5.1.3 , a avaliação constante no quadro anterior pode ser 
utilizada para ordenamento dos fatores no Formulário, conforme relevância, em cada uma de 
suas matérias , quando houver revisão do documento . Dado que determinados fatores  podem  
apresenta r mesmo resultado de avaliação, pode ser  utilizado como critério de desempate a 
ponderação de seu conteúdo frente aos demais, levando em consideração o alcance na 
instituição, perspectiva futura de materializaçã o e necessidade de adaptações diante de novo 
cenário, conforme julgamento do avaliador.  Dessa forma, instrumentos e metodologias de 
avaliação são integrados, buscando uniformização dos resultados de avaliação dos riscos.  
Ainda, as informações discutidas na  formulação da ECBB podem ser utilizadas na formulação 
dos fatores . Dessa forma, temos retroalimentação e sinergia  dos documentos .  
 
Aplicando o critério qualitativo que demonstra as possíveis perdas  indiret as dos riscos, 
observa -se o seguinte resultado:  
 
 
 
 
 
 
 
 
 

=== Página 55 ===

 
Anexo 1 da DDC Diris 89044, de 17.12.2021.  
__________________________________________________________________________________________  
_________________________________________________________________________________________  
 
Estudo Técnico de Identificação de Riscos                                                                                                             55 #Confidencial  
Quadro 22. Resultado Qualitativo Perda  Indiret a (Régua conforme Quadro 8, pág. 28)  
 
 
O quadro demonstra maior possibilidade de impacto  relacionad o à evasão/insatisfação de clientes, danos à imagem e reputação e alcance 
da estratégia . O foco na gestão detec tiva desses impactos  visa diminuir sua reincidência . 
 
Da mesma forma que os fatores podem dar indícios de fronteiras entre os riscos, s uas perdas também demonstram quais formas de 
manifestação são coincidentes entre determinados grupos de risco. Como exemplo, os riscos de estratégia e reputação  têm considerável 
possibilidade de gerar evasão/insatisfação de clientes , sendo necessário conhecer também seus fatores e eventos para correta 
classificação dos riscos, estabelecendo os limites de suas fronteiras.  
 
As categoria s de risco  têm suas formas de perdas avaliadas e a média das categorias de cada risco representam o resultado  da avaliação 
do risco.  O risco que têm a maior exposição sob o critério avaliado é o de reputação . 
Ambientes de Negócios
Tomada de Decisões
Execução
Tomador
Concentração
Contraparte
Convenente
País
Soberano
Transferência
Valor Justo dos Ativos
Passivo Atuarial
Descasamento 
Opcionalidades
Base
Cambial
TJ Carteira Negociação
Preços de Mercadorias
Preços Ações Cart Neg
Mercado
Fluxo de Caixa
Relacionamentos Societários
Demais Relacionamentos
Social
Ambiental
Climático
Terceiros
Legal
Conformidade
Modelo
Segurança
Conduta
TI
Cibernético
Percepção Mercado
Percepção Órgão Regulador
Tipo N1 Forma N2 Peso
Sanções Administrativas 1,0 2,0 2,0 2,0 1,0 1,0 1,0 1,0 1,0 1,0 1,0 3,0 3,0 2,0 2,0 2,0 2,0 2,0 2,0 2,0 2,0 2,0 3,0 3,0 5,0 5,0 2,0 5,0 1,0 5,0 1,0 3,0 4,0 3,0 5,0 3,0 3,0 88
Perda de Dados e Informações 1,0 3,0 3,0 4,0 3,0 2,0 2,0 2,0 2,0 2,0 2,0 4,0 4,0 2,0 2,0 2,0 2,0 2,0 2,0 2,0 2,0 2,0 3,0 3,0 2,0 2,0 1,0 5,0 4,0 3,0 4,0 4,0 5,0 3,0 5,0 5,0 5,0 105
Descontinuidade dos Negócios 1,0 3,0 4,0 5,0 3,0 2,0 2,0 2,0 2,0 2,0 2,0 1,0 1,0 2,0 2,0 2,0 3,0 3,0 3,0 3,0 2,0 2,0 2,0 2,0 1,0 1,0 3,0 5,0 4,0 3,0 3,0 4,0 3,0 5,0 5,0 4,0 4,0 100
Fardo ou Ônus Operacional 1,0 3,0 3,0 3,0 1,0 1,0 1,0 1,0 1,0 1,0 1,0 3,0 3,0 2,0 2,0 2,0 2,0 2,0 2,0 2,0 2,0 2,0 0,0 0,0 3,0 3,0 3,0 4,0 3,0 3,0 3,0 3,0 3,0 4,0 3,0 2,0 2,0 79
Evasão / Insatisfação de Clientes 1,5 4,5 7,5 4,5 4,5 3,0 3,0 3,0 3,0 3,0 3,0 1,5 1,5 3,0 3,0 3,0 3,0 3,0 3,0 3,0 6,0 6,0 3,0 3,0 1,5 1,5 1,5 7,5 4,5 4,5 4,5 3,0 4,5 6,0 6,0 7,5 7,5 141
Dano à Imagem e Reputação 1,5 4,5 4,5 4,5 3,0 3,0 3,0 3,0 3,0 3,0 3,0 6,0 6,0 4,5 4,5 4,5 4,5 4,5 4,5 4,5 6,0 6,0 4,5 4,5 6,0 6,0 6,0 7,5 6,0 6,0 6,0 6,0 6,0 6,0 7,5 7,5 7,5 183
Alcance da Estratégia 2,0 10,0 10,0 10,0 8,0 8,0 6,0 6,0 6,0 6,0 6,0 8,0 8,0 6,0 6,0 6,0 6,0 6,0 6,0 6,0 6,0 6,0 0,0 0,0 6,0 6,0 8,010,0 6,0 6,0 6,0 6,0 6,0 6,0 6,0 6,0 6,0 230
30,0 34,0 33,0 23,5 20,0 18,0 18,0 18,0 18,0 18,0 26,5 26,5 21,5 21,5 21,5 22,5 22,5 22,5 22,5 26,0 26,0 15,5 15,5 24,5 24,5 24,5 44,0 28,5 30,5 27,5 29,0 31,5 33,0 37,5 35,0 35,0
Fonte: Avaliação realizada pelo gestor dos riscos em segunda linha de defesa (Diris), data-base dezembro/2021.Contági oSocial, 
Ambiental, 
ClimáticoOperaci onalReputa-
ção Perda Indireta - Qualitativo - Gestão 
Detectiva
Qual é o impacto do risco na Forma (N2) abaixo:
(1) Muito Baixo
(2) Baixo
(3) Médio
(4) Alto
(5) Muito AltoEstratégia Crédito Atuarial Liquidez
Total Perda Indireta
Total Categoria
Total Risco 26,0 15,5 24,5 32,7 35,0 32,3 19,1 26,5 21,5 22,5IRRBB Mercado
Perdas 
indiretas

=== Página 56 ===

 
Anexo 1 da DDC Diris 89044, de 17.12.2021.  
__________________________________________________________________________________________  
_________________________________________________________________________________________  
 
Estudo Técnico de Identificação de Riscos                                                                                                             56 #Confidencial  
 
Dessa forma , a integração do s Quadros 2 1 (Resultado Qualitativo Fator ) e 22 (Resultado 
Qualitativo Perda I ndireta)  apresenta  a avaliação final em termos qualitativos, conforme 
abaixo:  
Quadro 23. Resultado Impacto Qualitativo         
 Impacto - Qualitativo   
 Risco  Categoria    Total 
Categoria  Total 
Risco  
 
 
Estratégia  Ambientes de Negócios    225,5  
214,0   
 Tomada de Decisões    242,0   
 Execução    174,5   
 
Crédito  Tomador    209,0  
149,3   
 Concentração    170,5   
 Contraparte    166,5   
 Convenente    136,0   
 País   121,0   
 Soberano    121,0   
 Transferência    121,0   
 Atuarial  Valor Justo dos Ativos    144,5  
145,0   
 Passivo Atuarial    145,5   
 
IRRBB  Descasamento    132,5  
132,3   
 Opcionalidades    132,0   
 Base    132,5   
 
Mercado  Cambial    133,5  
133,5   
 TJ Carteira Negociação    133,5   
 Preços de Mercadorias    133,5   
 Preços Ações Cart Neg    133,5   
 Liquidez  Mercado    143,0  
142,8   
 Fluxo de Caixa    142,5   
 Contágio  Relacionamentos Societários    134,5  
134,5   
 Demais Relacionamentos    134,5   
 Social, Ambiental e 
Climático  Social    157,5  
147,3   
 Ambiental    140,5   
 Climático    144,0   
 
Operacional  Terceiros    214,0  
181,5   
 Legal    178,5   
 Conformidade    169,5   
 Modelo    155,0   
 Segurança    173,5   
 Conduta    174,5   
 TI   186,0   
 Cibernético    201,0   
 Reputação  Percepção Mercado    202,5  
202,0   
 Percepção Órgão Regulador    201,5          
 
Escala 
Criticidade  Impacto 
Qualitativo  
Muito Baixo  60,5 a 90,7  
Baixo  90,8 a 120,9  
Médio  121,0 a 151,2  
Alto 151,3 a 181,4  
Muito Alto  181,5 a 302,5  

=== Página 57 ===

 
Anexo 1 da DDC Diris 89044, de 17.12.2021.  
__________________________________________________________________________________________  
_________________________________________________________________________________________  
 
Estudo Técnico de Identificação de Riscos                                                                                                             57 #Confidencial  
 
Em relação ao ano anterior, houve alteração da relevância dos seguintes riscos /categorias , 
levando em conta as diferentes réguas estabelecidas em 2020 e 2021 para o critério 
qualitativo :  
Quadro 24. Resultado Impacto Qualitativo  Compara tivo 
 
Impacto - Qualitativo  
 2020  
 2021  
Risco  Categoria    Total 
Categoria  Total 
Risco  
 Total 
Categoria  Total 
Risco  
Estratégia  Ambientes de Negócios    65,0 
74,5  225,5  
214,0  Tomada de Decisões    93,5  242,0  
Execução    93,5  174,5  
Crédito  Tomador    59,5 
49,8  209,0  
149,3  Contraparte    53,5  166,5  
Convenente    45,5  136,0  
País   45,5  121,0  
Soberano    43,5  121,0  
Transferência    43,5  121,0  
Atuarial  Valor Justo dos Ativos    39,5 39,5  144,5  145,0  Passivo Atuarial    39,5  145,5  
IRRBB  Descasamento    38,0 
38,0  132,5  
132,3  Opcionalidades    38,0  132,0  
Base    38,0  132,5  
Mercado  Cambial    38,0 
38,0  133,5  
133,5 TJ Carteira Negociação    38,0  133,5  
Preços de Mercadorias    38,0  133,5  
Preços Ações Cart Neg    38,0  133,5 
Social, Ambiental 
e Climático  Social    55,0 51,7 
 157,5  147,3  
Operacional  Terceiros    63,0 
56,4  214,0  
181,5  Legal    55,5  178,5  
Conformidade    53,5  169,5  
Modelo    50,7  155,0  
Segurança    56,5  173,5  
Conduta    50,5  174,5  
TI   84,0  186,0  
Cibernético    88,5  201,0  
Reputação  Percepção Mercado    58,0 57,0  202,5  202,0  Percepção Órgão Regulador    56,0  201,5  
 
 
5.2.3  Matriz de Relevância dos Riscos  
 
A figura demonstra o resultado da avaliação da relevância dos riscos , plotada na matriz  
conforme critérios quantitativo e qualitativo :  

=== Página 58 ===

 
Anexo 1 da DDC Diris 89044, de 17.12.2021.  
__________________________________________________________________________________________  
_________________________________________________________________________________________  
 
Estudo Técnico de Identificação de Riscos                                                                                                             58 #Confidencial  
 
Figura 12.  Matriz de Relevância dos Riscos  
 
Os riscos de crédito, IRRBB, social, ambiental e climático tiveram sua  relevância  ajustada  em 
função  da ponderação  entre os critérios quantitativos e qualitativos . A relevância do risco 
atuarial diminuiu principalmente em decorrência do critério quantitativo. Os riscos de 
reputação e mercado aumentaram sua relevância em função do critério qualitativo. De modo 
geral, as categorias dos riscos alteraram sua classificação em decorrência d a avaliação 
qualitativ a. A comparação do resultado final da avaliação de relevância entre o ano anterior e  
o atual (apenas alterações ocorridas no período) consta no quadro seguinte:  
Muito Alto2. Crédito
4. IRRBB
4.1  Descasamento
4.2 Opcionalidades
4.3 Base
9. Operacional2.1  TomadorAlto Médio3. Atuarial
3.1  Vl Justo Ativos
3.2 Passivo AtuarialBaixo5. Mercado
7. Contágio
7.1  Rel Societários
7.2 Demais Rel2.2 Contraparte
2.3 Conc CréditoMuito Baixo5. Cambial
5.1  TJ Cart Neg
5.2 Preços Mercadorias
5.3 Preços Ações
8. Social, ambiental 
e climático
8.2 Ambiental8.1  SocialAv Final
Muito Baixo Baixo Médio Alto Muito AltoSem informação do 
Impacto Quantitativo 6. Liquidez
6.1  Mercado
6.2 Fluxo de Caixa
2.4 Convenente
2.5 País
2.6 Soberano
2.7 Transferência
8.3 Climático1 .3 Execução da 
Estratégia
9.2 Legal
9.3 Conformidade
9.4 Modelo
9.5 Segurança
9.6 Conduta1. Estratégia
1 .1  Ambientes de 
Negócios
1 .2Tomada de Decisões
9.1  Terceiros
9.7 TI
9.8 Cibernético
10. Reputação
1 0.1  Mercado
1 0.2 Regulador
Muito Baixo Baixo Médio Alto Muito AltoImpacto Quantitativo
Impacto Qualitativo
Impacto QualitativoMuito
Alta
Alta
Média
Baixa
Muito 
Baixa

=== Página 59 ===

 
Anexo 1 da DDC Diris 89044, de 17.12.2021.  
__________________________________________________________________________________________  
_________________________________________________________________________________________  
 
Estudo Técnico de Identificação de Riscos                                                                                                             59 #Confidencial  
 
Quadro 25. Resultado Relevância  Comparativo  
 
   2020  
 2021  
Risco  Categoria    Relevância 
Categoria  Relevância 
Risco   Relevância 
Categoria  Relevância 
Risco  
Estratégia  Ambientes de Negócios    Alta Muito Alta   Muito Alta  Muito Alta  Execução    Muito Alta   Alta 
Crédito  Concentração de Crédito    Alta 
Muito Alta   Média  
Alta Contraparte    Média   Alta 
Convenente    Baixa   Média  
País   Baixa   Média  
Soberano    Baixa   Média  
Transferência    Baixa   Média  
Atuarial  Valor Justo dos Ativos    Baixa  Muito Alta   Média  Média  Passivo Atuarial    Baixa   Média  
IRRBB  Descasamento    Baixa  
Muito Alta   Alta 
Alta Opcionalidades    Baixa   Alta 
Base    Baixa   Alta 
Mercado  Cambial    Baixa  
Baixa   Baixa  
Média  TJ Carteira Negociação    Baixa   Baixa  
Preços de Mercadorias    Baixa   Baixa  
Preços Ações Cart Neg    Baixa   Baixa  
Social, 
Ambiental e 
Climático  Ambiental    Média  Média  
 Baixa  Baixa  
Operacional  Terceiros    Alta 
Alta  Muito Alta  
Alta Legal    Média   Alta 
Conformidade    Média   Alta 
Modelo    Média   Alta 
Segurança    Média   Alta 
Conduta    Média   Alta 
TI   Muito Alta   Muito Alta  
Cibernético    Muito Alta   Muito Alta  
Reputação  Percepção Mercado    Alta Média   Muito Alta  Muito Alta  Percepção Órgão Regulador    Média   Muito Alta  
 
 
5.2.4  Análise dos resultados e  override  
 
Neste estudo, em decorrência d a categoria Tomador d o Risco de Crédito se destaca r no grau 
de relevância e volume de operações em relação às demais categorias, foi realizado override  
na avaliação final  do risco de crédito , passando da relevância Alta, segundo os critérios 
definidos neste estudo, para Muito Alta . Dessa forma, mantém -se a relevância estabelecida 
no ano anterior (Muito Alta) para o risco de crédito.  
 
 
 
 
 
 
 
 
 

=== Página 60 ===

 
Anexo 1 da DDC Diris 89044, de 17.12.2021.  
__________________________________________________________________________________________  
_________________________________________________________________________________________  
 
Estudo Técnico de Identificação de Riscos                                                                                                             60 #Confidencial  
5.2.5  Matriz de Relevância dos Riscos após override  
 
 
 
Figura 13.  Matriz de Relevância dos Riscos  após o verride  
 
 
 
 
 


=== Página 61 ===

 
Anexo 1 da DDC Diris 89044, de 17.12.2021.  
__________________________________________________________________________________________  
_________________________________________________________________________________________  
 
Estudo Técnico de Identificação de Riscos                                                                                                             61 #Confidencial  
 
5.2.6  Conjunto Corporativo de Riscos Relevantes  
 
Com base no resultado do estudo, o quadro abaixo apresenta a avaliação final da relevância 
dos riscos e categorias da Taxonomia Corporativa de Riscos do CPBB:  
 
Quadro 26. Conjunto Corporativo de Riscos Relevantes  
             
    Impacto - 
Quantitativo  
 Impacto - 
Qualitativo  
 Relevância Final  
 
 Risco  Categoria    Total 
Categoria  Total 
Risco  
 Total 
Categoria  Total 
Risco  
 Relevância 
Categoria  Relevância 
Risco   
 
Estratégia  Ambientes de Negócios      
   225,5  
214,0   Muito Alta  
Muito Alta   
 Tomada de Decisões       242,0   Muito Alta   
 Execução       174,5   Alta  
 
Crédito  Tomador    79,4 
81,8  209,0  
149,3   Muito Alta  
Muito Alta   
 Concentração de Crédito    1,3  170,5   Média   
 Contraparte    1,1  166,5   Média   
 Convenente       136,0   Média   
 País      121,0   Média   
 Soberano       121,0   Média   
 Transferência       121,0   Média   
 Atuarial  Valor Justo dos Ativos    4,4 4,4  144,5  145,0   Média  Média   
 Passivo Atuarial     145,5   Média   
 
IRRBB  Descasamento    
15,0 15,0  132,5  
132,3   Alta 
Alta  
 Opcionalidades     132,0   Alta  
 Base     132,5   Alta  
 
Mercado  Cambial    0,9 
2,0  133,5  
133,5   Baixa  
Média   
 TJ Carteira Negociação    1,0  133,5   Baixa   
 Preços de Mercadorias    0,1  133,5   Baixa   
 Preços Ações Cart Neg    0,0  133,5   Baixa   
 Liquidez  Mercado         143,0  142,8   Média  Média   
 Fluxo de Caixa       142,5   Média   
 Contágio  Relacionamentos Societários    2,1 2,1  134,5  134,5   Média  Média   
 Demais Relacionamentos     134,5   Média   
 Social, 
Ambiental e 
Climático  Social    0,1 0,1  157,5  
147,3   Média  
Baixa   
 Ambiental     140,5   Baixa   
 Climático         144,0   Média   
 
Operacional  Terceiros      
17,5  214,0  
181,5   Muito Alta  
Alta  
 Legal       178,5   Alta  
 Conformidade       169,5   Alta  
 Modelo       155,0   Alta  
 Segurança       173,5   Alta  
 Conduta       174,5   Alta  
 TI      186,0   Muito Alta   
 Cibernético       201,0   Muito Alta   
 Reputação  Percepção Mercado      
   202,5  
202,0   Muito Alta  
Muito Alta   
 Percepção Órgão Regulador       201,5   Muito Alta                
O Quadro 26 demonstra o resultado de impacto quantitativo para categorias de riscos que 
possuem esse cá lculo, sendo sua soma  o resultado do impacto total do risc o. Para  as demais 
categorias, repetiu -se o valor d o impacto quan titativo  total do risco , em função d e não haver 
cálculo individualizado por categoria , ainda que seus aspectos  sejam considerados  na 
mensuração do risco . 

=== Página 62 ===

 
Anexo 1 da DDC Diris 89044, de 17.12.2021.  
__________________________________________________________________________________________  
_________________________________________________________________________________________  
 
Estudo Técnico de Identificação de Riscos                                                                                                             62 #Confidencial  
 
5.2.7  Mapa dos Riscos  
 
 
Figura 14.  Mapa dos Riscos  
Do mapa de riscos, pode -se observar o seguinte:  
a) Todos os riscos podem impactar o alcance dos objetivos definidos na estratégia;  
b) A origem, ou fatores de risco, pode ser interna ou externa. Os fatores internos advêm 
da própria instituição, enquanto  os externos são decorrentes de agentes externos;  
c) O risco de estratégia possui categoria de gestão relacionada a decisões estratégicas e 
execução , em que fatores de risco internos (processos, pessoas, sistemas) podem 
ocasionar eventos de risco operacionais, fazendo com que a tomada de decisão seja 
baseada  em premissas inadequadas  ou haja falhas na execução da estratégia ; 
d) O risco de estratégia possui também a categoria ambiente de negócios , que gerenciam 
fatores externos, dentre os quais os advindos de setores da economia;  
e) Além dos fatores e eventos próprios da elaboração e acompanhamento da estratégia, 
sua materialização em processos e negócios do Banco traz riscos que podem impactar 
a instituição em diversos níveis;  
f) Questões socioambientais são fatores externos que dão origem ao s risco s social, 
ambi ental e  climático . Os risco s pode m trazer  mudanças para os negócios e operações 
do Banco, devendo ser considerado s seus impacto s nos demais riscos d a Taxonomia  
tais como o risco de crédito e o risco operacional, nas categorias legal, conformidade, 
TI; 
g) Os fatores de risco “Controladas e coligadas” são essencialmente advindos de relação 
negocial com as entidades ligadas, em função da participação do Banco no seu capital. 
Os fatores “Contratados e terceirizados” são oriundos de relação operacional 
(forneciment o de bens e prestação de serviços);   
Contratados e 
TerceirizadosPesso-
asTomador
Concentração
Contraparte
Convenente
País
Soberano
Transferência
Valor Justo dos Ativos
Passivo Atuarial
Descasamento 
Opcionalidades
Base
Cambial
TJ Carteira Negociação
Preços de Mercadorias
Preços Ações Cart Neg
Mercado
Fluxo de Caixa
Relacionamentos Contratuais 
ou Societários
Demais relacionamentos
Terceiros
Legal
Conformidade
Modelo
Segurança
Conduta
TI
CibernéticoAmbientes de Negócios Tomada de Decisões Execução da EstratégiaFinanceiraIRRBB MERCADO LIQUIDEZ CONTÁGIOSocial Ambiental Climático
OPERACIONAL
Legenda Relevância dos Riscos: 
Reputa-
cionalREPUTAÇÃO
Percepção do MercadoPercepção do Órgão 
ReguladorNatureza do Risco
Estra-
tégicaESTRATÉGIA
CRÉDITO AtuarialSOCIAL, AMBIENTAL E CLIMÁTICO
Não Financeira Questões 
Socio 
ambientaisObjetivos EstratégicosFatores de 
RiscoExternos Internos
ClientesPaíses 
Estran-
geirosSetores da EconomiaAmbiente 
Legal/ 
RegulatórioSiste-
masProces-
sosControladas e 
Coligadas
Muito Baixa Baixa Média Alta Muito Alta

=== Página 63 ===

 
Anexo 1 da DDC Diris 89044, de 17.12.2021.  
__________________________________________________________________________________________  
_________________________________________________________________________________________  
 
Estudo Técnico de Identificação de Riscos                                                                                                             63 #Confidencial  
h) Os fatores relacionados ao ambiente regulatório e legal originam do risco operacional, 
categoria de conformidade , no caso de a  norma ser proveniente dos órgãos 
reguladores da atividade bancária;  
i) O risco de conduta, categoria do  risco operacional , é oriundo de conduta inadequada 
do Banco, administradores, funcionários e colaboradores;  
j) O fator de risco sistemas o risco de TI, cat egoria do risco  operacional , é caracterizado 
pela inadequação das soluções e serviços de TI. Esse risco pode acarretar a categoria 
cibernético, que advém da violação de dados do Banco, em função de falhas nas 
soluções e serviços de TI;  
k) O risco de segurança , categoria do  risco operacional , está relacionado a o fator  
processos , diferentemente da categoria  de TI e cibernético  que estão associado s ao 
fator sistemas ;  
l) O risco de modelo , categoria do risco operacional, tem natureza essencialmente 
operacional em função do risco advir do desenvolvimento ou uso inadequado de 
modelos, utilizados nos processos de gestão de riscos do Banco;  
m) Os riscos de natureza financeira, mais diretamente relacionados à decisão de se expor 
ao risco para obter resultado s, são oriundos de agentes externos;  
n) Todos os riscos, se mal geridos, podem causar risco de reputação, afetando 
adversamente a rentabilidade dos negócios;  
o) Na gestão de cada risco, deve -se buscar identificar eventuais concentrações  de seus 
fatores de risco , considerar as fronteiras  com os demais riscos e avaliar os riscos 
residuais  ponderando o  processo de gestão  existente .  
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 

=== Página 64 ===

 
Anexo 1 da DDC Diris 89044, de 17.12.2021.  
__________________________________________________________________________________________  
_________________________________________________________________________________________  
 
Estudo Técnico de Identificação de Riscos                                                                                                             64 #Confidencial  
 
6. Bibliografia  
 
ASSOCIAÇÃO BRASILEIRA DE NORMA TÉCNICA. NBR ISO Guia 31000 , de 03/2018. 
Gestão de Riscos – Diretrizes.  
 
ASSOCIAÇÃO BRASILEIRA DE NORMA TÉCNICA. NBR ISO Guia 31000 , de 11/2009. 
Gestão de Riscos – Princípios e Diretrizes.  
 
BANCO CENTRAL DO BRASIL. Carta Circular Bacen nº 3.907, de 10.09.2018 . Divulga os 
modelos dos relatórios do Processo Interno de Avaliação da Adequação de Capital (Icaap) e 
do Processo Interno Simplificado de Avaliação da Adequação de Capital (IcaapSimp), de 
que trata a Circular nº 3.846, de 13 de setembro de 2017. Disponível em: 
https://www.bcb.gov.br/pre/normat ivos/busca/normativo.asp?tipo=Carta%20Circular&data=2
018&numero=3907  
 
BANCO CENTRAL DO BRASIL. Circular nº 4.027, de 12.06.2020 . Institui o Sistema de 
Pagamentos Instantâneos (SPI) e a Conta Pagamentos Instantâneos (Conta PI) e aprova 
seu regulamento. Disp onível em: 
https://www.bcb.gov.br/estabilidadefinanceira/exibenormativo?tipo=Circular&numero=4027  
 
BANCO CENTRAL DO BRASIL.  Gestão Integrada de Riscos, de 13.09.2017 . Disponível em: 
https://www.bcb.gov.br/content/publicacoes/outras_pub_alfa/Gestao -Integrada -de-Riscos.pdf  
 
BANCO CENTRAL DO BRASIL. Resolução CMN nº 4.557, de 23 .02.2017 . Dispõe sobre a 
estrutura de gerenciamento de riscos e a estrutura de gerenciamento de capital. Dispon ível 
em: 
https://www.bcb.gov.br/pre/normativos/busca/normativo.asp?tipo=Resolu%C3%A7%C3%A3
o&data=2017&numero=4557  
 
BANCO CENTRAL DO BRASIL. Resolução CMN nº 4.595, de 28 .08.2017 . Dispõe sobre a 
política de conformidade (compliance) das instituições finance iras e demais instituições 
autorizadas a funcionar pelo Banco Central do Brasil. Disponível em: 
https://www.bcb.gov.br/estabilidadefinanceira/exibenormativo?tipo=Resolu%C3%A7%C3%A
3o&numero=4595  
 
BANCO CENTRAL DO BRASIL. Resolução CMN nº 4.737, de 29 .07.2019 . Dispõe sobre o 
fornecimento das informações de adimplemento de pessoas naturais e de pessoas jurídicas 
aos gestores de bancos de dados. Disponível em: 
https://www.bcb.gov.br/estabilidadefinanceira/exibenormativo?tipo=Resolu%C3%A7%C3%A
3o&numero=4737  
 
BANCO CENTRAL DO BRASIL. Resolução CMN nº 4. 943, de 15.09.2021. Altera a 
Resolução nº 4.557, de 23 de fevereiro de 2017, que dispõe sobre a estrutura de 
gerenciamento de riscos, a estrutura de gerenciamento de capital e a política de divulgação 
de informaç ões. Disponível em: 
https://www.bcb.gov.br/estabilidadefinanceira/exibenormativo?tipo=Resolu%C3%A7%C3%A
3o%20CMN&numero=4943  
 

=== Página 65 ===

 
Anexo 1 da DDC Diris 89044, de 17.12.2021.  
__________________________________________________________________________________________  
_________________________________________________________________________________________  
 
Estudo Técnico de Identificação de Riscos                                                                                                             65 #Confidencial  
BANCO CENTRAL DO BRASIL. Resolução CMN nº 4. 945, de 15.09.2021. Dispõe sobre a 
Política de Responsabilidade Social, Ambiental e Climática (PRSAC) e sobre as ações com 
vistas à sua efetividade. Disponível em: 
https://www.bcb.gov.br/estabilidadefinanceira/exibenormativo?tipo=Resolu%C3%A7%C3%A
3o%20CMN&numero=4945  
 
BANCO CENTRAL DO BRASIL. Resolução Conjunta Bacen e CMN nº 1, de 04 .05.2020 . 
Dispõe sobre a implementação do Sistema Financeiro Aberto (Open Banking). Disponível 
em: 
https://www.bcb.gov.br/pre/normativos/busca/downloadNormativo.asp?arquivo=/Lists/Normat
ivos/Atta chments/51028/Res_Conj_0001_v1_O.pdf  
 
BANCO DO BRASIL. Formulário de Referência 2021 . Disponível em:  
https://ri.bb.com.br/publicacoes -e-comunicados/formularios -de-referencia/  
 
BANCO DO BRASIL. Nota Técnica Eletrônica Diris 6953 6, de 09.12.20 20. Estudo Técnico 
de Identificação de Riscos.  
 
BANCO DO BRASIL. Nota Técnica Eletrônica Diris 75515 , de 12.02.2021. Formulário de 
Referência - Atualização Seção 4, item 4.1 (fatores de risco e textos explicativos) . 
 
BANCO DO BRASIL. Nota Técnica Eletrônica Diris 76369 , de 14.04.2021. Formulário de 
Referência - Atualização Seção 4, item 4.1 (fatores de risco e textos explicativos) . 
 
BANCO DO BRASIL. Nota Técnica Eletrônica Diris 79390 , de 18.05.2021. Formulário de 
Referência - Atualiza ção Seção 4, item 4.1 (fatores de risco e textos explicativos) . 
 
BANCO DO BRASIL. Nota Técnica Eletrônica Dir eo 85370, de 08.11.2021. Atribuição à 
Diretoria Gestão de Riscos (Diris) como área responsável pela gestão específica dos  Riscos 
Social, Ambiental e Climático . 
 
BANCO DO BRASIL. Nota Técnica Eletrônica Diris  86329, de 20.10.2021. Risco 
operacional taxonomia e limites.  
 
BANCO DO BRASIL. Nota Técnica Eletrônica Diris 87121 , de 26.10.2021 . Revisão 
ordinária da Declaração de Apetite e Tolerânc ia a Riscos (RAS).  
 
BANCO DO BRASIL. UGP/Gepro/BPCC.  Guia Aplicado para Modelagem de Processos , 
de 02/2019. Disponível em: https://proc essos.intranet.bb.com.br   
 
Basel Committee on Banking Supervision.  Identification and management of step-in-risk. 
October  2017 . Disponível em https://www.bis.org/bcbs/publ/d423.pdf Bank for International 
Settlements. Issues in the Governance of Central Banks - A report from the Central 
Bank Governance Group , May 2009. Disponível em https://www.bis.org/publ/othp04.pdf  
 
Basel Committee on Banking Supervision. Revisions to the Principles for the Sound 
Management of Operational Risk . March 2021. Disponível em 
https://www.bis.org/bcbs/publ/d515.pdf  
 

=== Página 66 ===

 
Anexo 1 da DDC Diris 89044, de 17.12.2021.  
__________________________________________________________________________________________  
_________________________________________________________________________________________  
 
Estudo Técnico de Identificação de Riscos                                                                                                             66 #Confidencial  
BNP PARIBAS. Gestão Integrada de Riscos e Capital, agosto 2021 . Disponível em: 
https://brasil.bnpparibas/pt/informacoes -para-pessoas -fisicas -financeiras -e-de-risco s/gestao -
integrada -de-riscos -e-capital/  
 
BRADESCO. Relatório de Gerenciamento de Riscos – Pilar 3:  2º trimestre 20 21. 
Disponível em: https://www.bradescori.com.br/informacoes -ao-mercado/relatorios -e-
planilhas/gerenciamento -de-riscos/  
 
CAIXA ECONOMICA FEDERAL. Relatório de Gerenciamento de Riscos – Pilar 3:  1º 
trimestre 2021 . Disponível em: https://www.caixa.gov.br/sobre -a-caixa/relacoes -com-
investidores/gerenciamento -riscos/Paginas/default.aspx  
 
COMISSÃO DE VALORES MOBILIÁRIOS. Ofício Circular CVM/SEP/nº 0 2/2020. 
Orientações gerais sobre procedimentos a serem observados pelas companhias abertas, 
estrangeiras e incentivadas.  
 
ERNST&YOUNG.  Projeto de Gerenciamento de Capital e Icaap . Ponto de Homologação 
nº 15 de 10/2013.  
 
FABER, M.; MANSTETTEN, R.; PROOPS, J. Ecological Economics: Concepts and 
Methods . Cheltenham: Edward Elgar Publishing Ltd, 1996.  
 
HOPKIN, P.  Fundamentals of Risk Management . Kogan Page, 2014.  
BANCO DO BRASIL.  
 
HSBC. Relatório Estratégico 2020 . Disponível em: https://www.hsbc.com/investors/results -
and-announcements/annual -report  
 
INSTITUTO BRASILEIRO DE GOVERNANÇA CORPORATIVA. Guia de Orientação para 
Gerenciamento d e Riscos Corporativos , de 2007.  
 
ITAÚ. Gerenciamento de Riscos e Capital - Pilar 3: 2º trimestre 20 21. Disponível em: 
https://www.itau.com.br/relacoes -com-
investidores/Download.aspx?Arquivo=RKVvctLwRjExpGrhW5Z98A==&linguagem=pt  
 
JORION, P.  Value at Risk . The New Benchmark for Managing Financial Risk. , 3º Edição.  
 
JP MORGAN. Reporte Anual  2020. Disponível em:  
https://www.jpmorganchase.com/ir/annual -report  
 
OLIVER WYMAN. Relatório Técnico referente à identificação e avaliação da relevância 
dos riscos corporativos (relevantes e emergentes) . Ponto de Homologação nº 05 de 
06/2019.  
 
ORX. Operational Risk Horizon 202 1. Emerging Risk in Focus.  
 
SANTANDER. Relatório Estrutura de Gerenciamento de Riscos e Capital. Consulta em 
16/08/2021. Disponível em: https://www.santander.com.br/ri/gerenciamento -de-risco  
 

In [ ]:
# Lendo os dados da tabela
query = """
    SELECT cd_atvd, cd_rel_atvd, cd_tip_rel_atvd, cd_tip_est_rel, tx_rel_atvd,
           cd_usu_rsp_rel, cd_uor_rsp_rel, ts_vld_rel, db2_generated_rowid_for_lobs,
           nr_trabalho, tx_constatacao, rel_hash, rel_s_hash
    FROM ptai.com_resultado_analise;
"""

df = pd.read_sql(query, engine)

In [ ]:
df.shape

In [ ]:
# Verifica se a coluna 'tx_rel_atvd_sem_html' existe; caso contrário, cria
cursor.execute("""
    SELECT column_name FROM information_schema.columns
    WHERE table_schema = 'ptai' AND table_name = 'com_resultado_analise' AND column_name = 'tx_rel_atvd_sem_html';
""")
column_exists = cursor.fetchone()

if not column_exists:
    cursor.execute("""
        ALTER TABLE ptai.com_resultado_analise ADD COLUMN tx_rel_atvd_sem_html TEXT;
    """)
    conn.commit()

cursor.execute("""
    CREATE TABLE IF NOT EXISTS ptai.com_resultado_analise_imagens (
        nr_trabalho INTEGER,
        image_number INTEGER,
        image_data BYTEA,
        image_type TEXT,
        path_imagem TEXT
    );
""")
conn.commit()

In [ ]:
def process_row(row):
    html_content = row['tx_rel_atvd']
    nr_trabalho = row['nr_trabalho']

    # Verifica se o conteúdo HTML é válido
    if not isinstance(html_content, str):
        return '', []

    # Utiliza o BeautifulSoup para analisar o conteúdo HTML
    soup = BeautifulSoup(html_content, 'html.parser')

    # Extrai o texto puro, removendo todas as tags HTML
    text_content = soup.get_text(separator='\n')

    # Contador para nomear as imagens na ordem em que aparecem
    image_counter = 1

    images_list = []

    # Encontra todas as tags <img> no conteúdo HTML
    for img_tag in soup.find_all('img'):
        img_src = img_tag.get('src', '')
        # Verifica se o 'src' é um data URL com imagem em base64
        data_url_match = re.match(r'data:image/(\w+);base64,(.*)', img_src)
        if data_url_match:
            img_type = data_url_match.group(1)  # Obtém o tipo da imagem (jpeg, png, etc.)
            base64_data = data_url_match.group(2)  # Obtém os dados base64
            # Decodifica os dados base64 para bytes de imagem
            try:
                img_data = base64.b64decode(base64_data)
            except Exception as e:
                print(f"Erro ao decodificar imagem base64 para nr_trabalho {nr_trabalho}: {e}")
                continue
            # Define o nome e o caminho do arquivo de imagem
            image_filename = f"{nr_trabalho}_{image_counter}.{img_type}"
            image_path = os.path.join('imagem_relatorio', image_filename)
            # Salva a imagem no diretório especificado
            with open(image_path, 'wb') as f:
                f.write(img_data)
            # Armazena os dados da imagem na lista, incluindo o caminho da imagem
            images_list.append((nr_trabalho, image_counter, psycopg2.Binary(img_data), img_type, image_path))
            # Incrementa o contador de imagens
            image_counter += 1

    # Retorna o texto puro e a lista de imagens
    return text_content, images_list

In [ ]:
# Verifica se a pasta 'imagem_relatorio' existe; caso contrário, cria
if not os.path.exists('imagem_relatorio'):
    os.makedirs('imagem_relatorio')

In [ ]:
# Aplica a função a cada linha do DataFrame para criar a nova coluna 'tx_rel_atvd_sem_html' e coletar as imagens
result = df.apply(process_row, axis=1, result_type='expand')
df['tx_rel_atvd_sem_html'] = result[0]
df['images'] = result[1]

In [ ]:
# Coleta todas as imagens para inserir no banco de dados
images_to_insert = []
for images_list in df['images']:
    images_to_insert.extend(images_list)

In [ ]:
# Insere as imagens no banco de dados
if images_to_insert:
    sql_insert_images = """
        INSERT INTO ptai.com_resultado_analise_imagens (nr_trabalho, image_number, image_data, image_type, path_imagem)
        VALUES (%s, %s, %s, %s, %s);
    """
    cursor.executemany(sql_insert_images, images_to_insert)
    conn.commit()

In [ ]:
# Remove a coluna 'images' do DataFrame
df.drop('images', axis=1, inplace=True)

# Prepara os dados para atualizar a tabela original
data_to_update = list(zip(df['tx_rel_atvd_sem_html'], df['cd_atvd']))

In [ ]:
# Atualiza a tabela original com o novo texto
sql_update = """
    UPDATE ptai.com_resultado_analise
    SET tx_rel_atvd_sem_html = %s
    WHERE cd_atvd = %s;
"""

cursor.executemany(sql_update, data_to_update)
conn.commit()

In [ ]:
# Fecha o cursor e a conexão
cursor.close()
conn.close()

In [ ]:
# Reabrir a conexão ao banco de dados
conn, engine = postgres()
cursor = conn.cursor()

# Consulta para recuperar as imagens
query_images = """
    SELECT nr_trabalho, image_number, image_data, image_type
    FROM ptai.com_resultado_analise_imagens;
"""

df_images = pd.read_sql(query_images, conn)


In [ ]:
import matplotlib.pyplot as plt
from PIL import Image
from io import BytesIO

# Função para exibir uma imagem
def display_image(row):
    img_data = row['image_data']
    img_type = row['image_type']
    # Converter os dados binários em uma imagem PIL
    image = Image.open(BytesIO(img_data))
    # Exibir a imagem
    plt.imshow(image)
    plt.axis('off')
    plt.title(f"nr_trabalho: {row['nr_trabalho']}, image_number: {row['image_number']}")
    plt.show()

# Exibir todas as imagens
for index, row in df_images.iterrows():
    display_image(row)
